# DATA ORGANIZATION

In [5]:
from utils.io import gen_id_dated, dataframe_viewer, gen_geodf_geom, data_merger, data_validation, \
na_col_drop, na_line_drop, col_ren, dble_col_drop
import re, os
import numpy as np
import geopandas as gpd
import pandas as pd
import datetime as dtm
import matplotlib.pyplot as plt

Data format (excel files)

In [6]:
def compute_BH_length(df, length_col_name='Profondeur', top_col='Litho_top', base_col='Litho_base', verbose=False):
    for i in df.index:
        try:
            float(df.loc[i, top_col])
        except ValueError:
            df.loc[i, top_col] = np.nan

        try:
            float(df.loc[i, base_col])
        except ValueError:
            df.loc[i, base_col] = np.nan

    df[top_col] = df[top_col].astype('float64')
    df[base_col] = df[base_col].astype('float64')

    # compute length based on litho_top and litho_base
    id_list = []

    for i in df.index:
        id_ = df.loc[i,'ID']
        
        if verbose : print(i, id_, df.loc[i, top_col], df.loc[i, base_col])
        if id_ not in id_list:
            id_list.append(id_)
            if isinstance(id_, str):
                sql_id = f"{id_}"
            elif isinstance(id_, float) or isinstance(id_, int):
                sql_id = id_
                
            tmp = df[df['ID'] == sql_id]
            
            if verbose : print(len(tmp))
            #if len(tmp) > 0:
            df.loc[tmp.index, length_col_name] = float(max(tmp[base_col])) - float(min(tmp[top_col]))
    
    df.drop(index=df.query(f'{base_col}.isnull() and {top_col}.isnull()').index, inplace=True)
    df.insert(df.columns.to_list().index('ID')+1, length_col_name, df.pop(length_col_name))
    #df.reset_index(drop=True, inplace=True)
    

In [116]:
pol_field_model={'Arsenic': 'As', 'Cadmium': 'Cd', 'Chrome': 'Cr', 'Chrome VI': 'Cr_VI', 'Cuivre': 'Cu', 
'Mercure': 'Hg', 'Plomb': 'Pb', 'Nickel': 'Ni', 'Zinc': 'Zn', 'Cyanure(?:s)? (?libre(?:s)?)?': 'CN_libre', 
'Cyanures (totaux)': 'CN_tot','Cyanure (totaux)': 'CN_tot','CN_totaux':'CN_tot','Cyanures (APE)': 'CN_APE',
'cyanure (totaux)':'CN_tot', 'cyanure (APE)':'CN_APE', 'cyanure complex': 'CN_comp','Cyanure (APE)': 'CN_APE', 
"Cyanures (libres)  -  NEN-EN-ISO 14403": 'CN_libre', 'Cyanures (libres)':'CN_libre', 'thiocyanate': 'thioCN',
'Benzène': 'Bnz', 'Toluène': 'Toln', 'Éthylbenzène': 'EthylBnz', 'Orthoxylène': 'O-Xyl', 
'Para- et métaxylène': 'P-M-Xyl', 'Xylènes': 'Xyl', 'Styrène': 'Styr', 'BTEX totaux': 'BTEX_tot', 
'Phénol': 'Phenol', 'Indice phénol': 'Idc_Phenol', 'Naphtalène': 'Naphta', 'Acénaphtylène': 'Acenaphtyl', 
'Acénaphtène': 'Acenaphtn', 'Fluorène': 'Fluorene', 'Phénanthrène': 'Phenanthr', 'Anthracène': 'Anthrc', 
'Fluoranthène': 'Flranth', 'Pyrène': 'Pyr', 'Benzo(a)anthracène': 'Bnz(a)anthrc', 'Chrysène': 'Chrys', 
'Benzo(b)fluoranthène': 'Bnz(b)flranth', 'Benzo(k)fluoranthène': 'Bnz(k)flranth', 
'Benzo(a)pyrène': 'Bnz(a)pyr','Dibenzo(ah)anthracène': 'Dibnz(ah)anthrc',
'Benzo(ghi)pérylène': 'Bnz(ghi)peryl', 
'Indéno(1,2,3-cd)pyrène': 'Indeno(1,2,3-cd)pyr', 'HAP Totaux (16) - EPA': 'HAP_tot_EPA', 
'1,1-Dichloroéthane': '1,1-DCE', '1,2-Dichloroéthane': '1,2-DCE', '1,1-dichloroéthène': '1,1-DCEn', 
'Cis-1,2-dichloroéthène': 'Cis-1,2-DCEn', 'Trans 1,2-dichloroéthylène': 'Trans 1,2-DCEyl', 
'Dichlorométhane': 'DCM', 'Totaux (cis,trans) 1,2-dichloroéthènes': '(cis,trans) 1,2-DCE_tot', 
'1,2-dichloropropane': '1,2-DCP', 'Tétrachloroéthylène': 'TetraCEyn', 'Tétrachlorométhane': 'TCM', 
'1,1,1-Trichloroéthane': '1,1,1-TCE', '1,1,2-Trichloroéthane': '1,1,2-TCE', 'Trichloroéthylène': 'TCEyn', 
'Chloroforme': 'Chloroforme', 'Chlorure de vinyle': 'CVinyl', 'EOX': 'EOX', 
'fraction aromat. >C6-C7': 'Arom_C6C7', 'fraction aromat. >C7-C8': 'Arom_C7C8', 
'fraction aromat. >C8-C10': 'Arom_C8C10', 'fraction aliphat. C5-C6': 'Aliphat_C5C6', 
'fraction aliphat. >C6-C8': 'Aliphat_C6C8', 'fraction aliphat. >C8-C10': 'Aliphat_C8C10', 
'Fraction C5 - C8': 'Fract_C5C8', 'Fraction C8-C10': 'Fract_C8C10', 'Fraction C10-C12': 'Fract_C10C12', 
'Fraction C12-C16': 'Fract_C12C16', 'Fraction C16 - C21': 'Fract_C16C21', 'Fraction C21 - C35': 'Fract_C21C35', 
'Fraction C35 - C40': 'Fract_C35C40', 'Hydrocarbures totaux C10-C35': 'HC_tot_C10C35','C5-C8':'Fract_C5C8', 
'C8-C10':'Fract_C8C10','C10-C12':'Fract_C10C12','C12-C16':'Fract_C12C16','C16-C21':'Fract_C16C21', 
'C21-C35':'Fract_C21C35','C35-C40':'Fract_C35C40', 'totaux C10-C35':'HC_tot_C10C35','C12-C22':'Fract_C12C22', 
'C22-C30':'Fract_C22C30','C30-C40':'Fract_C30C40', 'Totaux C10-C40':'HC_tot_C10C40',
'Hydrocarbures totaux C10-C40':'HC_tot_C10C40', 'MTBE': 'MTBE', 'PCB 28': 'PCB_28', 'PCB 52': 'PCB_52', 
'PCB 101': 'PCB_101', 'PCB 118': 'PCB_118', 'PCB 138': 'PCB_138', 'PCB 153': 'PCB_153', 'PCB 180': 'PCB_180', 
'PCB totaux (7)?': 'PCB_tot', 'Chlorure(?:s)?': 'Chlorure', 'Soufre Total': 'S_tot', 'sulfite(?:s)?': 'sulfite', 
'sulfate(?:s)?': 'sulfate', 'COT':'COT','DBO (5 jours)':'DBO_5j','DCO':'DCO', 
'Ammonium':'NH4','ammoniaque libre':'NH3_libre','Nitrate':'HNO3', 'Nitrite':'HNO2','azote Kjeldahl':'N_Kjdl','sulfures totaux':'Sulfure_tot', 
'sulfure(?:s)? (libre(?:s)?)':'Sulfure_libre','calcium':'Ca','potassium':'K', 'magnésium':'Mg', 'manganèse':'Mn', 
'sodium':"Na", 'fer':'Fe','phosphore (total)':'P_tot','carbonate':'CaCO3', 'bicarbonate':'Bicarb','Phoshore':'P',
'fer ((Fe))? total':'Fe_tot', 'fer (2\+)':'Fe2','fluorure(?:s)?':'Fluorure','bromure (libre)':'B_libre'}


source_dfs initialization

In [8]:
_df = pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau = _df, _df, _df, _df
source_prv_sol, source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df, _df

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:0 ;
source_prv_sol:0 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


## 1- Profils sols et données forages.xls
* **Sheet : 'Données de forage'**

In [9]:
tmp_dir='../../CF_data/Result_traitem/profils_sols_donnees_forages/'
sheet='donnees_forage'

In [10]:
df = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', 
                   sheet_name='Données de forage')#, skiprows=2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df)

Rows : 25, columns : 15


interactive(children=(IntSlider(value=10, description='rows', max=25, min=10, readout=False), IntSlider(value=…

In [11]:
df.rename(columns={'Date':'Date_ouv','Profondeur':'Long_for', 'Méthode':'Method', 
                        'Diamètre forage':'Diam_for','Niv. Eau p/r sol':'Niv_eau_sol',
                        'PZ Prof.':'Long_pz', 'PZ Diamètre':'Diam_pz','PZ L.crépinée':'Long_crep', 
                        'Société forage':'Societe'}, inplace=True)

df=df[['ID', 'X', 'Y', 'Z', 'Date_ouv', 'Long_for', 'Diam_for', 'Long_pz', 'Diam_pz', 'Long_crep',
                 'Remarque','Niv_eau_sol','Method', 'Societe']]

In [12]:
df['Type'] = df['Long_pz'].apply(lambda x: 'Forage' if pd.isnull(x) else 'Piezo')
df['Refus'] = ''
df['Type_refus']=''

for i in range(len(df['Remarque'])):
    val = str(df.loc[i,'Remarque'])
    if re.search('[Bb]loqué', val) :
        df.loc[i,'Refus'] = 'x'
        
        if re.search('[lL]aitier', val):
            df.loc[i,'Type_refus'] = 'Laitier'
        elif re.search('[Bb]éton', val):
            df.loc[i,'Type_refus'] = 'Béton'
        elif re.search('[Mm]atériaux', val):
            df.loc[i,'Type_refus'] = 'Matériaux indurés' 
    else: 
        df.loc[i,'Refus'] = '' 

df['Diam_int_pz'] = df['Diam_pz'].apply(lambda x: pd.to_numeric(x.replace(' mm','').split('x')[1].strip(' m')) if not pd.isnull(x) else x)
df['Diam_ext_pz'] = df['Diam_pz'].apply(lambda x: pd.to_numeric(x.replace(' mm','').split('x')[0].strip(' m')) if not pd.isnull(x) else x)
df['Diam_for'] = df['Diam_for'].apply(lambda x: pd.to_numeric(x) if not pd.isnull(x) else x)

df.insert(7, 'Diam_ext_pz', df.pop('Diam_ext_pz')) # move to a specified position
df.insert(8, 'Diam_int_pz', df.pop('Diam_int_pz'))
df.drop(columns=['Remarque', 'Diam_pz'], axis=1, inplace=True)
df.drop(df.query("ID!=ID").index, inplace=True) # delete all ID='NaN' lines

gen_id_dated(df,'ID','Date_ouv')  

Generation of ID-dated...
Using column ' Date_ouv ' in the (geo)dataframe !
Process ended, check the (geo)dataframe


In [13]:
pz = df.query("Type=='Piezo'")
bh = df.query("Type!='Piezo'")

pz.reset_index(inplace=True, drop=True)
bh.reset_index(inplace=True, drop=True)

In [14]:
bh.drop(columns=['Diam_ext_pz', 'Diam_int_pz', 'Long_pz', 'Long_crep', 'Niv_eau_sol',], axis=1, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
dataframe_viewer(bh, rows=3), dataframe_viewer(pz, rows=3)

Rows : 13, columns : 13


interactive(children=(IntSlider(value=3, description='rows', max=13, min=3, readout=False), IntSlider(value=12…

Rows : 12, columns : 18


interactive(children=(IntSlider(value=3, description='rows', max=12, min=3, readout=False), IntSlider(value=12…

(None, None)

In [16]:
source_pz = pz
source_bh = bh

In [17]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False) #all Boreholes data in the source
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False) #all Piezometers data in the source
print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)}')

source_bh:13 ; source_pz:12


* **Sheet : 'Piézométrie'**

In [18]:
tmp_dir='../../CF_data/Result_traitem/profils_sols_donnees_forages/'
sheet='piezometrie'

In [19]:
df = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Piézométrie', skiprows=1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df)

Rows : 37, columns : 21


interactive(children=(IntSlider(value=10, description='rows', max=37, min=10, readout=False), IntSlider(value=…

In [20]:
df=na_col_drop(df, 3)
sdf=df[:11].copy()
sdf.reset_index(inplace=True, drop=True)
sdf.rename(columns={'z':'Z',}, inplace=True)
sdf=sdf[['ID', 'Z']]
sdf['Type']='Piezo'


Columns dropped :['Label', 'Commentaires ', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']



In [21]:
a=0
for x in df.columns:
    if pd.isnull(df.loc[16,x]):
        df.loc[16,x]='col'+str(a)
    a+=1

In [22]:
df.loc[16]=df.loc[16].apply(lambda x : x if not pd.isnull(x) else '')
df.columns = df.loc[16]
df=df[17:]
df.reset_index(inplace=True, drop=True)

#df.drop(columns=[df.columns.to_list()[x] for x in range(0,8)
#                      if re.compile(r"col|unnamed").match(df.columns.to_list()[x])], axis=1, inplace=True) 

In [23]:
dataframe_viewer(df)

Rows : 20, columns : 16


interactive(children=(IntSlider(value=10, description='rows', max=20, min=10, readout=False), IntSlider(value=…

In [24]:
df.rename(columns={'col3':'Date_prv', 'col4':'Nappe', 'col5':'ID', 'NP/piézo [m]':'Niv_eau_pz', 
                        'dim. piezo hors sol [m]':'haut_pz-sol', 'NP/sol [m]':'Niv_eau_sol', 
                        'Prof. piézo/piézo [m]':'Long_pz', 'Prof. piézo/sol [m]':'Long_pz-sol', 
                        'CE [mS/cm]':'CE','t° [°C]':'Temp', 'Observations':'Organo'}, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [25]:
df.insert(0, 'ID', df.pop('ID')) # move to first column
df.replace('-', np.nan, inplace=True)
df['CE']=df['CE [µS/cm]'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)
df.drop('CE [µS/cm]', axis=1, inplace=True)
df.loc[18, 'Niv_eau_pz']=np.nan
#df['Date_prv']=df['Date_prv'].apply(lambda x : str(x.year) if not pd.isnull(x) else '')

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

In [26]:
df.rename_axis(None, inplace=True, axis=1)
df.drop(df.query("ID!=ID").index, inplace=True) # supprimer les lignes avec ID='NaN'
df.reset_index(inplace=True, drop=True)
df=na_col_drop(df,2)
df['Type_mes'] = 'phys-chim'


Columns dropped :['col0', 'col1', 'col2']



In [27]:
cut_list = ['Nappe', 'haut_pz-sol', 'Long_pz', 'Long_pz-sol']
for x in df.columns:
    if x in cut_list:
        sdf[x] = df[x]
        
df.drop(columns=cut_list, inplace=True)

In [28]:
sdf['ID_2']=df['ID']

In [29]:
mes_pz=df # piezometry and phys-chem measures
pz=sdf # piezometers

In [30]:
source_mes_pz = mes_pz
source_pz, error_df = data_merger(source_pz, pz, how='outer', on='ID', )

Conflict values present. Please resolve this manually !


In [31]:
error_df

,Check_col,ID,Long_pz_x,Long_pz_y
1,Long_pz,F3M,3.3,2.98
7,Long_pz,F13M,3.5,4.04
9,Long_pz,F15bM,4.0,4.67
10,Long_pz,F16M,4.8,4.85
11,Long_pz,F17dM,3.6,3.97


In [32]:
data_validation(overall_data=source_pz, conflict_data=error_df, index_col='index', 
                valid_dict={'Long_pz_x':list(error_df.index)})

all conflicts have been fixed!


In [33]:
dataset = source_pz
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [34]:
dataframe_viewer(source_pz, rows=3)#, dataframe_viewer(error_df)

Rows : 12, columns : 22


interactive(children=(IntSlider(value=3, description='rows', max=12, min=3, readout=False), IntSlider(value=12…

In [35]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)

source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False) #all Boreholes data in the source
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False) #all Piezometers data in the source

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_mes_pz:{len(source_mes_pz)}')

source_bh:13 ; source_pz:12 ; source_mes_pz:17


* **Sheet : 'Equipement'**

In [36]:
tmp_dir='../../CF_data/Result_traitem/profils_sols_donnees_forages/'
sheet='Equipement'

In [37]:
df = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Equipement')#, skiprows=1)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df)

Rows : 36, columns : 7


interactive(children=(IntSlider(value=10, description='rows', max=36, min=10, readout=False), IntSlider(value=…

In [38]:
df.drop(columns=['Déplacement'], inplace=True)
name=['ID', 'Equip_top', 'Equip_base', 'Diam_for','Diam_ext_pz', 'Legende']
df=col_ren(df, mode=1, name=name)

In [39]:
compute_BH_length(df, top_col='Equip_top', base_col='Equip_base')
df.rename(columns={'Profondeur':'Long_equip'}, inplace=True)

In [40]:
coi = ['ID', 'Long_equip', 'Diam_for', 'Diam_ext_pz']
pz=df[coi].drop_duplicates(['ID'])
pz['Type'] = 'Piezo'

In [41]:
dataframe_viewer(df, un_val='ID', rows=3)

Rows : 36, columns : 7, Unique values on col 'ID': 12


interactive(children=(IntSlider(value=3, description='rows', max=36, min=3, readout=False), IntSlider(value=7,…

In [42]:
pz = pz
equip = df
source_equip = equip

In [43]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
equip.to_csv(tmp_dir+sheet+'_Equipments.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)

source_equip.to_csv(tmp_dir+'source_Equipments.csv', index=False) #all Piezometers data in the source
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False) #all Boreholes data in the source
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False) #all Piezometers data in the source

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_mes_pz:{len(source_mes_pz)} ; '
     f'source_equip:{len(source_equip)}')

source_bh:13 ; source_pz:12 ; source_mes_pz:17 ; source_equip:36


* **Sheets: 'Echantillon' + 'Organoleptique'**

In [44]:
tmp_dir='../../CF_data/Result_traitem/profils_sols_donnees_forages/'
sheet='Echant-organo'

In [45]:
df = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Echantillon')#, skiprows=1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df)

Rows : 29, columns : 4


interactive(children=(IntSlider(value=10, description='rows', max=29, min=10, readout=False), IntSlider(value=…

In [46]:
df.rename(columns={'De':'Ech_top', 'A':'Ech_base', 'Numéro':'ID_ech'}, inplace=True)

In [47]:
sdf = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Organoleptique')#, skiprows=1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)
dataframe_viewer(sdf)

Rows : 7, columns : 5


interactive(children=(IntSlider(value=7, description='rows', max=7, min=7, readout=False), IntSlider(value=5, …

In [48]:
sdf.rename(columns={'De':'Pol_top', 'A':'Pol_base'}, inplace=True)

In [49]:
mdf, error_df = data_merger(df, sdf, 'outer', 'ID')
mdf['Type_ech']='Sol'
mdf.insert(4, 'Type_ech', mdf.pop('Type_ech'))

In [50]:
dataframe_viewer(mdf, rows=3)

Rows : 32, columns : 9


interactive(children=(IntSlider(value=3, description='rows', max=32, min=3, readout=False), IntSlider(value=9,…

In [51]:
prv_sol = mdf
source_prv_sol=prv_sol

In [52]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'Source_Samples-soil.csv', index=False) #all Samples and organoleptic data in the source
print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_mes_pz:{len(source_mes_pz)} ; '
     f'source_prv_sol:{len(source_prv_sol)} ; source_equip:{len(source_equip)}')

source_bh:13 ; source_pz:12 ; source_mes_pz:17 ; source_prv_sol:32 ; source_equip:36


* **Sheet : 'Log'**

In [53]:
tmp_dir='../../CF_data/Result_traitem/profils_sols_donnees_forages/'
sheet='Log'

In [54]:
df = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Log')#, skiprows=1)
dataframe_viewer(df)

Rows : 55, columns : 5


interactive(children=(IntSlider(value=10, description='rows', max=55, min=10, readout=False), IntSlider(value=…

In [55]:
df.rename(columns={'De':'Litho_top', 'A':'Litho_base'}, inplace=True)

In [56]:
q=df.query('Keyword.str.contains(".ointe", regex=True)', engine='python').index
df.drop(q, inplace=True)
df.reset_index(drop=True, inplace=True)

In [57]:
compute_BH_length(df)

In [58]:
coi = ['ID', 'Profondeur']
bh=df[coi].drop_duplicates(['ID'])
bh['Type'] = 'Forage'

In [59]:
litho=df
source_litho=litho

In [60]:
source_bh, conflict_df = data_merger(source_bh, bh, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [61]:
dataset = source_bh
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [62]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
source_bh.to_csv(tmp_dir+'source_boreholes.csv', index=False)
source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False) #all lithologies or descriptions data in the source

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_mes_pz:{len(source_mes_pz)} ; '
     f'source_prv_sol:{len(source_prv_sol)} ; source_equip:{len(source_equip)} ; source_litho:{len(source_litho)} ;')

source_bh:25 ; source_pz:12 ; source_mes_pz:17 ; source_prv_sol:32 ; source_equip:36 ; source_litho:54 ;


### $\color{red}{\textbf{Excel sheets data merging}}$

In [63]:
excel_bhs, conflict_df = data_merger(source_bh, source_pz, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

Conflict values present. Please resolve this manually !


In [64]:
data_validation(overall_data=excel_bhs, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Type_y':list(conflict_df.index)})

all conflicts have been fixed!


In [65]:
dataset = excel_bhs
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [66]:
excel_bh_litho, conflict_df = data_merger(source_bh, source_litho, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [67]:
excel_bh_soil_sp, conflict_df = data_merger(source_bh, source_prv_sol, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [68]:
excel_bh_equip, conflict_df = data_merger(source_bh, source_equip, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [69]:
excel_bh_mes, conflict_df = data_merger(source_bh, source_mes_pz, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [70]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)

#### ------------------------------------------------------------------------------------------------------------------

## 2-Database MEMORIS3.xlsx
* **Sheet : 'PROFILS_SOL'**

In [67]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_bh, source_pz, source_litho, source_prv_sol, source_mes_pz, source_equip= _df, _df, _df, _df, _df, _df

In [68]:
tmp_dir='../../CF_data/Result_traitem/database_Memoris3/'
sheet='Profils_sol'

In [69]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. '+
                        'Siterem - 2017/Database MEMORIS3.xlsx', sheet_name='PROFILS_SOL')#, skiprows=2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=3)

Rows : 2041, columns : 16


interactive(children=(IntSlider(value=3, description='rows', max=2041, min=3, readout=False), IntSlider(value=…

In [70]:
df = df[['Date', 'N°', 'Id', 'Profondeur', 'Description', 'Piézo', 'Unnamed: 6',
                             'Gouge Ø75', 'MFT Ø145', 'carottier', 'tarrière', 'Liner Ø60']] 

In [71]:
df.rename({'Date':'Date_ouv', 'N°':'Ref', 'Id':'idx', 'Piézo':'Type', 'Unnamed: 6':'Societe',
                'MFT Ø145':'MFT_145', 'Gouge Ø75':'Gouge_75', 'Liner Ø60': 'Liner_60'}, axis=1, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [72]:
print(list(set(df['Date_ouv'].apply(lambda x: x.year if not pd.isnull(x) else x))))

[NaT, 2009, 2010, 2015]


In [73]:
df.loc[df.fillna('').query("Societe.str.contains('x|X')").index, 'Type']='X'

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [74]:
df.loc[df.fillna('').query("Gouge_75.str.contains('SBS|SITER')").index, 'Societe']='SBS Environnement'
df.loc[df.fillna('').query("Gouge_75.str.contains('SBS|SITER')").index, 'Gouge_75']=''

In [75]:
for i in range(len(df['Date_ouv'])-1):
    if not pd.isnull(df.loc[i, 'Date_ouv']) and pd.isnull(df.loc[i+1, 'Date_ouv']):
        df.loc[i+1, 'Date_ouv']=df.loc[i, 'Date_ouv']
        
    if not pd.isnull(df.loc[i, 'Societe']) and pd.isnull(df.loc[i+1, 'Societe']):
        df.loc[i+1, 'Societe']=df.loc[i, 'Societe']
        
    if not pd.isnull(df.loc[i, 'Type']) and pd.isnull(df.loc[i+1, 'Type']) and \
       df.loc[i, 'Ref']==df.loc[i+1, 'Ref']:
        df.loc[i+1, 'Type']=df.loc[i, 'Type']

In [76]:
for i in range(len(df['idx'])-1):    
    if df.loc[i,'Ref']==df.loc[i+1,'Ref'] and not pd.isnull(df.loc[i, 'Profondeur'])\
    and re.findall('Forage',df.loc[i, 'Profondeur']):
        df.loc[i+1,'idx']=df.loc[i, 'Profondeur'][0]+str(df.loc[i, 'Ref'])
        w=df.loc[i, 'Profondeur'][0]
    elif df.loc[i,'Ref']==df.loc[i+1,'Ref'] and not pd.isnull(df.loc[i, 'Profondeur']):
        df.loc[i+1,'idx']=w+str(df.loc[i, 'Ref'])
    
    if df.loc[i,'Ref']==df.loc[i+1,'Ref'] and not pd.isnull(df.loc[i, 'Profondeur'])\
    and re.findall('Tranch',df.loc[i, 'Profondeur']):
        df.loc[i+1,'idx']=df.loc[i, 'Profondeur'][0]+str(df.loc[i, 'Ref'])
        w=df.loc[i, 'Profondeur'][0]
    elif df.loc[i,'Ref']==df.loc[i+1,'Ref'] and not pd.isnull(df.loc[i, 'Profondeur']):
        df.loc[i+1,'idx']=w+str(df.loc[i, 'Ref'])
     
   # if df.loc[i,'Ref']==df.loc[i+1,'Ref'] and not pd.isnull(df.loc[i, 'Profondeur'])\
   # and re.findall('Moni',df.loc[i, 'Profondeur']):
   #     df.loc[i+1,'idx']=df.loc[i, 'Profondeur'][0]+str(df.loc[i, 'Ref'])
   #     w=df.loc[i, 'Profondeur'][0]
   # elif df.loc[i,'Ref']==df.loc[i+1,'Ref'] and not pd.isnull(df.loc[i, 'Profondeur']):
   #     df.loc[i+1,'idx']=w+str(df.loc[i, 'Ref'])

In [77]:
df['Ref']=df['idx'].apply(lambda x : x if re.findall('F|T', str(x)) else '')
df['Ref']=df['idx'].apply(lambda x : x.replace('Monito ', 'Mon') if re.findall('Monit', str(x)) else x)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [78]:
df['Type']=df['Type'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')

In [79]:
df['Ref']=df['Ref'].apply(lambda x: re.sub("\.1","a",str(x)) if re.search(r"\.1", str(x)) else x)
df['Ref']=df['Ref'].apply(lambda x: re.sub("\.2","b",str(x)) if re.search(r"\.2", str(x)) else x)
df['Ref']=df['Ref'].apply(lambda x: re.sub("\.3","c",str(x)) if re.search(r"\.3", str(x)) else x)
df['Ref']=df['Ref'].apply(lambda x: re.sub("\.4","d",str(x)) if re.search(r"\.4", str(x)) else x)

In [80]:
gen_id_dated(df, ref_col='Ref', date_col='Date_ouv')

Generation of ID-dated...
Using column ' Date_ouv ' in the (geo)dataframe !
Process ended, check the (geo)dataframe


In [81]:
df.loc[df.query('Profondeur!=Profondeur' ).index,'Profondeur']=''

In [82]:
df['Method']=''
            
for i in range(len(df['Method'])):
    if not pd.isnull(df.loc[i, 'Gouge_75']) : df.loc[i, 'Method']='Gouge_75'
    if not pd.isnull(df.loc[i, 'MFT_145']) : df.loc[i, 'Method']='MFT_145'
    if not pd.isnull(df.loc[i, 'Liner_60']) : df.loc[i, 'Method']='Liner_60'
    if not pd.isnull(df.loc[i, 'carottier']) : df.loc[i, 'Method']='carrotier'
    if not pd.isnull(df.loc[i, 'tarrière']) : df.loc[i, 'Method']='tarrière'

In [83]:
df.drop(df.query('Profondeur.str.contains("Forage") and Profondeur!="Forage bloqué"', engine='python').index, inplace=True)
df.drop(df.query('Profondeur.str.contains("Tranc") and Profondeur!="Tranchée bloqué"', engine='python').index, inplace=True)
df.drop(df.query('Profondeur.str.contains(".orage|..ranch", regex=True)', engine='python').index, inplace=True)
df.drop(df.fillna('').query('Description.str.contains("^.orage bloq|^.ranc.* bloq|^.*efus", regex=True)', engine='python').index, inplace=True)
df.drop(df.query('Ref!=Ref').index, inplace=True)
df.drop(columns=['MFT_145','Gouge_75','Liner_60', 'carottier', 'tarrière', 'idx'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [84]:
df['Litho_top'] = df['Profondeur'].apply(lambda x: x.replace(',','.').split('-')[0].strip(' m'))
df['Litho_base'] = df['Profondeur'].apply(lambda x: x.replace(',','.').split('-')[-1].strip(' m'))

In [85]:
df.rename({'Ref':'ID'}, axis=1, inplace=True)
if 'Profondeur' in df.columns: df.drop(columns=['Profondeur'], axis=1, inplace=True)

In [86]:
set([x[0] for x in list(set(df.ID)) if isinstance(x,str)])

{'F', 'T'}

In [87]:
df.loc[df.query('ID_date.str.contains("T")', engine='python').index, 'Type'] = 'Tranchee'
df.loc[df.query('Type==""', engine='python').index, 'Type'] = 'Forage'

In [88]:
df.loc[1268, ['ID_date','ID']] = df.loc[1267, ['ID_date','ID']]
df.loc[df.query('Description.isnull() or Description.str.len()<1').index, 'Description'] = ''

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [89]:
df.drop(index=df.query('Litho_base.isnull() or Litho_base.str.len()<1 or ' 
                       'Description.str.contains("Bloqu")').index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [90]:
compute_BH_length(df)

In [91]:
# we only have lithologies here, but separate facilities could be interesting for the following
coi = ['ID_date', 'Date_ouv', 'ID', 'Profondeur','Type', 'Societe']

litho = df
bh = df.loc[df.query('Type=="Forage"', engine='python').index] # boreholes 
trch = df.loc[df.query('Type=="Tranchee"', engine='python').index] # trenches
pz = df.loc[df.query('Type=="Piezo"', engine='python').index] # piezometers

bh=bh[coi].drop_duplicates(['ID'])
pz=pz[coi].drop_duplicates(['ID'])
trch=trch[coi].drop_duplicates(['ID'])

In [92]:
len(bh), len(pz), len(trch)

(172, 111, 17)

In [93]:
dataframe_viewer(pz, un_val='ID', rows=3)

Rows : 111, columns : 6, Unique values on col 'ID': 111


interactive(children=(IntSlider(value=3, description='rows', max=111, min=3, readout=False), IntSlider(value=6…

In [94]:
source_litho = df
source_bh = bh
source_pz = pz
source_trch = trch

In [95]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
trch.to_csv(tmp_dir+sheet+'_Trenches.csv', index=False)

source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False) #all lithologies or descriptions data in the source
source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
source_trch.to_csv(tmp_dir+'source_Trenches.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_mes_pz:{len(source_mes_pz)} ; '
     f'source_prv_sol:{len(source_prv_sol)} ; source_litho:{len(source_litho)} ;')

source_bh:172 ; source_pz:111 ; source_mes_pz:0 ; source_prv_sol:0 ; source_litho:1626 ;


* **Sheet : 'DONNEES PIEZOS'**

In [96]:
tmp_dir='../../CF_data/Result_traitem/database_Memoris3/'
sheet='Donnees_piezos'

In [97]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. '+
                        'Siterem - 2017/Database MEMORIS3.xlsx', sheet_name='DONNEES PIEZOS', skiprows=2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=3)

Rows : 130, columns : 22


interactive(children=(IntSlider(value=3, description='rows', max=130, min=3, readout=False), IntSlider(value=1…

In [98]:
sdf=df[df.columns.to_list()[:9]+df.columns.to_list()[13:17]]
df=df[df.columns.to_list()[:13]+df.columns.to_list()[17:22]]

In [99]:
dataframe_viewer(df, un_val='ID', rows=3), dataframe_viewer(sdf, un_val='ID', rows=3)

Rows : 130, columns : 18, Unique values on col 'ID': 130


interactive(children=(IntSlider(value=3, description='rows', max=130, min=3, readout=False), IntSlider(value=1…

Rows : 130, columns : 13, Unique values on col 'ID': 130


interactive(children=(IntSlider(value=3, description='rows', max=130, min=3, readout=False), IntSlider(value=1…

(None, None)

In [100]:
df.rename(columns={'ID':'Ref_id', 'Campagne':'Societe','N_piezo.':'ID','X [m]':'X','Y [m]':'Y','Z tête PZ [m]':'Z',
                        'Zsol [m]':'Zsol', 'Prof_PZ [m]':'Long_pz','Section_crépinée [m]':'Long_crep',
                        'Aquifère':'Nappe', 'Caractéristique':'Caractere',
                        'Diamètre_int [m]':'Diam_int_pz','Surnageant [cm]':'Surnageant','Sousnageant [cm]':'Sousnageant',
                        'Description éch. \nOd/turb.':'Opacite_eau','Remarques':'Rmq'}, inplace=True)

df=df.query("ID==ID")
df.replace('-',np.nan, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [101]:
df['Type']=df['Long_crep'].apply(lambda x: 'Piezo' if not pd.isnull(x) else 'Unknown')

In [102]:
df=df[['ID','X','Y','Z','Zsol','Type','Societe','Nappe','Long_pz','Long_crep','Diam_int_pz', 'Surnageant', 
         'Sousnageant','Caractere', 'Rmq', 'Opacite_eau','Zone', 'Sous_zone']]
df['Sousnageant']=df['Sousnageant'].apply(lambda x: x/100 if not pd.isnull(x) else x) #convert unit in [m]
df['Surnageant']=df['Surnageant'].apply(lambda x: x/100 if not pd.isnull(x) else x)

In [103]:
dataframe_viewer(df, un_val='ID', rows=3) # all units in [m]

Rows : 130, columns : 18, Unique values on col 'ID': 130


interactive(children=(IntSlider(value=3, description='rows', max=130, min=3, readout=False), IntSlider(value=1…

In [104]:
prv_eau=df[['ID','X','Y','Z','Zsol','Type', 'Surnageant', 'Sousnageant', 'Caractere','Rmq', 'Opacite_eau']]
prv_eau['Type_ech']='Eau'

pz=df.query("Type=='Piezo'")
pz=pz[['ID','X','Y','Z','Zsol','Type','Societe','Nappe','Long_pz','Long_crep','Diam_int_pz','Zone','Sous_zone']]
pz['Diam_int_pz']=pz['Diam_int_pz'].apply(lambda x: x*1000 if not pd.isnull(x) else x) # millimeters

ouv=df.query("Type!='Piezo'") # unknown facilities' type (it seems they are not boreholes)
ouv=ouv[['ID','X','Y','Z','Type','Societe']]
ouv['Type'] = 'Unknown'

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [105]:
# data in the second part of the initial dataframe
sdf.rename(columns={'ID':'Ref_id', 'Campagne':'Societe','N_piezo.':'ID','X [m]':'X','Y [m]':'Y','Z tête PZ [m]':'Z',
                  'Zsol [m]':'Zsol'}, inplace=True)
sdf.drop(columns='Ref_id', inplace=True)
sdf=sdf.query("ID==ID")

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [106]:
sdf = sdf[['ID', '\n27/04/2010', '\n8/09/2010', '\n27/04/2010.1', '\n8/9/2010', 'Societe', 'Zone', 
          'Sous_zone', 'X', 'Y', 'Zsol', 'Z',]]

In [107]:
df_tmp= sdf[['ID', 'X', 'Y', 'Zsol', 'Z']] # pd.DataFrame()
col=['ID', '\n27/04/2010', '\n8/09/2010', '\n27/04/2010.1', '\n8/9/2010'] # sdf.columns.to_list()[:5]
ID_mes=0

for j in range(1,len(col)):
    if re.search('/', col[j]):
        for i in sdf.index: #range(len(sdf)):
            d=col[j].strip('\n|.1').split('/')
            df_tmp.loc[ID_mes,'Date_mes']=dtm.date(int(d[2]), int(d[1]), int(d[0]))
            df_tmp.loc[ID_mes,'ID']=str(sdf.loc[i,'ID'])

            if j<=2: 
                df_tmp.loc[ID_mes,'Niv_eau_pz']=sdf.iloc[i,j]
                df_tmp.loc[ID_mes,'Niv_eau_sol']=sdf.iloc[i,j+2]
                ID_mes+=1

df_tmp=df_tmp.sort_values('Date_mes').reset_index(drop=True)
df_tmp['ID_mes']=df_tmp['ID'].apply(lambda x: 'Mes_'+str(x))
df_tmp.insert(0, 'ID_mes', df_tmp.pop('ID_mes'))

mes_pz = df_tmp[:-1]

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/tmp/ipykernel_27445/1035597014.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp.loc[ID_mes,'Date_mes']=dtm.date(int(d[2]), int(d[1]), int(d[0]))


In [108]:
mes_pz['Type_mes'] = 'Phys-chim'

In [109]:
dataframe_viewer(mes_pz, rows=3), dataframe_viewer(ouv, rows=3)

Rows : 260, columns : 10


interactive(children=(IntSlider(value=3, description='rows', max=260, min=3, readout=False), IntSlider(value=1…

Rows : 13, columns : 6


interactive(children=(IntSlider(value=3, description='rows', max=13, min=3, readout=False), IntSlider(value=6,…

(None, None)

In [110]:
source_pz = pz
source_prv_eau = prv_eau
source_ouv = ouv
source_mes_pz = mes_pz

In [111]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)


source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False) #all lithologies or descriptions data in the source
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_prv_sol:{len(source_prv_sol)} ; source_prv_eau:{len(source_prv_eau)} ;\n'
     f'source_mes_pz:{len(source_mes_pz)} ; ')

source_bh:172 ; source_pz:117 ; source_litho:1626 ; source_Fac-uknw:13 ; source_prv_sol:0 ; source_prv_eau:130 ;
source_mes_pz:260 ; 


* **Sheet : 'DRAINS ET PIEZOS ENEL'**

In [112]:
tmp_dir='../../CF_data/Result_traitem/database_Memoris3/'
sheet='Drains_Pz_ENEL'

In [113]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/Database MEMORIS3.xlsx', 
                        sheet_name='DRAINS ET PIEZOS ENEL', skiprows=1)

df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df)

Rows : 21, columns : 65


interactive(children=(IntSlider(value=10, description='rows', max=21, min=10, readout=False), IntSlider(value=…

In [114]:
df.rename(columns={'N°':'ID', 'Date ':'Date_prv','Hauteur de la chambre ':'Ht_Chbre','T':'Temp'}, inplace=True)

df_tmp=df[df.columns.to_list()[:6]+df.columns.to_list()[10:-2]]
df=df[df.columns.to_list()[:7]]

In [115]:
df.columns.to_list()[:12]

['ID', 'Date_prv', 'ETUDE', 'X', 'Y', 'Zsol', 'Ht_Chbre']

In [116]:
df=df[['ID', 'X', 'Y', 'Zsol', 'Ht_Chbre']][2:21]
df.replace('-',np.nan, inplace=True)
df.drop(index=[5], inplace=True)
df.loc[:14,'Type']='Unknown'
df.loc[15:,'Type']='Piezo'
#df['Zsol']=df['Zsol'].apply(lambda x: x if not pd.isnull(x) else np.nan)

In [117]:
dataframe_viewer(df, un_val='ID', rows=3)

Rows : 18, columns : 6, Unique values on col 'ID': 18


interactive(children=(IntSlider(value=3, description='rows', max=18, min=3, readout=False), IntSlider(value=6,…

In [118]:
pz=df.query("Type=='Piezo'")
ouv=df.query("Type!='Piezo'")

In [119]:
sdf=df_tmp.query('ID==ID').reset_index(drop=True)

In [120]:
sdf.columns

Index(['ID', 'Date_prv', 'ETUDE', 'X', 'Y', 'Zsol', 'Niv_EAU_SOL (m)',
       'PZ absolue (m)', 'PZ relative (m)', 'Niv_EAU_SOL (m).1', 'pH', 'CE',
       'Temp', 'ORP', 'Odiss', 'arsenic', 'cadmium', 'chrome', 'cobalt',
       'cuivre', 'mercure', 'plomb', 'nickel', 'zinc', 'CN_libre', 'CN_totaux',
       'CN_totaux.1', 'CN_totaux.2', 'thiocyanate', 'benzène', 'toluène',
       'éthylbenzène', 'orthoxylène', 'para- et métaxylène', 'xylènes',
       'BTEX total', 'styrène', 'Iph.', 'naphtalène', 'anthracène',
       'phénanthrène', 'fluoranthène', 'benzo(a)anthracène', 'chrysène',
       'benzo(a)pyrène', 'benzo(ghi)pérylène', 'benzo(k)fluoranthène',
       'indéno(1,2,3-cd)pyrène', '\nC5-C8', 'C8-C10', 'C10-C12', 'C12-C16',
       ' C16 - C21', 'C21 - C35', 'C35 - C40', 'totaux C10-C35', 'C10-C12.1',
       'C12-C22', 'C22-C30'],
      dtype='object')

In [121]:
df_tmp=sdf[['ID', 'Date_prv', 'X', 'Y', 'Zsol']] #pd.DataFrame()
cols=['Niv_EAU_SOL (m)', 'PZ absolue (m)', 'Niv_EAU_SOL (m).1'] # sdf.columns.to_list()[6:10]
#cols.remove('PZ relative (m)')
print(cols)
ID_mes=0
d=['01/10/2013','01/11/2015','01/12/2016'] # 01/11/2015 added by me (according to data observation)

for i in sdf.index:
    k=0
    for j in cols:
        #df_tmp.loc[ID_mes,'ID_mes']='Mes_'+str(ID_mes)
        df_tmp.loc[ID_mes,'Date_mes']=dtm.date(int(d[k].split('/')[2]), int(d[k].split('/')[1]), 
                                                             int(d[k].split('/')[0]))
        df_tmp.loc[ID_mes,'ID']=str(sdf.loc[i,'ID'])
        df_tmp.loc[ID_mes,'Niv_eau_sol']=sdf.loc[i,j]
                
        if df_tmp.loc[ID_mes, 'Date_mes']==sdf.loc[i, 'Date_prv'] and \
        df_tmp.loc[ID_mes, 'ID']==sdf.loc[i, 'ID']:
            df_tmp.loc[ID_mes, sdf.columns.to_list()[6:-1]]=sdf.iloc[i, 6:-1]
        
        ID_mes+=1
        k+=1
df_tmp.replace('-', np.nan, inplace=True)
df_tmp=df_tmp.sort_values('Date_mes').reset_index(drop=True)
df_tmp['ID_mes']=df_tmp['ID'].apply(lambda x: 'Mes_'+str(x))
df_tmp.insert(0, 'ID_mes', df_tmp.pop('ID_mes'))

['Niv_EAU_SOL (m)', 'PZ absolue (m)', 'Niv_EAU_SOL (m).1']


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/tmp/ipykernel_27445/4227729424.py:17: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to mat

In [122]:
df_tmp=na_line_drop(df_tmp, 3)

In [123]:
df_tmp['ID_mes']=df_tmp['ID'].apply(lambda x: 'Mes_'+x)
df_tmp['CE']=df_tmp['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [124]:
mes_pz=df_tmp[['ID_mes', 'Date_mes', 'ID', 'X', 'Y', 'Zsol', 'Niv_eau_sol', 'pH', 'CE', 'Temp', 'ORP','Odiss']]

an=df_tmp[['ID','X', 'Y', 'Zsol', 'arsenic', 'cadmium', 'chrome', 'cobalt', 'cuivre', 'mercure',
       'plomb', 'nickel', 'zinc', 'CN_libre', 'CN_totaux', 'CN_totaux.1',
       'CN_totaux.2', 'thiocyanate', 'benzène', 'toluène', 'éthylbenzène',
       'orthoxylène', 'para- et métaxylène', 'xylènes', 'BTEX total',
       'styrène', 'Iph.', 'naphtalène', 'anthracène', 'phénanthrène',
       'fluoranthène', 'benzo(a)anthracène', 'chrysène', 'benzo(a)pyrène',
       'benzo(ghi)pérylène', 'benzo(k)fluoranthène', 'indéno(1,2,3-cd)pyrène',
       '\nC5-C8', 'C8-C10', 'C10-C12', 'C12-C16', ' C16 - C21', 'C21 - C35',
       'C35 - C40', 'totaux C10-C35', 'C10-C12.1', 'C12-C22']]

#another way to do this
#an=df_tmp[df_tmp.columns.to_list()[1:3]+df_tmp.columns.to_list()[9:]]

In [125]:
mes_pz['Type_mes'] = 'Phys-chim' 

In [126]:
an=na_line_drop(an, 2)
an.insert(1, 'Type_ech', 'Eau')
an.rename(columns={'ID':'ID_ech'}, inplace=True)
#an['Anl_ID']=an['ID'].apply(lambda x: 'Anl_'+str(x))
#an.insert(an.columns.to_list().index('ID')+1, 'Type_ech', an.pop('Type_ech'))

28 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [127]:
an=col_ren(an, name=pol_field_model, mode=1)

In [128]:
an=dble_col_drop(an)

column(s) dropped: ['16:CN_tot', '17:CN_tot', '46:Fract_C10C12']


In [129]:
dataframe_viewer(source_mes_pz, rows=5)

Rows : 260, columns : 10


interactive(children=(IntSlider(value=5, description='rows', max=260, min=5, readout=False), IntSlider(value=1…

Data merging

In [130]:
source_pz, error_df=data_merger(source_pz, pz, 'outer', 'ID')

In [131]:
source_mes_pz, error_df=data_merger(source_mes_pz, mes_pz, 'outer', 'ID_mes')

In [132]:
source_ouv, error_df=data_merger(source_ouv, ouv, 'outer', 'ID')

In [133]:
source_an=an

In [134]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False) 
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_prv_sol:{len(source_prv_sol)} ; source_prv_eau:{len(source_prv_eau)} ;\n'
     f'source_mes_pz:{len(source_mes_pz)} ; source_an:{len(source_an)} ;')

source_bh:172 ; source_pz:123 ; source_litho:1626 ; source_Fac-uknw:25 ; source_prv_sol:0 ; source_prv_eau:130 ;
source_mes_pz:317 ; source_an:29 ;


* **Sheet : 'RESULTS_EAU' (F)**

In [135]:
tmp_dir='../../CF_data/Result_traitem/database_Memoris3/'
sheet='Result_eau'

In [136]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/Database MEMORIS3.xlsx', 
                        sheet_name='RESULTS_EAU', skiprows=1)

df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

Rows : 131, columns : 185


interactive(children=(IntSlider(value=5, description='rows', max=131, min=5, readout=False), IntSlider(value=1…

In [137]:
df.drop(0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [138]:
df.rename(columns={'Campagne':'Societe','N_piezo.':'ID','Z tête PZ':'Z','Zsol':'Zsol', 'Prof_PZ':'Long_pz',
                        'Section_crépinée':'Long_crep','Diamètre_int':'Diam_int_pz','Surnageant':'Surnageant',
                        'Sousnageant':'Sousnageant','Description éch.':'Opacite_eau','Remarques':'Rmq',
                        'Aquifère_échantillonné':'Nappe', 'Caractéristique':'Caractere'}, inplace=True)

#df=df[['ID','X', 'Y', 'Z', 'Zsol', 'Long_pz', 'Long_crep', 'Diam_int_pz', 'Societe']]#[:130]
df=df.query("ID ==ID")
df.replace('-',np.nan, inplace=True)

In [139]:
dataframe_viewer(df, rows=5)

Rows : 130, columns : 185


interactive(children=(IntSlider(value=5, description='rows', max=130, min=5, readout=False), IntSlider(value=1…

In [140]:
# splitting
sdf=df[[df.columns.to_list()[0]]+['X','Y','Z','Zsol']+df.columns.to_list()[12:16]+df.columns.to_list()[21:26]]
an=df[[df.columns.to_list()[0]]+['X','Y','Z','Zsol']+df.columns.to_list()[26:]]
prv_eau=df[df.columns.to_list()[:3]+['X','Y','Z','Zsol']+df.columns.to_list()[16:21]+['Nappe']]
df=df[df.columns.to_list()[:12]]

In [141]:
df['Type']=df['Long_crep'].apply(lambda x: 'Piezo' if not pd.isnull(x) else 'Unknown')
df.insert(8, 'Type', df.pop('Type'))
df['Diam_int_pz'] = df['Diam_int_pz'].apply(lambda x: x*1000 if not pd.isnull(x) else x)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [142]:
pz=df.query("Type=='Piezo'")
ouv=df.query("Type!='Piezo'")[['ID', 'Societe', 'Zone', 'Sous_zone', 'X', 'Y', 'Z', 'Type']]

In [143]:
prv_eau['Surnageant']=prv_eau['Surnageant'].apply(lambda x: x/100) # to express value in [m]
prv_eau['Sousnageant']=prv_eau['Sousnageant'].apply(lambda x: x/100)

In [144]:
df_tmp=sdf[['ID','X','Y','Z','Zsol']]
cols=sdf.columns.to_list()[9:]
ID_mes=0
d=['27/04/2010', '08/09/2010']

for i in range(len(sdf)):
    for k in [0,1]:
        df_tmp.loc[ID_mes,'ID_mes']='Mes_'+str(ID_mes)
        df_tmp.loc[ID_mes,'ID']=str(sdf.loc[i,'ID'])
        
        dt=d[k].split('/')
        df_tmp.loc[ID_mes,'Date_mes']=dtm.date(int(dt[2]), int(dt[1]),int(dt[0]))
        df_tmp.loc[ID_mes,'Niv_eau_pz']=sdf.iloc[i,k+1]
        df_tmp.loc[ID_mes,'Niv_eau_sol']=sdf.iloc[i,k+3]
        df_tmp.loc[ID_mes, cols]=list(sdf.loc[i,cols])
        ID_mes+=1          
        
df_tmp.replace('-', np.nan, inplace=True)
df_tmp=df_tmp.sort_values('Date_mes').reset_index(drop=True)
df_tmp['ID_mes']=df_tmp['ID'].apply(lambda x: 'Mes_'+str(x))
df_tmp.insert(0, 'ID_mes', df_tmp.pop('ID_mes'))

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [145]:
q=df_tmp.query('Niv_eau_pz.isnull() and Niv_eau_sol.isnull()').index
df_tmp.drop(q, inplace=True)
df_tmp['CE']=df_tmp['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)
mes_pz=df_tmp
mes_pz['Type_mes'] = 'Phys-chim'

In [146]:
source_mes_pz=source_mes_pz.sort_values('Date_mes').reset_index(drop=True)

In [147]:
dataframe_viewer(mes_pz, rows=3), dataframe_viewer(source_mes_pz, rows=3)

Rows : 260, columns : 15


interactive(children=(IntSlider(value=3, description='rows', max=260, min=3, readout=False), IntSlider(value=1…

Rows : 317, columns : 15


interactive(children=(IntSlider(value=3, description='rows', max=317, min=3, readout=False), IntSlider(value=1…

(None, None)

In [150]:
dataframe_viewer(error_df, rows=3)

Rows : 189, columns : 11


interactive(children=(IntSlider(value=3, description='rows', max=189, min=3, readout=False), IntSlider(value=1…

In [151]:
pd.unique(error_df.Check_col)

array(['Niv_eau_pz, Niv_eau_sol', 'Niv_eau_pz, Niv_eau_sol, Z, Zsol',
       'Z, Zsol', 'Niv_eau_pz, Niv_eau_sol, Z', 'Niv_eau_pz'],
      dtype=object)

In [149]:
source_mes_pz, error_df = data_merger(source_mes_pz, mes_pz, on=['ID', 'Date_mes'], how='outer', crit_2nd_col='Date_mes')

Conflict values present. Please resolve this manually !


In [150]:
an=na_line_drop(an,col_n=3)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [151]:
an=dble_col_drop(an)

column(s) dropped: []


In [152]:
an.insert(1, 'Type_ech', 'Eau')
an.rename(columns={'ID':'ID_ech'}, inplace=True)
#an['Anl_ID']=an['ID'].apply(lambda x: 'Anl_'+str(x))
#an.insert(an.columns.to_list().index('ID')+1, 'Type_ech', 'Eau')

In [153]:
an=col_ren(an, name=pol_field_model, mode=1)

data merging

In [154]:
source_pz, error_df=data_merger(source_pz, pz, 'outer', 'ID')

In [155]:
source_mes_pz, error_df=data_merger(source_mes_pz, mes_pz, 'outer', 'ID')
if len(error_df)>0 :dataframe_viewer(error_df, rows=3)

Conflict values present. Please resolve this manually !
Rows : 769, columns : 12


interactive(children=(IntSlider(value=3, description='rows', max=769, min=3, readout=False), IntSlider(value=1…

In [156]:
dataframe_viewer(source_mes_pz, rows=3)

Rows : 1162, columns : 18


interactive(children=(IntSlider(value=3, description='rows', max=1162, min=3, readout=False), IntSlider(value=…

In [157]:
source_ouv, error_df=data_merger(source_ouv, ouv, 'outer', 'ID')
if len(error_df)>0 :dataframe_viewer(error_df, rows=3)

In [158]:
len(source_an.columns),len(set(source_an.columns))
source_an.columns

Index(['ID_ech', 'Type_ech', 'X', 'Y', 'Zsol', 'As', 'Cd', 'Cr', 'cobalt',
       'Cu', 'Hg', 'Pb', 'Ni', 'Zn', 'CN_libre', 'CN_tot', 'thioCN', 'Bnz',
       'Toln', 'EthylBnz', 'O-Xyl', 'P-M-Xyl', 'Xyl', 'BTEX total', 'Styr',
       'Iph.', 'Naphta', 'Anthrc', 'Phenanthr', 'Flranth', 'Bnz(a)anthrc',
       'Chrys', 'Bnz(a)pyr', 'Bnz(ghi)peryl', 'Bnz(k)flranth',
       'Indeno(1,2,3-cd)pyr', 'Fract_C5C8', 'Fract_C8C10', 'Fract_C10C12',
       'Fract_C12C16', 'Fract_C16C21', 'Fract_C21C35', 'Fract_C35C40',
       'HC_tot_C10C35', 'Fract_C12C22'],
      dtype='object')

In [159]:
source_an, error_df=data_merger(source_an, an, 'outer', 'ID_ech')
if len(error_df)>0 :dataframe_viewer(error_df, rows=3)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  exec(code_obj, self.user_global_ns, self.user_ns)


In [160]:
dataframe_viewer(source_an)

Rows : 159, columns : 167


interactive(children=(IntSlider(value=10, description='rows', max=159, min=10, readout=False), IntSlider(value…

In [161]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False) #all lithologies or descriptions data in the source
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_prv_sol:{len(source_prv_sol)} ; source_prv_eau:{len(source_prv_eau)} ;\n'
     f'source_mes_pz:{len(source_mes_pz)} ; source_an:{len(source_an)} ;')

source_bh:172 ; source_pz:123 ; source_litho:1626 ; source_Fac-uknw:25 ; source_prv_sol:0 ; source_prv_eau:130 ;
source_mes_pz:1162 ; source_an:159 ;


* **Sheet : 'RESULTS_SOL'**

In [162]:
tmp_dir='../../CF_data/Result_traitem/database_Memoris3/'
sheet='Result_sol'

In [163]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/'
                   'Database MEMORIS3.xlsx', sheet_name='RESULTS_SOL', skiprows=1)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Rows : 1423, columns : 94


interactive(children=(IntSlider(value=10, description='rows', max=1423, min=10, readout=False), IntSlider(valu…

In [164]:
df.rename(columns={'Unnamed: 92':'EOX', 'Unnamed: 93':'Idc_phenol','Campagne':'Societe','N_forage':'ID','refus':'Refus',
                   'Prof.\nforage':'Long', 'N_ech':'ID_ech', 'Min_Ech':'Ech_top','Max_Ech':'Ech_base',
                  'Terrain':'Nappe','Epaisseur remblais':'Ep_remb', 'Epaisseur alluvions':'Ep_alluv', 
                   'Nature':'Polluant','Min_organo':'Pol_top', 'Max_organo':'Pol_base', 'Fraction   2000 µm':'Fract_2000µ',
                   'Fraction   63 µm':'Fract_63µ', 'Fraction   45 µm':'Fract_45µ','Fraction   16 µm':'Fract_16µ',
                   'Fraction   2 µm':'Fract_2µ'}, inplace=True)

In [165]:
df.drop(columns=[df.columns.to_list()[x] for x in range(len(df.columns))
                      if re.search(r"Unnamed",df.columns.to_list()[x])], axis=1, inplace=True) 
df.replace(r'<|>','', inplace=True, regex=True)
df=df.query('ID==ID')
df['ID']=df['ID'].apply(lambda x : x.replace('Monito ', 'Mon') if re.findall('Monit', str(x)) else x)
df['ID_ech']=df['ID_ech'].apply(lambda x : x.replace('Monito ', 'Mon') if re.findall('Monit', str(x)) else x)
df.replace('-',np.nan, inplace=True)

In [166]:
for i in range(len(df['ID'])):
    #r=re.search('(\w+)/.+',str(df.loc[i, 'ID_ech']))
    #if r : df.loc[i, 'ID']=r.group(1)
    r=re.search('^\d+',str(df.loc[i, 'ID']))
    if r : df.loc[i, 'ID']='F'+str(df.loc[i, 'ID'])

In [167]:
sdf=df[['ID','X','Y','Z','Long','ID_ech', 'Ech_top', 'Ech_base', 'Description','Nappe','Ep_remb','Ep_alluv','Refus','Societe','Zone','Sous_zone']]
sdf.insert(5, 'Type', '')

prv_sol=df[['ID','ID_ech', 'X','Y','Z', 'Ech_top', 'Ech_base','Polluant','Intensité', 'Pol_top','Pol_base','MS','pH H2O',
            'T° pH H2O', 'T° pH CaCl2','pH CaCl2','T° pH KCl','pH KCl','T° CE','CE','MO',
       'Résidus chauffage', 'Argile ', 'Fract_2000µ','Fract_63µ','Fract_45µ','Fract_16µ','Fract_2µ']]
#mes_sol=df[['ID','ID_ech','MS','pH H2O', 'T° pH H2O', 'T° pH CaCl2','pH CaCl2','T° pH KCl','pH KCl','T° CE','CE','MO',
#       'Résidus chauffage', 'Argile ', 'Fract_2000','Fract_63','Fract_45','Fract_16','Fract_2']]
prv_sol.insert(2, 'Type_ech', 'Sol')

an=df[['ID','ID_ech','X','Y','Z','Arsenic','Cadmium','Chrome_total','Chrome_VI','Cobalt','Cuivre','Mercure','Plomb','Nickel','Zinc','Libres',
       'Totaux', 'Non chloro destruct.', 'Thiocyantes', 'Cyanures totaux EPA','Benzène', 'Toluène', 'Ethylbenzène',
       'o-Xylènes','mp-Xylènes','Xylènes','SOM BTEX','Styrène','Naphtalène','Anthracene','Phénanthrène',
       'Fluoranthène', 'Benzoaanthracène', 'Chrysène','Benzo(a)pyrene','Benzo(ghi)pérylène','Benzo(k)fluoranthène',
       'Indéno[123cd]pyrène', 'Acenaphtylene', 'Acenaphthene', 'Fluorène','Pyrène', 'Benzo_b_fluoranthene', 
       'Dibenzo[ah]anthracène','SOM VROM 10', 'SOM EPA 16', 'C5_C8', 'C8_C10', 'C10_C12', 'C12_C16','C16_C21', 
       'C21_C35', 'C35_C40', 'SOM_C5_C35', 'C21_C30', 'C30_C35','SOM C10_C40', 'EOX', 'Idc_phenol']]

dfs processing

In [168]:
sdf['Refus']=sdf['Refus'].apply(lambda x: 'x' if not pd.isnull(x) else '')

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [169]:
for i in range(len(sdf['Nappe'])):
    x=sdf.loc[i,'Nappe']
    if re.search('[R|r]em', str(x)) : sdf.loc[i,'Nappe']='Remblais'
    elif re.search('[A|a]ll', str(x)) : sdf.loc[i,'Nappe']='Alluvions'
    elif re.search('[S|s]oc', str(x)) : sdf.loc[i,'Nappe']='Socle'
    elif re.search('[A|a]rg', str(x)) : sdf.loc[i,'Nappe']='Argile'
    else : sdf.loc[i,'Nappe']=''

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [170]:
litho=sdf #lithologies and all facilities without distinction here (because type of facility not defined clearly !)

In [171]:
prv_sol=na_line_drop(prv_sol, 3)

9 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [172]:
prv_sol=na_col_drop(prv_sol, col_non_na=5, verbose=False)


Columns dropped :['Résidus chauffage']



In [173]:
an.replace('#',np.nan, inplace=True)
an=na_line_drop(an, 2)
an.insert(1, 'Type_ech', 'Sol')
#an['Anl_ID']=an['ID'].apply(lambda x: 'Anl_'+str(x))
#an.insert(0, 'Anl_ID', an.pop('Anl_ID'))

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


9 NaN lines dropped


data merging

In [177]:
dataframe_viewer(litho, rows=3), dataframe_viewer(source_litho, rows=3)

Rows : 1423, columns : 17


interactive(children=(IntSlider(value=3, description='rows', max=1423, min=3, readout=False), IntSlider(value=…

Rows : 1626, columns : 10


interactive(children=(IntSlider(value=3, description='rows', max=1626, min=3, readout=False), IntSlider(value=…

(None, None)

In [175]:
#source_mes_sol=mes_sol
source_prv_sol=prv_sol

In [176]:
dataframe_viewer(error_df, un_val='ID', rows=3) 

Rows : 6685, columns : 6, Unique values on col 'ID': 216


interactive(children=(IntSlider(value=3, description='rows', max=6685, min=3, readout=False), IntSlider(value=…

In [29]:
dataset = source_litho
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
dataframe_viewer(error_df.query("Societe_x!='Done' or Description_x!='Done'"), un_val='ID', rows=3) 

In [ ]:
source_an, error_df=data_merger(source_an, an, 'outer', 'ID_ech', ) 

In [ ]:
dataframe_viewer(prv_sol, un_val='ID', rows=3), dataframe_viewer(an, un_val='ID', rows=3) 

In [ ]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)
source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

### $\color{red}{\textbf{Excel sheets data merging}}$

In [ ]:
excel_bhs, conflict_df = data_merger(source_bh, source_pz, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [ ]:
data_validation(overall_data=excel_bhs, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Type_y':list(conflict_df.index)})

In [29]:
dataset = excel_bhs
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
excel_bh_litho, conflict_df = data_merger(source_bh, source_litho, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [ ]:
excel_bh_soil_sp, conflict_df = data_merger(source_bh, source_prv_sol, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [ ]:
excel_bh_equip, conflict_df = data_merger(source_bh, source_equip, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [ ]:
excel_bh_mes, conflict_df = data_merger(source_bh, source_mes_pz, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [ ]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)

In [ ]:
dataframe_viewer(excel_bh_mes, rows=3)

#### ------------------------------------------------------------------------------------------------------------------

## 3-obsrevations terrain et mesures piézos phase 2.xlsx

* **Sheet : 'Piézométrie'**

In [66]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [67]:
tmp_dir='../../CF_data/Result_traitem/observ_terrain/'
sheet='Piezometrie'

In [68]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/résultats phase 2/'
                   'obsrevations terrain et mesures piézos phase 2.xlsx', sheet_name='Piézométrie', skiprows=1)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df)


Columns dropped :['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']

Rows : 31, columns : 19


interactive(children=(IntSlider(value=10, description='rows', max=31, min=10, readout=False), IntSlider(value=…

In [69]:
sdf=df[df.columns.to_list()[:3]]
sdf=na_line_drop(sdf,0)
sdf.rename(columns={'Niveau \npiézométrique':'Niv_eau_sol', 'Commentaires ':'Date_prv'}, inplace=True)

9 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyW

In [70]:
sdf2=df.loc[:11, df.columns.to_list()[3:-1]]
sdf2.rename(columns={'Unnamed: 7':'Date_prv', 'Unnamed: 8':'Nappe', 'Unnamed: 9':'ID', 'NP/piézo [m]':'Niv_eau_pz',
       'dim. piezo hors sol [m]':'Dim_pz_sol', 'NP/sol [m]':'Niv_eau_sol', 'Prof. piézo/piézo [m]':'Long_pz',
       'Prof. piézo/sol [m]':'Long_pz_sol', 'CE [mS/cm]':'CE','t° [°C]':'Temp','O2 dissous\n[%]':'O_diss', 
        'Observations':'Rmq'}, 
           inplace=True)

In [71]:
for i in range(len(sdf2['ID'])):
    sdf2.loc[i,'ID']=re.sub(r'^P','F', sdf2.loc[i,'ID'])
    
    if pd.isnull(sdf2.loc[i,'CE']) and not pd.isnull(sdf2.loc[i,'CE [µS/cm]']):
        sdf2.loc[i,'CE']=sdf2.loc[i,'CE [µS/cm]']/1000

sdf2.drop(['CE [µS/cm]'], axis=1, inplace=True)

In [72]:
df=df.loc[14:, df.columns.to_list()[3:-1]]
df.rename(columns={'Unnamed: 7':'Date_prv', 'Unnamed: 8':'Nappe', 'Unnamed: 9':'ID', 'NP/piézo [m]':'Niv_eau_pz',
       'dim. piezo hors sol [m]':'Dim_pz_sol', 'NP/sol [m]':'Niv_eau_sol', 'Prof. piézo/piézo [m]':'Long_pz',
       'Prof. piézo/sol [m]':'Long_pz_sol', 'CE [mS/cm]':'CE','t° [°C]':'Temp','O2 dissous\n[%]':'O_diss', 
        'Observations':'Rmq'}, 
           inplace=True)
df.drop([19,20], inplace=True)
df.reset_index(drop=True, inplace=True)

In [74]:
for i in range(len(df['ID'])):
    df.loc[i,'ID']=re.sub(r'^P','F', df.loc[i,'ID'])
    
    if pd.isnull(df.loc[i,'CE']) and not pd.isnull(df.loc[i,'CE [µS/cm]']):
        df.loc[i,'CE']=df.loc[i,'CE [µS/cm]']/1000
        
df.drop(['CE [µS/cm]', 'O_diss'], axis=1, inplace=True)

In [75]:
df=na_col_drop(df, 5)
sdf2=na_col_drop(sdf2, 5,)


Columns dropped :['ORP', 'O_diss']



In [80]:
prv_eau, error_df=data_merger(sdf2, df, how='outer', on='ID')

In [81]:
dataframe_viewer(prv_eau, rows=5, un_val='ID')

Rows : 27, columns : 13, Unique values on col 'ID': 27


interactive(children=(IntSlider(value=5, description='rows', max=27, min=5, readout=False), IntSlider(value=12…

In [82]:
prv_eau=prv_eau[['ID','Date_prv','Long_pz', 'Long_pz_sol','Dim_pz_sol','Nappe','Niv_eau_sol', 'Niv_eau_pz',
                 'pH', 'Temp', 'CE', 'ORP','Rmq']]
prv_eau.insert(1,'Type_ech','Eau')

source_prv_eau=prv_eau

In [83]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)

source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:0 ;
source_prv_sol:0 ;source_prv_eau:27 ; source_mes_pz:0 ; source_mes_sol:0 ;


--------------------------------------------------------------------------------------------------------

## 4-profondeur de contact campagne de forages octobre 2019.xlsx

* **Sheet : 'Feuil1'**

In [84]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [85]:
tmp_dir='../../CF_data/Result_traitem/Prof_contact_sol_forage/'
sheet='Feuil1'

In [86]:
df = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/profondeur de contact campagne de forages octobre 2019.xlsx', 
                   sheet_name='Feuil1', skiprows=2)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df)

Rows : 8, columns : 5


interactive(children=(IntSlider(value=8, description='rows', max=8, min=8, readout=False), IntSlider(value=5, …

In [87]:
df.rename(columns={'n°forage ':'ID','profondeur(m)':'Long_for','x':'X', 'y':'Y', 'z':'Z'}, inplace=True)
df['Type']='Forage' # type is not defined clearly in data
df['ID']=df['ID'].apply(lambda x: 'F'+str(x).replace('.0',''))

bh=df

In [88]:
source_bh=bh

In [89]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:8 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:0 ;
source_prv_sol:0 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


--------------------------------------------------------------------------------------------------------

## 5-Forages_Pilote_Decoupe.xlsx

* **Sheet : 'leve'**

In [90]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [91]:
tmp_dir='../../CF_data/Result_traitem/Forage_Pilote/'
sheet='leve_Z_elect_pos'

In [92]:
df = pd.read_excel('../../CF_data/Data_UMONS/geometrie_electrodes_et_sondes/Forages_Pilote_Decoupe.xlsx', 
                   sheet_name='leve')#, skiprows=0)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)


Columns dropped :['Unnamed: 10']

Rows : 72, columns : 11


interactive(children=(IntSlider(value=5, description='rows', max=72, min=5, readout=False), IntSlider(value=11…

In [ ]:
df.columns

In [ ]:
df.rename(columns={'Ref_puits':'ID','Niveau mesuré':'Z_mes', 'Niveau corrigé':'Z','Z_diff [m] repere_local':'Diff_Z_local',
                   'long_fin [m]':'Long_for','Pos_Inox_#1 [m]':'Pos_Inox_#1', 
                   'Pos_Inox_#6 [m]':'Pos_Inox_#6', 'Pos_Impol_#3 [m]':'Pos_Impol_#3'}, inplace=True)

In [ ]:
df['Type']='Forage' # type is not defined clearly in data
df['ID']=df['ID'].apply(lambda x: 'F'+str(x).replace('.0',''))

elc = df[['ID','Pos_Inox_#6', 'Pos_Impol_#3']] # 'ID' is for boreholes
bh = df[['ID','Z','Diff_Z_local','Long_for', 'Type']]# Z_local origin = 145.5 [m]

In [ ]:
source_bh = bh
source_elc = elc

In [ ]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
elc.to_csv(tmp_dir+sheet+'_Electrodes.csv', index=False)

source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
source_elc.to_csv(tmp_dir+'source_Electrodes.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

--------------------------------------------------------------------------------------------------------

## 6-Liste XY investigations.xlsx
* **Sheet : 'SOL_EAU'**

In [93]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [94]:
tmp_dir='../../CF_data/Result_traitem/Liste_XY/'
sheet='Sol_Eau'

In [95]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Etude de caracterisation. SITEREM - 2011/Documents '
                   'supplémentaires/Liste XY investigations.xlsx', sheet_name='SOL')#, skiprows=4)
df['Type_ech']='Sol'

df1 = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Etude de caracterisation. SITEREM - 2011/Documents '
                   'supplémentaires/Liste XY investigations.xlsx', sheet_name='EAU PR')#, skiprows=4)
df1['Type_ech']='Eau'
df1['Nappe']='Socle'

df2 = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Etude de caracterisation. SITEREM - 2011/Documents '
                   'supplémentaires/Liste XY investigations.xlsx', sheet_name='EAU RB')#, skiprows=4)
df2['Type_ech']='Eau'
df2['Nappe']='remblais'

df3 = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Etude de caracterisation. SITEREM - 2011/Documents '
                   'supplémentaires/Liste XY investigations.xlsx', sheet_name='EAU ALL')#, skiprows=4)
df3['Type_ech']='Eau'
df3['Nappe']='Alluvions'

In [96]:
df2=na_line_drop(df2,0)
df2=na_col_drop(df2,1)

In [100]:
mdf, error_df=data_merger(df1, df2, 'outer', 'N°')

KeyError: '`N°`'

In [98]:
mdf=mdf.append(df3)
mdf=mdf.dropna(how='any', subset=['N°'])

In [ ]:
mdf, error_df=data_merger(mdf, df, 'outer', 'N°') 

In [ ]:
mdf.rename(columns={'N°':'ID'}, inplace=True)
mdf['Type'] = 'Piezo'
pz=mdf
source_pz = pz # we only have boreholes 'ID' here, no Z, no date

In [ ]:
dataframe_viewer(source_bh, rows=5)

In [ ]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)    
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

--------------------------------------------------------------------------------------------------------

## 7-Résultats phase 1_MEMORIS.xls
* **Sheet : 'Résult SOL'**

In [71]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [72]:
tmp_dir='../../CF_data/Result_traitem/Phase_1_Memoris/'
sheet='Result_sol'

In [73]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/résultats phase 1/'
                   'Résultats phase 1_MEMORIS.xls', sheet_name='Résult SOL', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

1 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 135, columns : 35


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=135, min=5, readout=False), IntSlider(value=1…

In [74]:
prv_sol=df.loc[:35]
an=df.loc[36:]

In [75]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [76]:
prv_sol=prv_sol.transpose()
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=col_ren(prv_sol, 1)

In [77]:
prv_sol.drop(list(range(5)), axis=0, inplace=True)
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=na_col_drop(prv_sol,1)
prv_sol=na_line_drop(prv_sol,3)
prv_sol.reset_index(drop=True, inplace=True)


Columns dropped :['col_1', "Nom / description d'échantillon", 'Date de prélèvement', "Nature de l'étude (*)", 'Terrain', 'Epaisseur de remblais', 'Epaisseur alluvions', "Nature de l'observation organoleptique", 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'zone', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'X Lambert', 'Y Lambert', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (***)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'Matières organiques', 'GRANULOMETRIE', 'Fraction argileuse']



In [78]:
name=['ID_ech', 'Description','Organo', 'Long_for', 'Refus','Ech_top', 'Ech_base', 'MS','Fract_2','Fract_2+']
prv_sol=col_ren(prv_sol, name=name, mode=1)

In [79]:
for i in range(len(prv_sol['Description'])):
    x = prv_sol.loc[i,'Description']
    if x=='R': prv_sol.loc[i,'Description']='Remblais'
    elif x=='L': prv_sol.loc[i,'Description']='Limons'

prv_sol['Refus']=prv_sol['Refus'].apply(lambda x: 'x' if not re.search('x|X', str(x)) else '')
prv_sol.insert(1,'Type_ech','Sol')

In [80]:
dataframe_viewer(prv_sol, rows=3)

Rows : 29, columns : 11


interactive(children=(IntSlider(value=3, description='rows', max=29, min=3, readout=False), IntSlider(value=11…

In [81]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [82]:
an=col_ren(an, 1)

In [83]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [84]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Sol')


Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX totaux', 'PHENOLS', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', 'col_63', 'EOX (****)', 'HYDROCARBURES TOTAUX', 'fraction aromat. C6-C7', 'fraction aromat. C7-C8', 'fraction aromat. C8-C10', 'fraction aliphat. C5-C6', 'fraction aliphat. C6-C8', 'fraction aliphat. C8-C10', 'Fraction C35 - C40', 'Hydrocarbures totaux C10-C40', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYL (PCB)', 'PCB 28', 'PCB 52', 'PCB 101', 'PCB 118', 'PCB 138', 'PCB 153', 'PCB 180', 'PCB totaux (7)', 'F4/2M*', 'Teneur mesurée', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = Etude d'orientation; EC = Etude de caractérisation; SA = Suivi d'assainissement", '(**) L: limon, A: Argile, S: Sable, R: Remblai', '(***) ib : imperméable (béton) ; ih : imperméable hydrocarboné ; p : perméable (gra

In [85]:
an=col_ren(an, name=pol_field_model, mode=1)

In [86]:
dataframe_viewer(prv_sol, rows=3)

Rows : 29, columns : 11


interactive(children=(IntSlider(value=3, description='rows', max=29, min=3, readout=False), IntSlider(value=11…

In [87]:
dataframe_viewer(an, rows=5) 

Rows : 29, columns : 64


interactive(children=(IntSlider(value=5, description='rows', max=29, min=5, readout=False), IntSlider(value=12…

In [88]:
source_prv_sol=prv_sol
source_an=an

In [89]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:29 ;
source_prv_sol:29 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


In [90]:
excel_soil_an, conflict_df = data_merger(source_an, source_prv_sol, how='outer', on='ID_ech', dist_max=1., drop_skip_col=['index'])

In [91]:
excel_soil_an['ID'] = excel_soil_an['ID_ech'].apply(lambda x: x.split('/')[0]) 

In [92]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
#excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
#excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
#excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
#excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
#excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
#excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)

excel_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)

* **Sheet : 'Résult EAU'**

In [93]:
tmp_dir='../../CF_data/Result_traitem/Phase_1_Memoris/'
sheet='Result_eau'

In [94]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/résultats phase 1/'
                   'Résultats phase 1_MEMORIS.xls', sheet_name='Résult EAU', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)
dataframe_viewer(df, rows=5)

1 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 136, columns : 23


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=136, min=5, readout=False), IntSlider(value=1…

In [95]:
prv_eau=df.loc[:32]
an=df.loc[33:]

In [96]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [97]:
prv_eau=prv_eau.transpose()
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=col_ren(prv_eau, 1)

In [98]:
prv_eau['CE']=prv_eau['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [99]:
prv_eau.drop(list(range(5)), axis=0, inplace=True)
prv_eau.reset_index(drop=True, inplace=True)

In [100]:
prv_eau=dble_col_drop(prv_eau)

column(s) dropped: ['21:DESCRIPTION SOMMAIRE', '27:Prof. arrêt du forage']


In [101]:
prv_eau=na_col_drop(prv_eau,1)
prv_eau=na_line_drop(prv_eau,3)
prv_eau.reset_index(drop=True, inplace=True)


Columns dropped :['col_1', "Nature de l'étude (*)", 'Observations organoleptiques', 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (**)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'PARAMETRES PHYSICO-CHIMIQUES ', 'ORP']



In [102]:
name=['ID_ech','Date_prv','Num_maille','Affectation','X','Y','Zsol','Long_for','Prof_crep','Long_pz',
      'Niv_eau_sol','pH','CE','T']
prv_eau=col_ren(prv_eau, name=name, mode=1)
prv_eau.insert(1,'Type_ech','Eau')

In [103]:
prv_eau['Prof_crep'].replace('\[|\]','', regex=True, inplace=True)
for i in range(len(prv_eau)):
    c=prv_eau.loc[i,'Prof_crep']
    prv_eau.loc[i,'Equip_top']=c.split('-')[0]
    prv_eau.loc[i,'Equip_base']=c.split('-')[1]

prv_eau['Type_equip'] = 'Crepine'
prv_eau.drop(columns=['Prof_crep'], inplace=True)

In [104]:
prv_eau['ID_ech'].replace('Canne ', 'Can', inplace=True, regex=True)
prv_eau['ID_ech'].replace('\n', ' ', inplace=True, regex=True)

In [105]:
pz=prv_eau[['ID_ech','X', 'Y', 'Zsol', 'Long_for','Long_pz', 'Equip_top', 'Equip_base', 'Type_equip']]
pz.rename(columns={'ID_ech':'ID'}, inplace=True)
pz['Type'] = 'Piezo'

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [106]:
for i in range(len(pz.ID)):
    c=pz.loc[i, 'ID']
    pz.loc[i, 'ID']=re.search("(\w+\d+(?:\w)?)",c).group(1)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [107]:
pz.drop_duplicates(subset=['ID'], keep='first', ignore_index=True, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [108]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [109]:
an=col_ren(an, 1)

In [110]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [111]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)

In [112]:
an.drop(columns=an.columns[[-6,-5]], axis=1, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Eau')


Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'PHENOLS', 'crésols (total)', 'CHLOROPHENOLS', '2-chlorophénol', 'monochlorophénol total', 'dichlorophénol total', '2,4,5-trichlorophénol', '2,4,6-trichlorophénol', 'trichlorophénol total', '2,3,4,6- tétrachlorophénol', 'tétrachlorophénol total', 'pentachlorophénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOCHLORES VOLATILS', 'CHLOROBENZENES', 'monochlorobenzène', '1,3-dichlorobenzène', '1,2-dichlorobenzène', '1,4-Dichlorobenzène', '1,2,4,5- et 1,2,3,5-tétrachlorobenzènes', 'pentachlorobenzène', 'hexachlorobenzène', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYLS (PCB)', 'PCB totaux (7)', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = Etude d'orientation; EC = Etude de caractérisation; SA = Suivi d'assainissement", '(**) ib : imperméable (béton) ; ih : imperméable hydrocar

In [117]:
an=col_ren(an, name=pol_field_model, mode=1)

In [118]:
an.columns

Index(['ID_ech', 'Type_ech', 'As', 'Cd', 'Cr', 'Cr_VI', 'Cu', 'Hg', 'Pb', 'Ni',
       'Zn', 'CN_libre', 'CN_tot', 'CN_APE', 'CN_comp', 'thioCN', 'Bnz',
       'Toln', 'EthylBnz', 'O-Xyl', 'P-M-Xyl', 'Xyl', 'Styr', 'Phenol',
       'Naphta', 'Acenaphtyl', 'Acenaphtn', 'Fluorene', 'col_43', 'Anthrc',
       'Flranth', 'Pyr', 'Bnz(a)anthrc', 'Chrys', 'Bnz(b)flranth',
       'Bnz(k)flranth', 'Bnz(a)pyr', 'Dibnz(ah)anthrc', 'Bnz(ghi)peryl',
       'Indeno(1,2,3-cd)pyr', 'HAP_tot_EPA', '1,1-DCE', '1,2-DCE', '1,1-DCEn',
       'Cis-1,2-DCEn', '(cis,trans) 1,2-DCE_tot', 'Trans 1,2-DCEyl', 'DCM',
       '1,2-DCP', 'TetraCEyn', 'TCM', '1,1,1-TCE', '1,1,2-TCE', 'TCEyn',
       'Chloroforme', 'CVinyl', 'Arom_C6C7', 'Arom_C7C8', 'Arom_C8C10',
       'Aliphat_C5C6', 'Aliphat_C6C8', 'Aliphat_C8C10', 'Fract_C5C8',
       'Fract_C8C10', 'Fract_C10C12', 'Fract_C12C16', 'Fract_C16C21',
       'Fract_C21C35', 'HC_tot_C10C35', 'MTBE'],
      dtype='object')

In [119]:
dataframe_viewer(an, rows=5) 

Rows : 17, columns : 70


interactive(children=(IntSlider(value=5, description='rows', max=17, min=5, readout=False), IntSlider(value=12…

In [120]:
source_pz=pz
source_prv_eau=prv_eau
#source_an=source_an.append(an)

In [121]:
excel_water_an, conflict_df = data_merger(an, source_prv_eau, how='outer', on='ID_ech', dist_max=1., drop_skip_col=['index'])

In [122]:
dataframe_viewer(excel_water_an, rows=5)

Rows : 17, columns : 85


interactive(children=(IntSlider(value=5, description='rows', max=17, min=5, readout=False), IntSlider(value=12…

In [123]:
for i in range(len(excel_water_an.ID_ech)):
    c=excel_water_an.loc[i, 'ID_ech']
    excel_water_an.loc[i, 'ID']=re.search("(\w+\d+(?:\w)?)",c).group(1)

In [124]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
#excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
#excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
#excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
#excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
#excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
#excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)

excel_water_an.to_csv(save_dir+'Water_analysis.csv', index=False)

In [125]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:14 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:29 ;
source_prv_sol:29 ;source_prv_eau:17 ; source_mes_pz:0 ; source_mes_sol:0 ;


## 8-Résultats phase 2_MEMORIS.xls
* **Sheet : 'Résult SOL'**

In [126]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [127]:
tmp_dir='../../CF_data/Result_traitem/Phase_2_Memoris/'
sheet='Result_SOL'

In [128]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/résultats phase 2/'
                   'Résultats phase 2_MEMORIS.xls', sheet_name='Résult SOL', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)
dataframe_viewer(df, rows=5)

1 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 135, columns : 31


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=135, min=5, readout=False), IntSlider(value=1…

In [129]:
prv_sol=df.loc[:35]
an=df.loc[36:]

In [130]:
an.loc[0.5] = df.loc[1] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [131]:
prv_sol=prv_sol.transpose()
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=col_ren(prv_sol, 1)

In [132]:
prv_sol.drop(list(range(5)), axis=0, inplace=True)
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=na_col_drop(prv_sol,1)
prv_sol=na_line_drop(prv_sol,3)
prv_sol.reset_index(drop=True, inplace=True)


Columns dropped :['col_0', "Nom / description d'échantillon", "Nature de l'étude (*)", 'Terrain', 'Epaisseur de remblais', 'Epaisseur alluvions', "Nature de l'observation organoleptique", 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'zone', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'X Lambert', 'Y Lambert', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (***)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'Matières organiques', 'GRANULOMETRIE', 'Fraction argileuse']



In [133]:
name=['ID_ech', 'Date_prv', 'Description','Organo', 'Long_for', 'Refus','Ech_top', 'Ech_base', 'MS','Fract_2','Fract_2+']
prv_sol=col_ren(prv_sol, name=name, mode=1)

In [134]:
set(prv_sol['Description'])

{'L', 'LA', 'LS', 'R'}

In [135]:
for i in range(len(prv_sol['Description'])):
    x = prv_sol.loc[i,'Description']
    if x=='R': prv_sol.loc[i,'Description']='Remblais'
    elif x=='L': prv_sol.loc[i,'Description']='Limons'
    elif x=='LA': prv_sol.loc[i,'Description']='Limons et argiles'
    elif x=='LS': prv_sol.loc[i,'Description']='Limons et sables'

prv_sol['Refus']=prv_sol['Refus'].apply(lambda x: 'x' if not re.search('#', str(x)) else '')
prv_sol.insert(1,'Type_ech','Sol')

In [136]:
dataframe_viewer(prv_sol, rows=3)

Rows : 25, columns : 12


interactive(children=(IntSlider(value=3, description='rows', max=25, min=3, readout=False), IntSlider(value=12…

In [137]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [138]:
an=col_ren(an, 1)

In [139]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [140]:
an.drop(list(range(5)), axis=0, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Sol')


Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX totaux', 'PHENOLS', 'Phénol', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', '1,1-Dichloroéthane', '1,2-Dichloroéthane', '1,1-dichloroéthène', 'Cis-1,2-dichloroéthène', 'Trans 1,2-dichloroéthylène', 'Dichlorométhane', 'Totaux (cis,trans) 1,2-dichloroéthènes', '1,2-dichloropropane', 'Tétrachloroéthylène', 'Tétrachlorométhane', '1,1,1-Trichloroéthane', '1,1,2-Trichloroéthane', 'Trichloroéthylène', 'Chloroforme', 'Chlorure de vinyle', 'col_63', 'EOX (****)', 'HYDROCARBURES TOTAUX', 'Fraction C35 - C40', 'Hydrocarbures totaux C10-C40', 'METHYL-TERT-BUTYL-ETHER', 'MTBE', 'POLYCHLOROBIPHENYL (PCB)', 'PCB 28', 'PCB 52', 'PCB 101', 'PCB 118', 'PCB 138', 'PCB 153', 'PCB 180', 'PCB totaux (7)', 'F4/2M*', 'Teneur mesurée', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = 

In [141]:
an=col_ren(an, name=pol_field_model, mode=1)

In [142]:
dataframe_viewer(an, rows=5, cols=20) 

Rows : 25, columns : 53


interactive(children=(IntSlider(value=5, description='rows', max=25, min=5, readout=False), IntSlider(value=20…

In [143]:
source_prv_sol=prv_sol
source_an=an

In [144]:
excel_soil_an, conflict_df = data_merger(prv_sol, an, how='outer', on='ID_ech', dist_max=1., drop_skip_col=['index'])

In [145]:
excel_soil_an['ID'] = excel_soil_an['ID_ech'].apply(lambda x: x.split('/')[0]) 

In [146]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
#excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
#excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
#excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
#excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
#excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
#excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)

excel_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)

In [147]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:25 ;
source_prv_sol:25 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Résult EAU'**

In [148]:
tmp_dir='../../CF_data/Result_traitem/Phase_2_Memoris/'
sheet='Result_eau'

In [149]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/résultats phase 2/'
                   'Résultats phase 2_MEMORIS.xls', sheet_name='Résult EAU', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)


Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 138, columns : 17


interactive(children=(IntSlider(value=5, description='rows', max=138, min=5, readout=False), IntSlider(value=1…

In [150]:
prv_eau=df.loc[:32]
an=df.loc[33:]

In [151]:
an.loc[0.5] = df.loc[1] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [152]:
prv_eau=prv_eau.transpose()
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=col_ren(prv_eau, 1)

In [153]:
prv_eau['CE']=prv_eau['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [154]:
prv_eau.drop(list(range(5)), axis=0, inplace=True)
prv_eau.reset_index(drop=True, inplace=True)

In [155]:
prv_eau=dble_col_drop(prv_eau)

column(s) dropped: ['21:DESCRIPTION SOMMAIRE', '27:Prof. arrêt du forage']


In [156]:
prv_eau=na_col_drop(prv_eau,1)
prv_eau=na_line_drop(prv_eau,3)
prv_eau.reset_index(drop=True, inplace=True)


Columns dropped :['Nom du piézomètre', "Nature de l'étude (*)", 'Observations organoleptiques', 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'Numéro de maille', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (**)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'PARAMETRES PHYSICO-CHIMIQUES ', 'ORP']



In [157]:
prv_eau.columns

Index(['col_1', 'Date de prélèvement', 'Type d'affectation (Plan de secteur)',
       'X Lambert', 'Y Lambert', 'Z Sol', 'Prof. arrêt du forage',
       'Profondeur crépine ', 'Prof. piézo/sol mesurée sur site',
       'Niveau de la nappe/sol', 'pH', 'CE', 'T'],
      dtype='object')

In [158]:
name=['ID_ech', 'Date_prv','Affectation','X', 'Y','Zsol', 'Long_for','Prof_crep','Long_pz_sol',
      'Niv_eau_sol','pH', 'CE', 'T']
prv_eau=col_ren(prv_eau, name=name, mode=1)
prv_eau.insert(1,'Type_ech','Eau')

In [159]:
prv_eau['ID_ech'].replace('Canne ', 'Can', inplace=True, regex=True)
prv_eau['ID_ech'].replace('\n', ' ', inplace=True, regex=True)

In [160]:
prv_eau['Prof_crep'].replace('\[|\]','', regex=True, inplace=True)
for i in range(len(prv_eau)):
    c=prv_eau.loc[i,'Prof_crep']
    prv_eau.loc[i,'Equip_top']=c.split('-')[0]
    prv_eau.loc[i,'Equip_base']=c.split('-')[1]
    
prv_eau.drop(columns=['Prof_crep'], inplace=True)
prv_eau['Type_equip'] = 'Crepine'

In [161]:
prv_eau['ID_ech'].replace('Canne ', 'Can', inplace=True, regex=True)
prv_eau['ID_ech'].replace('\n', ' ', inplace=True, regex=True)

In [162]:
pz=prv_eau[['ID_ech', 'X', 'Y', 'Zsol', 'Long_for','Long_pz_sol', 'Equip_top', 'Equip_base']]
pz.rename(columns={'ID_ech':'ID'}, inplace=True)
pz['Type']='Piezo'

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [163]:
for i in range(len(pz.ID)):
    c=pz.loc[i, 'ID']
    pz.loc[i, 'ID']=re.search("(\w+\d+(?:\w)?)",c).group(1)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [164]:
pz.drop_duplicates(subset=['ID'], keep='first', ignore_index=True, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [165]:
prv_eau=prv_eau[['ID_ech', 'Date_prv', 'X', 'Y', 'Zsol','Niv_eau_sol', 'pH', 'CE', 'T','Affectation']]

In [166]:
dataframe_viewer(prv_eau, rows=5)

Rows : 11, columns : 10


interactive(children=(IntSlider(value=5, description='rows', max=11, min=5, readout=False), IntSlider(value=10…

In [167]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [168]:
an=col_ren(an, 1)

In [169]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [170]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)

In [171]:
an.drop(columns=an.columns[[-6,-5]], axis=1, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Eau')


Columns dropped :['METAUX LOURDS', 'Chrome VI', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'PHENOLS', 'crésols (total)', 'CHLOROPHENOLS', '2-chlorophénol', 'monochlorophénol total', 'dichlorophénol total', '2,4,5-trichlorophénol', '2,4,6-trichlorophénol', 'trichlorophénol total', '2,3,4,6- tétrachlorophénol', 'tétrachlorophénol total', 'pentachlorophénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOCHLORES VOLATILS', 'CHLOROBENZENES', 'monochlorobenzène', '1,3-dichlorobenzène', '1,2-dichlorobenzène', '1,4-Dichlorobenzène', '1,2,4,5- et 1,2,3,5-tétrachlorobenzènes', 'pentachlorobenzène', 'hexachlorobenzène', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYLS (PCB)', 'PCB totaux (7)', 'AUTRES ANALYSES ', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = Etude d'orientation; EC = Etude de caractérisation; SA = Suivi d'assainissement", '(**) ib : imperméable (b

In [172]:
an=col_ren(an, name=pol_field_model, mode=1)

In [173]:
dataframe_viewer(an, rows=5) 

Rows : 11, columns : 70


interactive(children=(IntSlider(value=5, description='rows', max=11, min=5, readout=False), IntSlider(value=12…

In [174]:
source_prv_eau=prv_eau
source_an=source_an.append(an)

In [175]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:36 ;
source_prv_sol:25 ;source_prv_eau:11 ; source_mes_pz:0 ; source_mes_sol:0 ;


In [176]:
excel_water_an, conflict_df = data_merger(an, source_prv_eau, how='outer', on='ID_ech', dist_max=1., drop_skip_col=['index'])

In [177]:
dataframe_viewer(excel_water_an, rows=5)

Rows : 11, columns : 79


interactive(children=(IntSlider(value=5, description='rows', max=11, min=5, readout=False), IntSlider(value=12…

In [178]:
for i in range(len(excel_water_an.ID_ech)):
    c=excel_water_an.loc[i, 'ID_ech']
    excel_water_an.loc[i, 'ID']=re.search("(\w+\d+(?:\w)?)",c).group(1)

In [179]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
#excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
#excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
#excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
#excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
#excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
#excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)

excel_water_an.to_csv(save_dir+'Water_analysis.csv', index=False)

## 9-Ensemble des résultats Memoris version Seafile.xls
* **Sheet : 'Résult SOL'**

In [114]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [115]:
tmp_dir='../../CF_data/Result_traitem/Memoris_seafile/'
sheet='Result_SOL'

In [116]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/'
                   'Ensemble des résultats Memoris version Seafile.xls', sheet_name='Résult SOL', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
dataframe_viewer(df, rows=5)

2 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 138, columns : 66


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=138, min=5, readout=False), IntSlider(value=1…

In [117]:
prv_sol=df.loc[:37]
an=df.loc[38:]

In [118]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [119]:
prv_sol=prv_sol.transpose()
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=col_ren(prv_sol, 1)

In [120]:
prv_sol.drop(list(range(5)), axis=0, inplace=True)
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=na_col_drop(prv_sol,1)
prv_sol=na_line_drop(prv_sol,3)
prv_sol.reset_index(drop=True, inplace=True)


Columns dropped :['col_1', "Nom / description d'échantillon", "Nature de l'étude (*)", 'Terrain', 'Epaisseur de remblais', 'Epaisseur alluvions', "Nature de l'observation organoleptique", 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'zone', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'X Lambert', 'Y Lambert', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (***)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'Matières organiques', 'GRANULOMETRIE', 'Fraction argileuse']



In [121]:
prv_sol.drop(columns=prv_sol.columns[[-3,-4]], axis=1, inplace=True)

In [122]:
name=['ID_ech', 'Date_prv', 'Description','Organo', 'Long_for', 'Refus','Ech_top', 'Ech_base', 'MS','Fract_2','Fract_2+']
prv_sol=col_ren(prv_sol, name=name, mode=1)

In [123]:
set(prv_sol['Description'])

{'L', 'LA', 'LS', 'R', 'R '}

In [124]:
for i in range(len(prv_sol['Description'])):
    x = prv_sol.loc[i,'Description']
    if x=='R' or x=='R ': prv_sol.loc[i,'Description']='Remblais'
    elif x=='L': prv_sol.loc[i,'Description']='Limons'
    elif x=='LA': prv_sol.loc[i,'Description']='Limons et argiles'
    elif x=='LS': prv_sol.loc[i,'Description']='Limons et sables'

prv_sol['Refus']=prv_sol['Refus'].apply(lambda x: 'x' if not re.search('#', str(x)) else '')
prv_sol.insert(1,'Type_ech','Sol')

In [125]:
dataframe_viewer(prv_sol, rows=3)

Rows : 60, columns : 12


interactive(children=(IntSlider(value=3, description='rows', max=60, min=3, readout=False), IntSlider(value=12…

In [126]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [127]:
an=col_ren(an, 1)

In [128]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [129]:
an=dble_col_drop(an)

column(s) dropped: []


In [130]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Sol')


Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX totaux', 'PHENOLS', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', 'col_63', 'EOX (****)', 'HYDROCARBURES TOTAUX', 'Hydrocarbures totaux C10-C40', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYL (PCB)', 'PCB 28', 'PCB 52', 'PCB 101', 'PCB 118', 'PCB 138', 'PCB 153', 'PCB 180', 'PCB totaux (7)', 'F4/2M*', 'Teneur mesurée', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = Etude d'orientation; EC = Etude de caractérisation; SA = Suivi d'assainissement", '(**) L: limon, A: Argile, S: Sable, R: Remblai', '(***) ib : imperméable (béton) ; ih : imperméable hydrocarboné ; p : perméable (gravier, fissuré,…) ; tvh : terres végétation haute ; tvb : terres végétation basse ', '(****) 3 mg/kg = Seuil limite défini dans le GREO ', "(1) l'échantillon n'a pas pu être extrait ni

In [131]:
an=col_ren(an, name=pol_field_model, mode=1)

In [132]:
dataframe_viewer(an, rows=5) 

Rows : 60, columns : 71


interactive(children=(IntSlider(value=5, description='rows', max=60, min=5, readout=False), IntSlider(value=12…

In [133]:
source_prv_sol=prv_sol
source_an=an

In [134]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:60 ;
source_prv_sol:60 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Résult EAU'**

In [135]:
tmp_dir='../../CF_data/Result_traitem/Memoris_seafile/'
sheet='Result_eau'

In [136]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/'
                   'Ensemble des résultats Memoris version Seafile.xls', sheet_name='Résult EAU', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

4 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 154, columns : 51


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=154, min=5, readout=False), IntSlider(value=1…

In [137]:
prv_eau=df.loc[:32]
an=df.loc[33:]

In [138]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [139]:
prv_eau=prv_eau.transpose()
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=col_ren(prv_eau, 1)

In [140]:
prv_eau['CE']=prv_eau['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [141]:
prv_eau.drop(list(range(5)), axis=0, inplace=True)
prv_eau.reset_index(drop=True, inplace=True)

In [142]:
prv_eau=dble_col_drop(prv_eau)

column(s) dropped: ['21:DESCRIPTION SOMMAIRE', '27:Prof. arrêt du forage']


In [143]:
prv_eau=na_col_drop(prv_eau,1)
prv_eau=na_line_drop(prv_eau,3)
prv_eau.reset_index(drop=True, inplace=True)


Columns dropped :['col_1', "Nature de l'étude (*)", 'Observations organoleptiques', 'LOCALISATION/AFFECTATION(S) - USAGE(S)', 'parcelle (selon dénom.)', "Type d'usage de fait actuel", "Type d'usage de fait futur", "Type d'usage utilisé pour comparer les résultats", 'Lieux de prélèvement', 'DESCRIPTION SOMMAIRE', 'Type de recouvrement de surface (**)', 'Sol en place (S : souillé, NS : non souillé)', 'Remblais de terre (S : souillé, NS : non souillé)', 'Remblais technique (S : souillé, NS : non souillé)', 'Déchet', 'PARAMETRES PHYSICO-CHIMIQUES ', 'ORP']



In [144]:
prv_eau.drop(columns=prv_eau.columns[[2]], axis=2, inplace=True)

In [145]:
name=['ID_ech', 'Date_prv','Affectation','X', 'Y','Zsol', 'Long_for','Prof_crep','Long_pz_sol', 
      'Niv_eau_sol','pH', 'CE', 'T']
prv_eau=col_ren(prv_eau, name=name, mode=1)
prv_eau.insert(1,'Type_ech','Eau')

In [146]:
prv_eau['Prof_crep'].replace('\[|\]','', regex=True, inplace=True)
for i in range(len(prv_eau)):
    c=prv_eau.loc[i,'Prof_crep']
    prv_eau.loc[i,'Equip_top']=c.split('-')[0]
    prv_eau.loc[i,'Equip_base']=c.split('-')[1]
    
prv_eau.drop(columns=['Prof_crep'], inplace=True)
prv_eau['Type_equip'] = 'Crepine'

In [147]:
prv_eau['ID_ech'].replace('Canne ', 'Can', inplace=True, regex=True)
prv_eau['ID_ech'].replace('\n', ' ', inplace=True, regex=True)

In [148]:
pz=prv_eau[['ID_ech', 'X', 'Y', 'Zsol', 'Long_for','Long_pz_sol', 'Equip_top', 'Equip_base', 'Type_equip']]
pz.rename(columns={'ID_ech':'ID'}, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [149]:
for i in range(len(pz.ID)):
    c=pz.loc[i, 'ID']
    pz.loc[i, 'ID']=re.search("(\w+\d+)",c).group(1)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [150]:
pz.drop_duplicates(subset=['ID'], keep='first', ignore_index=True, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [151]:
prv_eau=prv_eau[['ID_ech', 'Date_prv', 'X', 'Y', 'Zsol','Niv_eau_sol', 'pH', 'CE', 'T','Affectation']]

In [152]:
dataframe_viewer(prv_eau, rows=3)

Rows : 45, columns : 10


interactive(children=(IntSlider(value=3, description='rows', max=45, min=3, readout=False), IntSlider(value=10…

In [153]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [154]:
an=col_ren(an, 1)

In [155]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [156]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)

In [157]:
an=dble_col_drop(an)

column(s) dropped: ['104:nitrite', '106:nitrate', '112:ammonium', '117:Teneur mesurée']


In [158]:
an.drop(columns=an.columns[[-6,-5]], axis=1, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Eau')


Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'PHENOLS', 'crésols (total)', 'CHLOROPHENOLS', '2-chlorophénol', 'monochlorophénol total', 'dichlorophénol total', '2,4,5-trichlorophénol', '2,4,6-trichlorophénol', 'trichlorophénol total', '2,3,4,6- tétrachlorophénol', 'tétrachlorophénol total', 'pentachlorophénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOCHLORES VOLATILS', 'CHLOROBENZENES', 'monochlorobenzène', '1,3-dichlorobenzène', '1,2-dichlorobenzène', '1,4-Dichlorobenzène', '1,2,4,5- et 1,2,3,5-tétrachlorobenzènes', 'pentachlorobenzène', 'hexachlorobenzène', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYLS (PCB)', 'PCB totaux (7)', 'AUTRES ANALYSES ', 'azote Kjeldahl', 'COMPOSES INORGANIQUES ', 'sulfures totaux', 'Teneur mesurée (souligne/gras)', 'Teneur mesurée (gras/grisé)', "(*) CP = Campagne de prélèvements; Ex = Expertise; ES = Etude de sol; EO = Etude d'orientation; EC = Etude de caractérisation; SA = 

In [159]:
an['ID_ech'].replace('Canne ', 'Can', inplace=True, regex=True)
an['ID_ech'].replace('\n', ' ', inplace=True, regex=True)

In [160]:
an=col_ren(an, name=pol_field_model, mode=1)

In [161]:
dataframe_viewer(an, rows=5) 

Rows : 45, columns : 80


interactive(children=(IntSlider(value=5, description='rows', max=45, min=5, readout=False), IntSlider(value=12…

In [162]:
source_pz=pz
source_prv_eau=prv_eau
source_an=source_an.append(an)

In [163]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:30 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:105 ;
source_prv_sol:60 ;source_prv_eau:45 ; source_mes_pz:0 ; source_mes_sol:0 ;


## 10-Résultats SOL container phyto t=0_décret sol.xls
* **Sheet : 'Résult SOL'**

In [164]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [165]:
tmp_dir='../../CF_data/Result_traitem/Container_phyto/'
sheet='Result_SOL'

In [166]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/'
                   'Résultats SOL container phyto t=0_décret sol.xls', sheet_name='Résult SOL', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

2 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 16', 'Unnamed: 17']

Rows : 121, columns : 15


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=121, min=5, readout=False), IntSlider(value=1…

In [167]:
prv_sol=df.loc[:21]
an=df.loc[22:]

In [168]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [169]:
prv_sol=prv_sol.transpose()
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=col_ren(prv_sol, 1)

In [170]:
prv_sol=dble_col_drop(prv_sol)

column(s) dropped: ['8:Autre zone suspecte investiguée']


In [171]:
prv_sol.drop(list(range(5)), axis=0, inplace=True)
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=na_col_drop(prv_sol,2)
prv_sol=na_line_drop(prv_sol,3)
prv_sol.reset_index(drop=True, inplace=True)


Columns dropped :['Matières organiques', 'SPP/zone suspecte invetiguée', 'Autre zone suspecte investiguée', 'Parcelle', 'X Lambert', 'Y Lambert', "Type d'étude (*)", 'Refus de forage', 'Terrain naturel/Remblai (**)', 'Organoleptique couleur suspecte', 'Organoleptique odeur intensité (***)', 'Organoleptique odeur type', 'GRANULOMETRIE']

4 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [172]:
prv_sol.drop(columns=prv_sol.columns[[-3]], axis=1, inplace=True)

In [173]:
name=['ID_ech', 'Ech_top', 'Ech_base','MS','Date_prv','Fract_2','Fract_2+']
prv_sol=col_ren(prv_sol, name=name, mode=1)
prv_sol=prv_sol.query('ID_ech==ID_ech')
prv_sol.insert(1,'Type_ech','Sol')

In [174]:
dataframe_viewer(prv_sol, rows=3)

Rows : 5, columns : 8


interactive(children=(IntSlider(value=3, description='rows', max=5, min=3, readout=False), IntSlider(value=8, …

In [175]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [176]:
an=col_ren(an, 1)

In [177]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [178]:
an=dble_col_drop(an)

column(s) dropped: ['92:Teneur mesurée', '93:Teneur mesurée', '94:Teneur mesurée']


In [179]:
an.drop(list(range(5)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Sol')


Columns dropped :['METAUX LOURDS', 'Cobalt', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX totaux', 'PHENOLS', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', 'col_64', 'EOX (****)', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER', 'POLYCHLOROBIPHENYL (PCB)', 'PCB 28', 'PCB 52', 'PCB 101', 'PCB 118', 'PCB 138', 'PCB 153', 'PCB 180', 'PCB totaux (7)', 'Teneur mesurée', "VR : Valeur de référence; VS : Valeur seuil; VI : Valeur d'intervention", "(*) RP : Rapport de prélèvements; ES : Etude de sol; EO : Etude d'orientation; EC : Etude de caractérisation; SA : Suivi d'assainissement; EF : Evaluation finale", '(**) TN : Terrain naturel; R : Remblai', "(***) - : Pas d'impression organoleptique; + : Impression organoleptique faible; ++ : Impression organoleptique forte", '(****) 3 mg/kg = Seuil limite défini dans le GREO ']



In [180]:
an=col_ren(an, name=pol_field_model, mode=1)

In [181]:
dataframe_viewer(an, rows=5) 

Rows : 9, columns : 70


interactive(children=(IntSlider(value=5, description='rows', max=9, min=5, readout=False), IntSlider(value=12,…

In [182]:
source_prv_sol=prv_sol
source_an=an

In [183]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:9 ;
source_prv_sol:5 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Paramètres agro.'**

In [184]:
tmp_dir='../../CF_data/Result_traitem/Container_phyto/'
sheet='Param_agro'

In [185]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/'
                   'Résultats SOL container phyto t=0_décret sol.xls', sheet_name='Paramètres agro.', skiprows=4)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

1 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2']

Rows : 28, columns : 10


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=28, min=5, readout=False), IntSlider(value=10…

In [186]:
df=df.transpose()
df.reset_index(drop=True, inplace=True)
df=col_ren(df, 0)

In [187]:
df.drop(list(range(1)), axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [188]:
df=dble_col_drop(df)

column(s) dropped: ['22:température pour mes. pH']


In [189]:
df=na_col_drop(df,1)
df=na_line_drop(df,3)
df.reset_index(drop=True, inplace=True)


Columns dropped :['Matières organiques', 'GRANULOMETRIE', 'pH', 'Composés inorganiques ', 'Autres analyses chimiques ']

3 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [190]:
df.drop(columns=df.columns[[5,6]], axis=2, inplace=True)

In [681]:
df.columns

Index(['ID_ech', 'Type_ech', 'Periode', 'Emplacement', 'Date_prv',
       'Niv_eau_pz', 'Long_pz', 'Temp_prv ', 'pH', 'CE', 'ORP', 'O_diss',
       'Rmq', 'line_na'],
      dtype='object')

In [192]:
name=['ID_ech','Ech_top','Ech_base','MS','Date_prv','MO','Residu_perte_feu','COT','Fract_arg','Fract_min_2µ', 
      'Fract_min_50µ', 'Fract_min_2', 'Fract_2', 'Fract_2+', 'pH_KCl','Tem_pH_mes', 'pH_H20', 'sulfures_tot', 
      'chlorures', 'azote_Kjeldahl']
df=col_ren(df, name=name, mode=1)
df.insert(1,'Type_ech','Sol')

In [193]:
prv_sol=df

In [194]:
dataframe_viewer(prv_sol, rows=5)

Rows : 5, columns : 21


interactive(children=(IntSlider(value=5, description='rows', max=5, min=5, readout=False), IntSlider(value=12,…

In [195]:
data_merger(source_prv_sol, prv_sol, on='ID_ech', how='outer', )[0]

TypeError: data_merger() got an unexpected keyword argument 'col'

In [ ]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
#source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

--------------------------------------------------------------------------------------------------------

## 11-Resultats_Siterem_eau_extension_pilote_jusque_decembre_2020.xlsx
* **Sheet : 'Résult EAU'**

In [180]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [181]:
tmp_dir='../../CF_data/Result_traitem/Siterem_Ext_Pilote/'
sheet='Result_eau'

In [182]:
df = pd.read_excel('../../CF_data/Data_SITEREM/Resultats_Siterem_eau_extension_pilote_jusque_decembre_2020.xlsx', 
                   sheet_name='Résult EAU', skiprows=2)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

5 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 115, columns : 37


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=115, min=5, readout=False), IntSlider(value=1…

In [183]:
prv_eau=df.loc[:31]
an=df.loc[list(range(0,4))+list(range(32, len(df)))]

In [184]:
an = an.sort_index().reset_index(drop=True)

In [185]:
prv_eau=prv_eau.transpose()
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=col_ren(prv_eau, 1)

In [186]:
prv_eau=dble_col_drop(prv_eau)

column(s) dropped: ['6:Autre zone suspecte investiguée', '25:pH', '30:pH']


In [187]:
prv_eau.drop(list(range(5)), axis=0, inplace=True)
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=na_col_drop(prv_eau,2)
prv_eau=na_line_drop(prv_eau,3)
prv_eau.reset_index(drop=True, inplace=True)


Columns dropped :['SPP/zone suspecte investiguée', 'Autre zone suspecte investiguée', 'Parcelle', 'X Lambert', 'Y Lambert', "Type d'étude (*)", 'Profondeur minimum de la crépine', 'Profondeur maximum de la crépine', 'Profondeur du fond du piézomètre', 'Profondeur de la nappe/piezo', 'Profondeur de la nappe/chambre visite', 'Organoleptique odeur intensité (**)', 'Organoleptique odeur type', 'Limpidité (***)', 'Température', 'Conductivité électrique', 'Profondeur de la nappe/sol ', 'PARAMETRES PHYSICO-CHIMIQUES \n(lors du prélèvement)', 'CE', 'ORP', 'Oxygène dissous', 'PARAMETRES PHYSICO-CHIMIQUES \n(mesures au labo)']



In [188]:
name=['ID_ech', 'Periode', 'Emplacement','Date_prv','pH','Temp_prv','Temp_pH_mes']
prv_eau=col_ren(prv_eau, name=name, mode=1)
prv_eau=prv_eau.query('ID_ech==ID_ech')
prv_eau.insert(1,'Type_ech','Eau')

In [189]:
dataframe_viewer(prv_eau, rows=3)

Rows : 31, columns : 8


interactive(children=(IntSlider(value=3, description='rows', max=31, min=3, readout=False), IntSlider(value=8,…

In [190]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [191]:
an=col_ren(an, 1)

In [192]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [193]:
an=dble_col_drop(an)

column(s) dropped: []


In [194]:
# rename in a first time before dropping columns (because of names like 'col_xx' in columns)
name=['ID_ech', 'Periode', 'Emplacement','Date_prv', 'METAUX LOURDS', 'Arsenic', 'Cadmium', 'Chrome', 'Cuivre',
      'Mercure', 'Plomb', 'Nickel', 'Zinc', 
      'CYANURES',"Cyanures (libres)  -  NEN-EN-ISO 14403", "CN_totaux - NEN-EN-ISO 14403", 
      "cyanure (APE) - méthode basée sur EPA 335.3", "cyanure complex - méthode interne ", 
      "thiocyanate - méthode interne", "COMPOSES AROMATIQUES VOLATILS", "Benzène", "Toluène", "Éthylbenzène",
      "Orthoxylène", "Para- et métaxylène", "Xylènes", "Styrène", "PHENOLS", "Phénol", "Indice phénol", 
      "HYDROCARBURES AROMATIQUES POLYCYCLIQUES", "Naphtalène", "Acénaphtylène", "Acénaphtène", "Fluorène", 
      "Phénanthrène", "Anthracène", "Fluoranthène", "Pyrène", "Benzo(a)anthracène", "Chrysène", 
      "Benzo(b)fluoranthène", "Benzo(k)fluoranthène", "Benzo(a)pyrène", "Dibenzo(ah)anthracène", 
      "Benzo(ghi)pérylène", "Indéno(1,2,3-cd)pyrène", "HAP Totaux (16) - EPA", 
      "COMPOSES ORGANOCHLORES VOLATILS", "1,1-Dichloroéthane", "1,2-Dichloroéthane", 
      "1,1-dichloroéthène", "Cis-1,2-dichloroéthène", "Totaux (cis,trans) 1,2-dichloroéthènes", 
      "Trans 1,2-dichloroéthylène", "Dichlorométhane", "1,2-dichloropropane", "Tétrachloroéthylène ", 
      "Tétrachlorométhane", "1,1,1-Trichloroéthane", "1,1,2-Trichloroéthane", "Trichloroéthylène",
      "Chloroforme", "Chlorure de vinyle", "HYDROCARBURES TOTAUX", "fraction aromat. >C6-C7", 
      "fraction aromat. >C7-C8", "fraction aromat. >C8-C10", "fraction aliphat. C5-C6", "fraction aliphat. >C6-C8",
      "fraction aliphat. >C8-C10", "Fraction C5 - C8", "Fraction C8 - C10", "Fraction C10-C12", "Fraction C12-C16",
      "Fraction C16 - C21", "Fraction C21 - C35", "Hydrocarbures totaux C10-C35", "METHYL-TERT-BUTYL-ETHER", "MTBE",
     "a","b","c","d","e","f","g"]

an=col_ren(an, name=name, mode=1)
an=an.iloc[:,:-7]

In [195]:
an.drop(list(range(3)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Eau')


Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'Styrène', 'PHENOLS', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOCHLORES VOLATILS', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER', 'MTBE']



In [196]:
an=col_ren(an, name=pol_field_model, mode=1)

In [197]:
dataframe_viewer(an, rows=3)

Rows : 33, columns : 70


interactive(children=(IntSlider(value=3, description='rows', max=33, min=3, readout=False), IntSlider(value=12…

In [198]:
source_prv_eau=prv_eau
source_an=an

In [199]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:33 ;
source_prv_sol:0 ;source_prv_eau:31 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Param physico'**

In [200]:
tmp_dir='../../CF_data/Result_traitem/Siterem_Ext_Pilote/'
sheet='Param_physico'

In [201]:
df = pd.read_excel('../../CF_data/Data_SITEREM/Resultats_Siterem_eau_extension_pilote_jusque_decembre_2020.xlsx', 
                   sheet_name='param. physico', skiprows=2)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

8 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 53, columns : 77


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=53, min=5, readout=False), IntSlider(value=12…

In [202]:
df=df.transpose()
df.reset_index(drop=True, inplace=True)

In [203]:
df=col_ren(df, 1)

In [204]:
df.drop(list(range(2)), axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [205]:
sdf=df.iloc[:,:33]
df=df.iloc[:,34:]

In [206]:
df=dble_col_drop(df)
sdf=dble_col_drop(sdf)

column(s) dropped: ['3:Période ']
column(s) dropped: ['6:Autre zone suspecte investiguée', '26:pH', '32:pH']


In [207]:
df=na_line_drop(df,1)
sdf=na_line_drop(sdf,1)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


41 NaN lines dropped
27 NaN lines dropped


In [208]:
df=na_col_drop(df,1)
sdf=na_col_drop(sdf,1)


Columns dropped :['Profondeur de la nappe/chambre visite', 'Organoleptique odeur intensité (**)', 'Organoleptique odeur type', 'Profondeur de la nappe/sol ', 'PARAMETRES PHYSICO-CHIMIQUES \n(lors du prélèvement)', "*paramètres n'ont pas été pris en débit continu (dans seau) - peu de débit", 'col_52']


Columns dropped :['SPP/zone suspecte investiguée', 'Autre zone suspecte investiguée', 'Parcelle', 'X Lambert', 'Y Lambert', "Type d'étude (*)", 'Profondeur minimum de la crépine', 'Profondeur maximum de la crépine', 'Profondeur du fond du piézomètre', 'Organoleptique odeur intensité (**)', 'Organoleptique odeur type', 'Limpidité (***)', 'Température', 'Conductivité électrique', 'PARAMETRES PHYSICO-CHIMIQUES \n(lors du prélèvement)', 'PARAMETRES PHYSICO-CHIMIQUES \n(mesures au labo)']



In [209]:
name=['ID_ech','Periode','Emplacement','Date_prv','Niv_eau_pz','Long_pz','Temp_prv ','pH', 'CE', 'ORP','O_diss']
df=col_ren(df, mode=1, name=name)

In [210]:
sdf=sdf.iloc[:,:-1]
name=['ID_ech','Periode','Emplacement','Date_prv','Niv_eau_pz','Niv_eau_chbre','pH','Niv_eau_sol','Long_pz',
      'Temp_prv ','CE','ORP','O_diss']
sdf=col_ren(sdf, mode=1, name=name)

In [211]:
df['CE']=df['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)
sdf['CE']=sdf['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [212]:
sdf['Periode'].replace('\n',' ', regex=True, inplace=True)
sdf.replace('\n','', regex=True, inplace=True)

In [213]:
data=[df, sdf]
for d in data:
    d['Rmq']=''
    for i in range(len(d['ID_ech'])):
        e=str(d.loc[i, 'Emplacement'])
        n=str(d.loc[i, 'ID_ech'])
        d.loc[i,'ID_ech']=n.replace('*', '')
        
        if re.match('S',e, re.I): 
            d.loc[i,'Emplacement']='Simulateur'
        elif re.match('HZS',e, re.I): 
            d.loc[i,'Emplacement']='Hors simulateur'
        else:
            d.loc[i,'Emplacement']=np.nan
        
        if re.match('\d+\*{1}$',n, re.I): 
            d.loc[i,'Rmq']="mesures faites dans un seau (débit non continu ou peu de débit)"
        elif re.match('\d+\*{2}$',n, re.I): 
            d.loc[i,'Rmq']="mésures faites dans une eau quasi-stagnante (Piezo rempli de sédiment et débit très faible)"

In [214]:
df.insert(1, 'Type_ech', 'Eau')
sdf.insert(1, 'Type_ech', 'Eau')

In [215]:
prv_eau=data_merger(sdf, df, 'outer', 'ID_ech')[0]

In [216]:
prv_eau=na_col_drop(df,2)
prv_eau=na_line_drop(df,1)
prv_eau.reset_index(drop=True, inplace=True)

In [217]:
for i in prv_eau.index:
    if not pd.isnull(prv_eau.loc[i, 'Emplacement']):
        val = prv_eau.loc[i, 'Emplacement']
    else:
        prv_eau.loc[i, 'Emplacement'] = val

In [218]:
dataframe_viewer(prv_eau, rows=3)

Rows : 33, columns : 13


interactive(children=(IntSlider(value=3, description='rows', max=33, min=3, readout=False), IntSlider(value=12…

In [219]:
source_prv_eau, conflict_df = data_merger(source_prv_eau, prv_eau, on=['ID_ech', 'Date_prv'], how='outer')

Conflict values present. Please resolve this manually !


In [220]:
dataframe_viewer(conflict_df, rows=3)

Rows : 21, columns : 9


interactive(children=(IntSlider(value=3, description='rows', max=21, min=3, readout=False), IntSlider(value=9,…

In [221]:
data_validation(overall_data=source_prv_eau, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Emplacement_x':list(conflict_df.index), 'pH_y':list(conflict_df.index), 
                           'Periode_x':list(conflict_df.index)})

all conflicts have been fixed!


In [222]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
#source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:33 ;
source_prv_sol:0 ;source_prv_eau:43 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Inorganiques et composés majeurs'**

In [223]:
tmp_dir='../../CF_data/Result_traitem/Siterem_Ext_Pilote/'
sheet='Inorganic_major'

In [224]:
df = pd.read_excel('../../CF_data/Data_SITEREM/Resultats_Siterem_eau_extension_pilote_jusque_decembre_2020.xlsx', 
                   sheet_name='inorganiques et composés majeur', skiprows=2)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

3 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34']

Rows : 68, columns : 28


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=68, min=5, readout=False), IntSlider(value=12…

In [225]:
prv_eau=df.loc[:21]
an=df.loc[list(range(0,4))+list(range(22, len(df)))]

In [226]:
an = an.sort_index().reset_index(drop=True)

In [227]:
prv_eau=prv_eau.transpose()
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=col_ren(prv_eau, 1)

In [228]:
prv_eau=dble_col_drop(prv_eau)

column(s) dropped: ['6:Autre zone suspecte investiguée']


In [229]:
prv_eau.drop(list(range(2)), axis=0, inplace=True)
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=na_col_drop(prv_eau,2)
prv_eau=na_line_drop(prv_eau,2)
prv_eau.reset_index(drop=True, inplace=True)


Columns dropped :['SPP/zone suspecte investiguée', 'Autre zone suspecte investiguée', 'Parcelle', 'X Lambert', 'Y Lambert', "Type d'étude (*)", 'Profondeur minimum de la crépine', 'Profondeur maximum de la crépine', 'Profondeur du fond du piézomètre', 'Profondeur de la nappe', 'Organoleptique odeur intensité (**)', 'Organoleptique odeur type', 'Limpidité (***)', 'Température', 'Conductivité électrique', 'pH']

1 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [230]:
prv_eau.columns

Index(['Nom échantillon', 'Période ', 'Emplacement ', 'Date de prélèvement',
       'Température de prélèvement '],
      dtype='object')

In [231]:
name=['ID_ech', 'Periode', 'Emplacement','Date_prv','Temp_prv']
prv_eau=col_ren(prv_eau, name=name, mode=1)
prv_eau=prv_eau.query('ID_ech==ID_ech')
prv_eau.insert(1,'Type_ech','Eau')

In [232]:
dataframe_viewer(prv_eau, rows=3)

Rows : 24, columns : 6


interactive(children=(IntSlider(value=3, description='rows', max=24, min=3, readout=False), IntSlider(value=6,…

In [233]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [234]:
an=col_ren(an, 1)

In [235]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [236]:
an=dble_col_drop(an)

column(s) dropped: ['11:ammonium', '15:nitrite', '17:nitrate']


In [237]:
an=na_col_drop(an,3)


Columns dropped :['CARBONE ORGANIQUE', 'DEMANDE EN O2', 'COMPOSES AZOTES', 'COMPOSES SOUFRES ', 'ELEMENTS MAJEURS', 'AUTRES ANALYSES', 'cyanure (libre)', 'METHYL-TERT-BUTYL-ETHER', 'MTBE', 'Teneur mesurée ', 'Teneur mesurée', 'VS : Valeur seuil', "(*) RP : Rapport de prélèvements; ES : Etude de sol; EO : Etude d'orientation; EC : Etude de caractérisation; SA : Suivi d'assainissement; EF : Evaluation finale", "(**) - : Pas d'impression organoleptique; + : Impression organoleptique faible; ++ : Impression organoleptique forte", '(***) + : Limpide; - : Trouble; -- : Opaque', "Le contenu des tableaux est conforme au modèle repris à l'annexe IX du GREO V03. Le formalisme a été adapté par SITEREM tout en garantissant la lisibilité du document imprimé. "]



In [238]:
an.columns

Index(['ID_ech', 'Période ', 'Emplacement ', 'Date de prélèvement', 'COT',
       'DBO (5 jours)', 'DCO', 'ammonium', 'ammoniaque - libre',
       'azote Kjeldahl', 'nitrite', 'nitrate', 'sulfures totaux',
       'sulfures (libre)', 'Soufre Total', 'sulfites', 'sulfate', 'calcium',
       'potassium', 'magnésium', 'manganèse', 'sodium', 'fer',
       'fer (Fe) total', 'fer (2+)', 'chlorures', 'fluorures',
       'bromure (libre)', 'phosphore (total)', 'carbonate', 'bicarbonate'],
      dtype='object')

In [239]:
an.rename(columns={'ammoniaque - libre':'ammoniaque libre'}, inplace=True)

In [240]:
an.drop(list(range(2)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Eau')

In [241]:
an=col_ren(an, name=pol_field_model, mode=1)
an.rename(columns={'Période ':'Periode', 'Date de prélèvement':'Date_prv'}, inplace=True)
#an=an.iloc[:,:-7]

In [242]:
dataframe_viewer(an, rows=3)

Rows : 25, columns : 32


interactive(children=(IntSlider(value=3, description='rows', max=25, min=3, readout=False), IntSlider(value=12…

In [243]:
dataframe_viewer(source_prv_eau, rows=3)

Rows : 43, columns : 15


interactive(children=(IntSlider(value=3, description='rows', max=43, min=3, readout=False), IntSlider(value=12…

In [244]:
source_prv_eau.Date_prv = source_prv_eau.Date_prv.astype(object)

In [245]:
source_prv_eau, conflict_df=data_merger(source_prv_eau, prv_eau, 'outer', ['ID_ech', 'Date_prv'])

Conflict values present. Please resolve this manually !


In [246]:
data_validation(overall_data=source_prv_eau, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Periode_y':list(conflict_df.index), 'Emplacement_y':list(conflict_df.index)})

all conflicts have been fixed!


In [247]:
source_an, conflict_df=data_merger(source_an, an, 'outer', ['ID_ech', 'Date_prv'])

In [248]:
source_an.Date_prv = source_an.Date_prv.astype(object)

In [249]:
dataframe_viewer(source_an, rows=5, cols=13), dataframe_viewer(source_prv_eau, rows=5, cols=13)

Rows : 38, columns : 98


interactive(children=(IntSlider(value=5, description='rows', max=38, min=5, readout=False), IntSlider(value=13…

Rows : 43, columns : 15


interactive(children=(IntSlider(value=5, description='rows', max=43, min=5, readout=False), IntSlider(value=13…

(None, None)

In [250]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:38 ;
source_prv_sol:0 ;source_prv_eau:43 ; source_mes_pz:0 ; source_mes_sol:0 ;


### $\color{red}{\textbf{Excel sheets data merging}}$

In [251]:
excel_bh_water_an, conflict_df = data_merger(source_an, source_prv_eau, how='outer', on=['ID_ech', 'Date_prv'], drop_skip_col=['index'])

In [252]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
#excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
#excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
#excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
#excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
#excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)

## 12-Resultats_Siterem_eau_pilote_jusque_decembre_2020.xlsx
* **Sheet : 'Résult EAU'**

In [253]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [254]:
tmp_dir='../../CF_data/Result_traitem/Siterem_Pilote/'
sheet='Result_eau'

In [255]:
df = pd.read_excel('../../CF_data/Data_SITEREM/Resultats_Siterem_eau_pilote_jusque_decembre_2020.xlsx', 
                   sheet_name='Résult EAU', skiprows=2)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

3 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 117, columns : 91


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=117, min=5, readout=False), IntSlider(value=1…

In [256]:
prv_eau=df.loc[:32]
an=df.loc[list(range(0,4))+list(range(33, len(df)))]

In [257]:
an = an.sort_index().reset_index(drop=True)

In [258]:
prv_eau=prv_eau.transpose()
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=col_ren(prv_eau, 1)

In [259]:
prv_eau=dble_col_drop(prv_eau)

column(s) dropped: ['6:Autre zone suspecte investiguée', '25:pH', '31:pH']


In [260]:
prv_eau.drop(list(range(3)), axis=0, inplace=True)
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=na_col_drop(prv_eau,2)
prv_eau=na_line_drop(prv_eau,3)
prv_eau.reset_index(drop=True, inplace=True)


Columns dropped :['SPP/zone suspecte investiguée', 'Autre zone suspecte investiguée', 'Parcelle', 'X Lambert', 'Y Lambert', "Type d'étude (*)", 'Profondeur minimum de la crépine', 'Profondeur maximum de la crépine', 'Profondeur du fond du piézomètre', 'Organoleptique odeur intensité (**)', 'Organoleptique odeur type', 'Limpidité (***)', 'Température', 'Conductivité électrique', 'Profondeur de la nappe/sol ', 'PARAMETRES PHYSICO-CHIMIQUES \n(lors du prélèvement)', 'PARAMETRES PHYSICO-CHIMIQUES \n(mesures au labo)']



In [261]:
prv_eau.columns

Index(['Nom échantillon', 'Période ',
       'Emplacement \n- P : Pilote \n- HZP : Hors zone pilote',
       'Date de prélèvement', 'Profondeur de la nappe/piezo',
       'Profondeur de la nappe/chambre visite', 'pH',
       'Température de prélèvement ', 'CE', 'ORP', 'Oxygène dissous', 'col_29',
       'température pour mes. pH'],
      dtype='object')

In [262]:
name=['ID_ech', 'Periode', 'Emplacement','Date_prv','Niv_eau_pz','Niv_eau_chbre','pH','Temp_prv','CE','ORP',
      'O_diss','col_29','Temp_pH_mes']
prv_eau=col_ren(prv_eau, name=name, mode=1)
prv_eau=prv_eau.query('ID_ech==ID_ech')
prv_eau.insert(1,'Type_ech','Eau')

In [263]:
prv_eau.drop(columns=['col_29'], inplace=True)
prv_eau['CE']=prv_eau['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [264]:
prv_eau['Periode'].replace('\n',' ', regex=True, inplace=True)
prv_eau.replace('\n','', regex=True, inplace=True)

In [265]:
data=[prv_eau]
for d in data:
    for i in range(len(d['ID_ech'])):
        e=str(d.loc[i, 'Emplacement'])        
        if re.match('P',e, re.I): 
            d.loc[i,'Emplacement']='Pilote'
        elif re.match('HZP',e, re.I): 
            d.loc[i,'Emplacement']='Hors Pilote'
        else:
            d.loc[i,'Emplacement']=np.nan

In [266]:
dataframe_viewer(prv_eau, rows=3)

Rows : 87, columns : 13


interactive(children=(IntSlider(value=3, description='rows', max=87, min=3, readout=False), IntSlider(value=12…

In [267]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [268]:
an=col_ren(an, 1)

In [269]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [270]:
an=dble_col_drop(an)

column(s) dropped: []


In [271]:
an.rename(columns={'Période ':'Periode', 'Emplacement \n- P : Pilote \n- HZP : Hors zone pilote':'Emplacement',
                  'Date de prélèvement':'Date_prv'}, inplace=True)

In [272]:
# rename in a first time before dropping columns (because of names like 'col_xx' in columns)
name=['ID_ech', 'Periode', 'Emplacement', 'Date_prv', 'METAUX LOURDS', 'Arsenic', 'Cadmium', 'Chrome', 'Cuivre','Mercure', 'Plomb', 'Nickel', 'Zinc', 
      'CYANURES',"Cyanures (libres)  -  NEN-EN-ISO 14403", "CN_totaux - NEN-EN-ISO 14403", 
      "cyanure (APE) - méthode basée sur EPA 335.3", "cyanure complex - méthode interne ", 
      "thiocyanate - méthode interne", "COMPOSES AROMATIQUES VOLATILS", "Benzène", "Toluène", "Éthylbenzène",
      "Orthoxylène", "Para- et métaxylène", "Xylènes", "Styrène", "PHENOLS", "Phénol", "Indice phénol", 
      "HYDROCARBURES AROMATIQUES POLYCYCLIQUES", "Naphtalène", "Acénaphtylène", "Acénaphtène", "Fluorène", 
      "Phénanthrène", "Anthracène", "Fluoranthène", "Pyrène", "Benzo(a)anthracène", "Chrysène", 
      "Benzo(b)fluoranthène", "Benzo(k)fluoranthène", "Benzo(a)pyrène", "Dibenzo(ah)anthracène", 
      "Benzo(ghi)pérylène", "Indéno(1,2,3-cd)pyrène", "HAP Totaux (16) - EPA", 
      "COMPOSES ORGANOCHLORES VOLATILS", "1,1-Dichloroéthane", "1,2-Dichloroéthane", 
      "1,1-dichloroéthène", "Cis-1,2-dichloroéthène", "Totaux (cis,trans) 1,2-dichloroéthènes", 
      "Trans 1,2-dichloroéthylène", "Dichlorométhane", "1,2-dichloropropane", "Tétrachloroéthylène ", 
      "Tétrachlorométhane", "1,1,1-Trichloroéthane", "1,1,2-Trichloroéthane", "Trichloroéthylène",
      "Chloroforme", "Chlorure de vinyle", "HYDROCARBURES TOTAUX", "fraction aromat. >C6-C7", 
      "fraction aromat. >C7-C8", "fraction aromat. >C8-C10", "fraction aliphat. C5-C6", "fraction aliphat. >C6-C8",
      "fraction aliphat. >C8-C10", "Fraction C5 - C8", "Fraction C8 - C10", "Fraction C10-C12", "Fraction C12-C16",
      "Fraction C16 - C21", "Fraction C21 - C35", "Hydrocarbures totaux C10-C35", "METHYL-TERT-BUTYL-ETHER", "MTBE",
     "a","b","c","d","e","f","g","h"]

In [273]:
an=col_ren(an, name=name, mode=1)
an=an.iloc[:,:-8]

In [274]:
an.drop(list(range(3)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Eau')


Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'PHENOLS', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOCHLORES VOLATILS', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER', 'MTBE']



In [275]:
an=col_ren(an, name=pol_field_model, mode=1)

In [276]:
an['Periode'].replace('\n',' ', regex=True, inplace=True)
an.replace('\n','', regex=True, inplace=True)

In [277]:
data=[an]
for d in data:
    for i in range(len(d['ID_ech'])):
        e=str(d.loc[i, 'Emplacement'])        
        if re.match('P',e, re.I): 
            d.loc[i,'Emplacement']='Pilote'
        elif re.match('HZP',e, re.I): 
            d.loc[i,'Emplacement']='Hors Pilote'
        else:
            d.loc[i,'Emplacement']=np.nan

In [278]:
dataframe_viewer(an, rows=3)

Rows : 87, columns : 71


interactive(children=(IntSlider(value=3, description='rows', max=87, min=3, readout=False), IntSlider(value=12…

In [279]:
data = [prv_eau, an]
for d in data:
    print('-------------')
    for i, r in d.iterrows():
        for c in d.columns:
            if c not in ['ID_ech', 'Type_ech', 'Periode', 'Emplacement', 'Date_prv'] and \
            str(type(r[c])) not in ["<class 'float'>", "<class 'int'>"]:
                d.loc[i, c] = np.nan
                #print(f'{i}- {str(type(r[c]))}- {c} : {r[c]}')

-------------
-------------


In [280]:
source_prv_eau=prv_eau
source_an=an

In [281]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:87 ;
source_prv_sol:0 ;source_prv_eau:87 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Param physico'**

In [282]:
tmp_dir='../../CF_data/Result_traitem/Siterem_Pilote/'
sheet='Param_physico'

In [283]:
df = pd.read_excel('../../CF_data/Data_SITEREM/Resultats_Siterem_eau_pilote_jusque_decembre_2020.xlsx', 
                   sheet_name='param. physico', skiprows=2)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

7 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 52, columns : 92


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=52, min=5, readout=False), IntSlider(value=12…

In [284]:
df=df.transpose()
df.reset_index(drop=True, inplace=True)

In [285]:
df=col_ren(df, 1)

In [286]:
df.drop(list(range(2)), axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [287]:
sdf=df.iloc[:,:33]
df=df.iloc[:,34:]

In [288]:
df=dble_col_drop(df)
sdf=dble_col_drop(sdf)

column(s) dropped: []
column(s) dropped: ['6:Autre zone suspecte investiguée', '25:pH', '31:pH']


In [289]:
df=na_line_drop(df,1)
sdf=na_line_drop(sdf,1)

83 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [290]:
df=na_col_drop(df,1)
sdf=na_col_drop(sdf,1)


Columns dropped :['Profondeur de la nappe/chambre visite', 'Organoleptique odeur intensité (**)', 'Organoleptique odeur type', 'Profondeur de la nappe/sol ', 'PARAMETRES PHYSICO-CHIMIQUES \n(lors du prélèvement)', 'col_51']


Columns dropped :['SPP/zone suspecte investiguée', 'Autre zone suspecte investiguée', 'Parcelle', 'X Lambert', 'Y Lambert', "Type d'étude (*)", 'Profondeur minimum de la crépine', 'Profondeur maximum de la crépine', 'Profondeur du fond du piézomètre', 'Organoleptique odeur intensité (**)', 'Organoleptique odeur type', 'Limpidité (***)', 'Température', 'Conductivité électrique', 'PARAMETRES PHYSICO-CHIMIQUES \n(lors du prélèvement)', 'PARAMETRES PHYSICO-CHIMIQUES \n(mesures au labo)']



In [291]:
df=df.iloc[:,:-1]
name=['ID_ech','Periode','Emplacement','Date_prv','Niv_eau_pz','Long_pz','Temp_prv ','pH', 'CE', 'ORP','O_diss']
df=col_ren(df, mode=1, name=name)

In [292]:
sdf.drop(columns=['col_29'], inplace=True)
name=['ID_ech','Periode','Emplacement','Date_prv','Niv_eau_pz','Long_pz','pH','Niv_eau_sol','Temp_prv ','CE',
      'ORP','O_diss','Temp_pH_mes']
sdf=col_ren(sdf, mode=1, name=name)

In [293]:
df['CE']=df['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)
sdf['CE']=sdf['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [294]:
sdf['Periode'].replace('\n',' ', regex=True, inplace=True)
sdf.replace('\n','', regex=True, inplace=True)
sdf.drop(columns=["Niv_eau_sol"], inplace=True)

In [295]:
set(sdf['Emplacement'])

{'HZP', 'P'}

In [296]:
data=[df, sdf]
for d in data:
    for i in range(len(d['ID_ech'])):
        e=str(d.loc[i, 'Emplacement'])
        
        if re.match('P',e, re.I): 
            d.loc[i,'Emplacement']='Pilote'
        elif re.match('HZP',e, re.I): 
            d.loc[i,'Emplacement']='Hors Pilote'
        else:
            d.loc[i,'Emplacement']=np.nan

In [297]:
df.insert(1, 'Type_ech', 'Eau')
sdf.insert(1, 'Type_ech', 'Eau')

In [298]:
df.replace('\*|à compléter',np.nan, inplace=True, regex=True)

In [299]:
prv_eau, conflict_df = data_merger(sdf, df, 'outer', 'ID_ech')

In [300]:
source_prv_eau, conflict_df=data_merger(source_prv_eau, prv_eau, on=['ID_ech', 'Date_prv', 'Periode'], how='outer')

In [301]:
dataframe_viewer(source_prv_eau, rows=3)

Rows : 178, columns : 16


interactive(children=(IntSlider(value=3, description='rows', max=178, min=3, readout=False), IntSlider(value=1…

In [302]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
#source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:87 ;
source_prv_sol:0 ;source_prv_eau:178 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Inorganiques et composés majeurs'**

In [303]:
tmp_dir='../../CF_data/Result_traitem/Siterem_Pilote/'
sheet='Inorganic_major'

In [304]:
df = pd.read_excel('../../CF_data/Data_SITEREM/Resultats_Siterem_eau_pilote_jusque_decembre_2020.xlsx', 
                   sheet_name='inorganiques et composés majeur', skiprows=2)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

4 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 68, columns : 54


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=68, min=5, readout=False), IntSlider(value=12…

In [305]:
prv_eau=df.loc[:21]
an=df.loc[list(range(0,4))+list(range(22, len(df)))]

In [306]:
an = an.sort_index().reset_index(drop=True)

In [307]:
prv_eau=prv_eau.transpose()
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=col_ren(prv_eau, 1)

In [308]:
prv_eau=dble_col_drop(prv_eau)

column(s) dropped: ['6:Autre zone suspecte investiguée']


In [309]:
prv_eau.drop(list(range(2)), axis=0, inplace=True)
prv_eau.reset_index(drop=True, inplace=True)
prv_eau=na_col_drop(prv_eau,2)
prv_eau=na_line_drop(prv_eau,2)
prv_eau.reset_index(drop=True, inplace=True)


Columns dropped :['SPP/zone suspecte investiguée', 'Autre zone suspecte investiguée', 'Parcelle', 'X Lambert', 'Y Lambert', "Type d'étude (*)", 'Profondeur minimum de la crépine', 'Profondeur maximum de la crépine', 'Profondeur du fond du piézomètre', 'Profondeur de la nappe', 'Organoleptique odeur intensité (**)', 'Organoleptique odeur type', 'Limpidité (***)', 'Température', 'Conductivité électrique', 'pH']



In [310]:
prv_eau.columns

Index(['Nom échantillon', 'Période ',
       'Emplacement \n- S : Simulateur \n- HZS : Hors zone simulateur',
       'Date de prélèvement', 'Température de prélèvement '],
      dtype='object')

In [311]:
name=['ID_ech', 'Periode', 'Emplacement','Date_prv','Temp_prv']
prv_eau.replace(r'\n',' ', inplace=True, regex=True)
prv_eau=col_ren(prv_eau, name=name, mode=1)
prv_eau=prv_eau.query('ID_ech==ID_ech')
prv_eau.insert(1,'Type_ech','Eau')

In [312]:
dataframe_viewer(prv_eau, rows=3)

Rows : 51, columns : 6


interactive(children=(IntSlider(value=3, description='rows', max=51, min=3, readout=False), IntSlider(value=6,…

In [313]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [314]:
an=col_ren(an, 1)

In [315]:
an.replace(r'<|>','', inplace=True, regex=True)
an.replace(r'-',np.nan, inplace=True, regex=True)
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [316]:
an=dble_col_drop(an)

column(s) dropped: ['11:ammonium', '15:nitrite', '17:nitrate']


In [317]:
an=na_col_drop(an,3)


Columns dropped :['CARBONE ORGANIQUE', 'DEMANDE EN O2', 'COMPOSES AZOTES', 'COMPOSES SOUFRES ', 'ELEMENTS MAJEURS', 'AUTRES ANALYSES', 'cyanure (libre)', 'METHYL-TERT-BUTYL-ETHER', 'MTBE', 'Teneur mesurée ', 'Teneur mesurée', 'VS : Valeur seuil', "(*) RP : Rapport de prélèvements; ES : Etude de sol; EO : Etude d'orientation; EC : Etude de caractérisation; SA : Suivi d'assainissement; EF : Evaluation finale", "(**) - : Pas d'impression organoleptique; + : Impression organoleptique faible; ++ : Impression organoleptique forte", '(***) + : Limpide; - : Trouble; -- : Opaque', "Le contenu des tableaux est conforme au modèle repris à l'annexe IX du GREO V03. Le formalisme a été adapté par SITEREM tout en garantissant la lisibilité du document imprimé. "]



In [318]:
an.rename(columns={'Période ':'Periode', 'Emplacement \n- S : Simulateur \n- HZS : Hors zone simulateur':'Emplacement',
                  'Date de prélèvement':'Date_prv', 'col_9':'ammoniaque libre'}, inplace=True)

In [319]:
an.drop(list(range(2)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Eau')

In [320]:
an=col_ren(an, name=pol_field_model, mode=1)
#an=an.iloc[:,:-7]

In [321]:
data=[prv_eau, an]
for d in data:
    for i in range(len(d['ID_ech'])):
        e=str(d.loc[i, 'Emplacement'])
        
        if re.match('S',e, re.I): 
            d.loc[i,'Emplacement']='Simulateur'
        elif re.match('HZS',e, re.I): 
            d.loc[i,'Emplacement']='Hors simulateur'
        else:
            d.loc[i,'Emplacement']=np.nan

In [322]:
data = [prv_eau, an]
for d in data:
    print('-------------')
    for i, r in d.iterrows():
        for c in d.columns:
            if c not in ['ID', 'ID_ech', 'Type', 'Type_ech', 'Periode', 'Emplacement', 'Date_prv'] and \
            str(type(r[c])) not in ["<class 'float'>", "<class 'int'>"]:
                d.loc[i, c] = np.nan
                #print(f'{i}- {str(type(r[c]))}- {c} : {r[c]}')

-------------
-------------


In [323]:
dataframe_viewer(an, rows=3)

Rows : 51, columns : 32


interactive(children=(IntSlider(value=3, description='rows', max=51, min=3, readout=False), IntSlider(value=12…

In [324]:
source_prv_eau.Date_prv = source_prv_eau.Date_prv.astype(object)

In [325]:
source_prv_eau, conflict_df=data_merger(source_prv_eau, prv_eau, 'outer', ['ID_ech', 'Date_prv'] )

Conflict values present. Please resolve this manually !


In [326]:
data_validation(overall_data=source_prv_eau, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Periode_y':list(conflict_df.index), 'Temp_prv_y':list(conflict_df.index), 
                            'Emplacement_y':list(conflict_df.index)})

all conflicts have been fixed!


In [327]:
source_an, conflict_df=data_merger(source_an, an, 'outer', ['ID_ech', 'Date_prv', 'Periode'])

Conflict values present. Please resolve this manually !


In [328]:
data_validation(overall_data=source_an, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Emplacement_y':list(conflict_df.index)})

all conflicts have been fixed!


In [329]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:115 ;
source_prv_sol:0 ;source_prv_eau:181 ; source_mes_pz:0 ; source_mes_sol:0 ;


### $\color{red}{\textbf{Excel sheets data merging}}$

In [330]:
excel_bh_water_an, conflict_df = data_merger(source_an, source_prv_eau, how='outer', on=['ID_ech', 'Date_prv', 'Periode'],
                                             drop_skip_col=['index'])

In [331]:
dataframe_viewer(excel_bh_water_an, rows=5)

Rows : 212, columns : 109


interactive(children=(IntSlider(value=5, description='rows', max=212, min=5, readout=False), IntSlider(value=1…

In [332]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
#excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
#excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
#excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
#excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
#excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)

## 13-Resultats_Siterem_SOL.xlsx
* **Sheet : 'Résult SOL ext. pilote'**

In [333]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [334]:
tmp_dir='../../CF_data/Result_traitem/Siterem_Result_Sol/'
sheet='Result_sol_ExtP'

In [335]:
df = pd.read_excel('../../CF_data/Data_SITEREM/Resultats_Siterem_SOL.xlsx', 
                   sheet_name='Résult SOL ext. pilote', skiprows=5)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

2 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 49']

Rows : 103, columns : 48


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=103, min=5, readout=False), IntSlider(value=1…

In [336]:
prv_sol=df.loc[:22]
an=df.loc[23:]

In [337]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [338]:
prv_sol=prv_sol.transpose()
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=col_ren(prv_sol, 1)

In [339]:
prv_sol=dble_col_drop(prv_sol)

column(s) dropped: ['16:température pour mes. pH']


In [340]:
prv_sol.drop(list(range(3)), axis=0, inplace=True)
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=na_col_drop(prv_sol,1)
prv_sol=na_line_drop(prv_sol,3)
prv_sol.reset_index(drop=True, inplace=True)


Columns dropped :['MO et COT', 'pH', 'GRANULOMETRIE']



In [341]:
dataframe_viewer(prv_sol, rows=3)

Rows : 44, columns : 19


interactive(children=(IntSlider(value=3, description='rows', max=44, min=3, readout=False), IntSlider(value=12…

In [342]:
prv_sol=prv_sol[:-1]
prv_sol.drop(columns=['broyage'], inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [343]:
name=['ID_ech','Ech_top', 'Ech_base','MS','Date_prv','Long_for','Refus','Description','MO','COT','pH_KCl', 
      'Temp_pH_mes','pH_H20','Fract_2','Fract_2+', 'Fract_min_2µ','Fract_min_50µ','Fract_min_2']
prv_sol=col_ren(prv_sol, name=name, mode=1)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [344]:
set(prv_sol.Description)

{'R', 'R ', 'TN', 'TN '}

In [345]:
for i in range(len(prv_sol['Description'])):
    x = prv_sol.loc[i,'Description']
    if x in ['R','R ']: prv_sol.loc[i,'Description']='Remblais'
    elif x in ['TN','TN ']: prv_sol.loc[i,'Description']='Terrain naturel'

prv_sol['Refus']=prv_sol['Refus'].apply(lambda x: 'x' if not pd.isnull(x) else '')
prv_sol.insert(1,'Type_ech','Sol')#

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [346]:
dataframe_viewer(prv_sol, rows=3)

Rows : 43, columns : 19


interactive(children=(IntSlider(value=3, description='rows', max=43, min=3, readout=False), IntSlider(value=12…

In [347]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [348]:
an=col_ren(an, 1)

In [349]:
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [350]:
name=['ID_ech','METAUX LOURDS','Arsenic','Cadmium','Chrome','Chrome VI','Cuivre','Mercure','Plomb','Nickel',
'Zinc','CYANURES','cyanure (libre)','cyanure (totaux)','cyanure (APE)','cyanure complex','thiocyanate',
'COMPOSES AROMATIQUES VOLATILS','Benzène','Toluène','Éthylbenzène','Orthoxylène','Para- et métaxylène','Xylènes',
'Styrène','BTEX totaux','PHENOLS','Phénol','HYDROCARBURES AROMATIQUES POLYCYCLIQUES','Naphtalène','Acénaphtylène',
'Acénaphtène','Fluorène','Phénanthrène','Anthracène','Fluoranthène','Pyrène','Benzo(a)anthracène','Chrysène',
'Benzo(b)fluoranthène','Benzo(k)fluoranthène','Benzo(a)pyrène','Dibenzo(ah)anthracène','Benzo(ghi)pérylène',
'Indéno(1,2,3-cd)pyrène','HAP Totaux (16) - EPA','COMPOSES ORGANOHALOGENES VOLATILS','Tétrachloroéthylène',
'Trichloroéthylène','1,1-dichloroéthène','Cis-1,2-dichloroéthène','Trans 1,2-dichloroéthylène',
'Totaux (cis,trans) 1,2-dichloroéthènes','Chlorure de vinyle','1,1,1-Trichloroéthane','1,1,2-Trichloroéthane',
'1,1-Dichloroéthane','1,2-Dichloroéthane','Tétrachlorométhane','Chloroforme','Dichlorométhane',
'1,2-dichloropropane','HYDROCARBURES TOTAUX','fraction aromat. >C6-C7','fraction aromat. >C7-C8',
'fraction aromat. >C8-C10','fraction aliphat. C5-C6','fraction aliphat. >C6-C8','fraction aliphat. >C8-C10',
'Fraction C5 - C8','Fraction C8 - C10','Fraction C10-C12','Fraction C12-C16','Fraction C16 - C21',
'Fraction C21 - C35','Fraction C35 - C40','Hydrocarbures totaux C10-C35','Hydrocarbures totaux C10-C40',
'Teneur mesurée','Teneur mesurée','VS : Valeur seuil']

an=col_ren(an, name=name, mode=1)

In [351]:
an=dble_col_drop(an)

column(s) dropped: ['79:Teneur mesurée']


In [352]:
an.drop(list(range(3)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an.insert(1,'Type_ech','Sol')


Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX totaux', 'PHENOLS', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', 'HYDROCARBURES TOTAUX', 'Teneur mesurée', 'VS : Valeur seuil']



In [353]:
an=col_ren(an, name=pol_field_model, mode=1)

In [354]:
an.rename(columns={'cyanure (totaux)':'CN_tot', 'cyanure (APE)':'CN_EPA'}, inplace=True)

In [355]:
dataframe_viewer(an, rows=5) 

Rows : 44, columns : 71


interactive(children=(IntSlider(value=5, description='rows', max=44, min=5, readout=False), IntSlider(value=12…

In [356]:
dataframe_viewer(prv_sol, rows=5)

Rows : 43, columns : 19


interactive(children=(IntSlider(value=5, description='rows', max=43, min=5, readout=False), IntSlider(value=12…

In [357]:
source_prv_sol=prv_sol
source_an=an

In [358]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:44 ;
source_prv_sol:43 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'SOL T1 pilote'**

In [359]:
tmp_dir='../../CF_data/Result_traitem/Siterem_Result_Sol/'
sheet='SOL_T1_Pilote'

In [360]:
df = pd.read_excel('../../CF_data/Data_SITEREM/Resultats_Siterem_SOL.xlsx', 
                   sheet_name='SOL T1 pilote', skiprows=5)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

3 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36']

Rows : 135, columns : 29


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=135, min=5, readout=False), IntSlider(value=1…

In [361]:
prv_sol=df.loc[:35]
an=df.loc[36:]

In [362]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [363]:
prv_sol=prv_sol.transpose()
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=col_ren(prv_sol, 1)

In [364]:
prv_sol=dble_col_drop(prv_sol)

column(s) dropped: ['9:Autre zone suspecte investiguée', '27:température pour mes. pH', '30:pH (H20)']


In [365]:
prv_sol.drop(list(range(3)), axis=0, inplace=True)
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=na_col_drop(prv_sol,1)
prv_sol=na_line_drop(prv_sol,3)
prv_sol.reset_index(drop=True, inplace=True)


Columns dropped :['Matières organiques', 'SPP/zone suspecte investiguée', 'Autre zone suspecte investiguée', 'Parcelle', 'X Lambert', 'Y Lambert', "Type d'étude (*)", 'Organoleptique couleur suspecte', 'Organoleptique odeur intensité (***)', 'Organoleptique odeur type', 'MO et COT', 'matières organiques', 'COT', 'pH', 'pH (KCl)', 'température pour mes. pH', 'pH (H20)', 'GRANULOMETRIE', 'Fraction argileuse', 'parties min. 2µm', 'parties min. 50µm', 'parties min. 2mm']

9 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [366]:
prv_sol=prv_sol[:-1]
prv_sol.drop(columns=['broyage'], inplace=True)

In [367]:
prv_sol.columns

Index(['Nom de l'échantillon', 'Profondeur échantillon de', 'à',
       'Matière sèche', 'Date de prélèvement', 'Profondeur d'arrêt du forage',
       'Refus de forage (seulement si oui)', 'Terrain naturel/Remblai (**)',
       'fraction  2 mm (prép. séché à 40°C) ',
       'fraction 2 mm (prép. séché à 40°C) '],
      dtype='object')

In [368]:
name=['ID_ech','Ech_top', 'Ech_base','MS','Date_prv','Long_for','Refus','Nature_ech','Fract_2','Fract_2+']
prv_sol=col_ren(prv_sol, name=name, mode=1)

In [369]:
for i in range(len(prv_sol['Nature_ech'])):
    x = prv_sol.loc[i,'Nature_ech']
    if x in ['R','R ']: prv_sol.loc[i,'Nature_ech']='Remblais'
    elif x in ['TN','TN ']: prv_sol.loc[i,'Nature_ech']='Terrain naturel'

prv_sol['Refus']=prv_sol['Refus'].apply(lambda x: 'x' if not pd.isnull(x) else '')
prv_sol.insert(1,'Type_ech','Sol')#

In [370]:
dataframe_viewer(prv_sol, rows=3)

Rows : 15, columns : 11


interactive(children=(IntSlider(value=3, description='rows', max=15, min=3, readout=False), IntSlider(value=11…

In [371]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [372]:
an=col_ren(an, 1)

In [373]:
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [374]:
name=['ID_ech','METAUX LOURDS','Arsenic','Cadmium','Chrome','Chrome VI','Cobalt','Cuivre','Mercure','Plomb', 
'Nickel','Zinc','CYANURES','cyanure (libre)','cyanure (totaux)','cyanure (APE)','cyanure complex','thiocyanate',
'COMPOSES AROMATIQUES VOLATILS','Benzène','Toluène','Éthylbenzène', 'Orthoxylène','Para- et métaxylène','Xylènes',
'Styrène','BTEX totaux','PHENOLS','Phénol','Indice phénol','HYDROCARBURES AROMATIQUES POLYCYCLIQUES','Naphtalène',
'Acénaphtylène','Acénaphtène', 'Fluorène','Phénanthrène','Anthracène','Fluoranthène','Pyrène','Benzo(a)anthracène',
'Chrysène','Benzo(b)fluoranthène','Benzo(k)fluoranthène','Benzo(a)pyrène','Dibenzo(ah)anthracène',
'Benzo(ghi)pérylène','Indéno(1,2,3-cd)pyrène','HAP Totaux (16) - EPA','COMPOSES ORGANOHALOGENES VOLATILS',
'Tétrachloroéthylène','Trichloroéthylène','1,1-dichloroéthène','Cis-1,2-dichloroéthène',
'Trans 1,2-dichloroéthylène','Totaux (cis,trans) 1,2-dichloroéthènes','Chlorure de vinyle',
'1,1,1-Trichloroéthane','1,1,2-Trichloroéthane','1,1-Dichloroéthane','1,2-Dichloroéthane','Tétrachlorométhane',
'Chloroforme','Dichlorométhane','1,2-dichloropropane','EOX','HYDROCARBURES TOTAUX',
'fraction aromat. >C6-C7','fraction aromat. >C7-C8','fraction aromat. >C8-C10','fraction aliphat. C5-C6',
'fraction aliphat. >C6-C8','fraction aliphat. >C8-C10','Fraction C5 - C8','Fraction C8 - C10','Fraction C10-C12',
'Fraction C12-C16','Fraction C16 - C21','Fraction C21 - C35','Fraction C35 - C40','Hydrocarbures totaux C10-C35',
'Hydrocarbures totaux C10-C40','METHYL-TERT-BUTYL-ETHER','MTBE']

an=an.iloc[:,:-17]
an=col_ren(an, name=name, mode=1)

In [375]:
an=dble_col_drop(an)

column(s) dropped: []


In [376]:
an.drop(list(range(3)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,3)
an.insert(1,'Type_ech','Sol')


Columns dropped :['METAUX LOURDS', 'Chrome VI', 'Cobalt', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX totaux', 'PHENOLS', 'Phénol', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', 'EOX', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER']



In [377]:
an=col_ren(an, name=pol_field_model, mode=1)

In [378]:
an.rename(columns={'cyanure (totaux)':'CN_tot', 'cyanure (APE)':'CN_EPA'}, inplace=True)

In [379]:
dataframe_viewer(an, rows=5) 

Rows : 25, columns : 70


interactive(children=(IntSlider(value=5, description='rows', max=25, min=5, readout=False), IntSlider(value=12…

In [380]:
dataframe_viewer(prv_sol, rows=5) 

Rows : 15, columns : 11


interactive(children=(IntSlider(value=5, description='rows', max=15, min=5, readout=False), IntSlider(value=11…

In [381]:
#source_prv_sol.info()#, prv_sol.info()

In [382]:
source_prv_sol=source_prv_sol[['ID_ech', 'Type_ech', 'Date_prv','Long_for', 'Refus', 'Description', 
                               'Ech_top', 'Ech_base', 'MS', 'Fract_2', 'Fract_2+']]

In [383]:
source_prv_sol=data_merger(source_prv_sol, prv_sol, on='ID_ech', how='outer')[0]

In [384]:
source_an, conflict=data_merger(source_an,an, on='ID_ech', how='outer')

In [385]:
source_an=source_an.query('ID_ech==ID_ech')

In [386]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:58 ;
source_prv_sol:58 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


### $\color{red}{\textbf{Excel sheets data merging}}$

In [387]:
excel_bh_soil_an, conflict_df = data_merger(source_prv_sol, source_an, how='outer', on='ID_ech', drop_skip_col=['index'])

In [388]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
#excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
#excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
#excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
#excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
#excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)

## 14-Logs_forages_vUmons_2018-03-20.xlsx
* **Sheet : 'Analyse_eau_Phases1&2'**

In [389]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [390]:
tmp_dir='../../CF_data/Result_traitem/vUmons_logsFor/'
sheet='Analyse_eau_Phases1&2'

In [391]:
df = pd.read_excel('../../CF_data/Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', 
                   sheet_name='Analyse_eau_Phases1&2', skiprows=0)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

1 NaN lines dropped
Rows : 51, columns : 85


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=51, min=5, readout=False), IntSlider(value=12…

In [392]:
df.drop(list(range(4)), axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [393]:
df.replace(9999,np.nan, inplace=True, regex=True) #int
df.replace(f'[{9999}|9999].',np.nan, inplace=True, regex=True) #float, str

In [394]:
df['CE']=df['CE'].apply(lambda x: pd.to_numeric(x)/1000 
                                  if re.search('^\d+', str(x)) and not pd.isnull(x) else np.nan)

In [395]:
df=col_ren(df,mode=1,name=[re.sub('9999','-',x) for x in df.columns])
df=col_ren(df,mode=1, name=pol_field_model)

In [396]:
name=['ID', 'ID_ech', 'Date_prv', 'X', 'Y', 'Z', 'Long_for','Long_pz_sol', 'Niv_eau_sol', 'pH', 'CE', 'T', 
      'As', 'Cd', 'Cr', 'Cr_VI', 'Cu', 'Hg','Pb', 'Ni', 'Zn', 'CN_libre', 'CN_tot', 'CN_APE', 'CN_comp',
      'thioCN', 'Bnz_vn', 'Bnz', 'Toln_vn', 'Toln', 'EthylBnz','O-Xyl', 'P-M-Xyl', 'Xyl_vn', 'Xyl', 'Styr', 
      'Phenol','Naphta_vn', 'Naphta', 'Acenaphtyl', 'Acenaphtn', 'Fluorene',
       'Phenanthr', 'Anthrc', 'Flranth', 'Pyr', 'Bnz(a)anthrc', 'Chrys',
       'Bnz(b)flranth', 'Bnz(k)flranth', 'Bnz(a)pyr', 'Dibnz(ah)anthrc',
       'Bnz(ghi)peryl', 'Indeno(1,2,3-cd)pyr', 'HAP_tot_EPA',
       '1,1-DCE', '1,2-DCE', '1,1-DCEn', 'Cis-1,2-DCEn',
       '(cis,trans) 1,2-DCE_tot', 'Trans 1,2-DCEyl', 'DCM', '1,2-DCP',
       'TetraCEyn', 'TCM', '1,1,1-TCE', '1,1,2-TCE', 'TCEyn', 'Chloroforme',
       'CVinyl', 'Arom_C6C7', 'Arom_C7C8', 'Arom_C8C10', 'Aliphat_C5C6',
       'Aliphat_C6C8', 'Aliphat_C8C10', 'Fract_C5C8', 'Fract_C8C10',
       'Fract_C10C12', 'Fract_C12C16', 'Fract_C16C21', 'Fract_C21C35',
       'HC_tot_C10C35', 'MTBE', 'Chlorure']
df=col_ren(df, mode=1,name=name)

In [397]:
df['ID_ech'].replace('Canne ', 'Can', inplace=True, regex=True)
df['ID_ech'].replace('\n', ' ', inplace=True, regex=True)
df.insert(1,'Type_ech','Eau')

In [398]:
df.drop([20,39], axis=0,inplace=True)
df.reset_index(drop=True, inplace=True)

In [399]:
df.loc[38:,'Date_prv']=df.loc[38:,'Date_prv'].apply(lambda x : dtm.datetime.fromordinal(dtm.datetime(1900, 1, 1).toordinal() + x - 2))

In [400]:
for i in range(len(df['ID_ech'])):
    if pd.isnull(df.loc[i,'ID_ech']): 
        df.loc[i,'ID_ech']=df.loc[i,'ID']

In [401]:
pz=df[['ID', 'X', 'Y', 'Z', 'Long_for','Long_pz_sol']]
pz['Type'] = 'Piezo'

prv_eau=df[['ID','ID_ech','Type_ech','Date_prv', 'X', 'Y', 'Z','Niv_eau_sol', 'pH', 'CE', 'T']]
an=df[['ID','ID_ech','Type_ech','Date_prv', 'X', 'Y', 'Z','As', 'Cd', 'Cr', 'Cr_VI', 'Cu', 'Hg','Pb', 'Ni', 'Zn', 'CN_libre', 'CN_tot', 'CN_APE', 
       'CN_comp','thioCN', 'Bnz_vn', 'Bnz', 'Toln_vn', 'Toln', 'EthylBnz','O-Xyl', 'P-M-Xyl', 'Xyl_vn', 'Xyl',
       'Styr', 'Phenol','Naphta_vn', 'Naphta', 'Acenaphtyl', 'Acenaphtn', 'Fluorene',
       'Phenanthr', 'Anthrc', 'Flranth', 'Pyr', 'Bnz(a)anthrc', 'Chrys',
       'Bnz(b)flranth', 'Bnz(k)flranth', 'Bnz(a)pyr', 'Dibnz(ah)anthrc',
       'Bnz(ghi)peryl', 'Indeno(1,2,3-cd)pyr', 'HAP_tot_EPA',
       '1,1-DCE', '1,2-DCE', '1,1-DCEn', 'Cis-1,2-DCEn',
       '(cis,trans) 1,2-DCE_tot', 'Trans 1,2-DCEyl', 'DCM', '1,2-DCP',
       'TetraCEyn', 'TCM', '1,1,1-TCE', '1,1,2-TCE', 'TCEyn', 'Chloroforme',
       'CVinyl', 'Arom_C6C7', 'Arom_C7C8', 'Arom_C8C10', 'Aliphat_C5C6',
       'Aliphat_C6C8', 'Aliphat_C8C10', 'Fract_C5C8', 'Fract_C8C10',
       'Fract_C10C12', 'Fract_C12C16', 'Fract_C16C21', 'Fract_C21C35',
       'HC_tot_C10C35', 'MTBE', 'Chlorure']]

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [402]:
pz.drop_duplicates(subset=['ID'], keep='first', ignore_index=True, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [403]:
dataframe_viewer(an, rows=5)

Rows : 45, columns : 80


interactive(children=(IntSlider(value=5, description='rows', max=45, min=5, readout=False), IntSlider(value=12…

In [404]:
source_an=an
source_pz=pz
source_prv_eau=prv_eau

In [405]:
for i in range(len(excel_water_an.ID_ech)):
    c=excel_water_an.loc[i, 'ID_ech']
    excel_water_an.loc[i, 'ID']=re.search("(\w+\d+(?:\w)?)",c).group(1)

In [406]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
#excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
#excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
#excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
#excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
#excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
#excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)

an.to_csv(save_dir+'Water_analysis.csv', index=False)

In [407]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:29 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:45 ;
source_prv_sol:0 ;source_prv_eau:45 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Analyse_sol_Phases1&2'**

In [408]:
tmp_dir='../../CF_data/Result_traitem/vUmons_logsFor/'
sheet='Analyse_sol_Phases1&2'

In [409]:
df = pd.read_excel('../../CF_data/Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', 
                   sheet_name='Analyse_sol_Phases1&2', skiprows=0)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

Rows : 64, columns : 84


interactive(children=(IntSlider(value=5, description='rows', max=64, min=5, readout=False), IntSlider(value=12…

In [410]:
df=col_ren(df, mode=1, name=pol_field_model)

In [411]:
name=['ID_ech','Date_prv','ID','X','Y','Z','Nature_ech','Organo','Long_for','Refus','Ech_top','Ech_base',
      'MS','Broyage < 150 µm','Broyage ','Fract_2','Fract_2+','As','Cd','Cr','Cr_VI','Cu',
       'Hg','Pb','Ni','Zn','CN_libre','CN_tot','CN_APE',
       'CN_comp','thioCN','Bnz','Toln','EthylBnz','O-Xyl','P-M-Xyl',
       'Xyl','Styr','Phenol','Naphta','Acenaphtyl','Acenaphtn',
       'Fluorene','Phenanthr','Anthrc','Flranth','Pyr','Bnz(a)anthrc',
       'Chrys','Bnz(b)flranth','Bnz(k)flranth','Bnz(a)pyr',
       'Dibnz(ah)anthrc','Bnz(ghi)peryl','Indeno(1,2,3-cd)pyr',
       'HAP_tot_EPA','1,1-DCE','1,2-DCE','1,1-DCEn',
       'Cis-1,2-DCEn','Trans 1,2-DCEyl','DCM',
       '(cis,trans) 1,2-DCE_tot','1,2-DCP','TetraCEyn','TCM',
       '1,1,1-TCE','1,1,2-TCE','TCEyn','Chloroforme','CVinyl','Arom_C6C7',
       'Arom_C7C8','Arom_C8C10','Aliphat_C5C6','Aliphat_C6C8',
       'Aliphat_C8C10','Fract_C5C8','Fract_C8C10','Fract_C10C12',
       'Fract_C12C16','Fract_C16C21','Fract_C21C35','HC_tot_C10C35']
df=col_ren(df, mode=1, name=name)

In [412]:
df.drop(list(range(4)), axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [413]:
df.replace(9999,np.nan, inplace=True, regex=True) #int
df.replace(f'[{9999}|9999].',np.nan, inplace=True, regex=True) #float, str

In [414]:
for i in range(len(df['Nature_ech'])):
    x = df.loc[i,'Nature_ech']
    if x in ['R','R ']: df.loc[i,'Nature_ech']='Remblais'
    elif x in ['L']: df.loc[i,'Nature_ech']='Limons'
    elif x in ['LA']: df.loc[i,'Nature_ech']='Limons et argiles'
    elif x in ['LS']: df.loc[i,'Nature_ech']='Limons et sables'

df['Refus']=df['Refus'].apply(lambda x: 'x' if not pd.isnull(x) else '')
df.insert(1,'Type_ech','Sol')

In [415]:
df.drop(14, axis=0, inplace=True)
df.drop(['Broyage < 150 µm', 'Broyage '], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

In [416]:
df.loc[8, 'ID_ech']='F4/2M'
df.loc[31, 'ID_ech']='P19/1'
df.loc[32, 'ID_ech']='P19/2'

In [417]:
pz=df[['ID', 'X', 'Y', 'Z', 'Long_for','Refus']]
pz['Type'] = 'Piezo'

prv_sol=df[['ID_ech', 'Type_ech', 'Date_prv', 'X', 'Y', 'Z', 'Nature_ech','Organo', 
            'Ech_top', 'Ech_base', 'MS', 'Fract_2','Fract_2+']]
an=df[['ID','ID_ech', 'Date_prv', 'X', 'Y', 'Z', 'Type_ech','As', 'Cd', 'Cr', 'Cr_VI', 'Cu', 'Hg', 'Pb', 'Ni', 'Zn',
       'CN_libre', 'CN_tot', 'CN_APE', 'CN_comp', 'thioCN', 'Bnz', 'Toln',
       'EthylBnz', 'O-Xyl', 'P-M-Xyl', 'Xyl', 'Styr', 'Phenol', 'Naphta',
       'Acenaphtyl', 'Acenaphtn', 'Fluorene', 'Phenanthr', 'Anthrc', 'Flranth',
       'Pyr', 'Bnz(a)anthrc', 'Chrys', 'Bnz(b)flranth', 'Bnz(k)flranth',
       'Bnz(a)pyr', 'Dibnz(ah)anthrc', 'Bnz(ghi)peryl', 'Indeno(1,2,3-cd)pyr',
       'HAP_tot_EPA', '1,1-DCE', '1,2-DCE', '1,1-DCEn', 'Cis-1,2-DCEn',
       'Trans 1,2-DCEyl', 'DCM', '(cis,trans) 1,2-DCE_tot', '1,2-DCP',
       'TetraCEyn', 'TCM', '1,1,1-TCE', '1,1,2-TCE', 'TCEyn', 'Chloroforme',
       'CVinyl', 'Arom_C6C7', 'Arom_C7C8', 'Arom_C8C10', 'Aliphat_C5C6',
       'Aliphat_C6C8', 'Aliphat_C8C10', 'Fract_C5C8', 'Fract_C8C10',
       'Fract_C10C12', 'Fract_C12C16', 'Fract_C16C21', 'Fract_C21C35',
       'HC_tot_C10C35']]

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [418]:
an['ID'] = an['ID'].apply(lambda x: re.sub('^P', 'F', x)) 

In [419]:
dataframe_viewer(an, rows=5)

Rows : 59, columns : 74


interactive(children=(IntSlider(value=5, description='rows', max=59, min=5, readout=False), IntSlider(value=12…

In [420]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
#excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
#excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
#excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
#excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
#excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
#excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)

an.to_csv(save_dir+'Soil_analysis.csv', index=False)

In [421]:
source_an=source_an.append(an, ignore_index=True)
source_pz=pz
source_prv_sol=prv_sol

In [422]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:59 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:104 ;
source_prv_sol:59 ;source_prv_eau:45 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Synthèse'**

In [423]:
tmp_dir='../../CF_data/Result_traitem/vUmons_logsFor/'
sheet='Synthese'

In [424]:
df = pd.read_excel('../../CF_data/Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', 
                   sheet_name='Synthèse', skiprows=1)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

3 NaN lines dropped

Columns dropped :['Unnamed: 0']

Rows : 33, columns : 14


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=33, min=5, readout=False), IntSlider(value=12…

In [425]:
df=df[:29]
df.replace('\*','', inplace=True, regex=True)
df['Refus']=df['Refus'].apply(lambda x: 'x' if not pd.isnull(x) else '')

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [426]:
name=['ID','X','Y','Z', 'Refus','Long_for', 'RB', 'ALL', 'S_A', 'S_S', 
      'Rb_base','All_top', 'Soc_alt_top','Soc_sn_top']
df=col_ren(df, mode=1, name=name)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [427]:
cols=['ID','X','Y','Z', 'Refus','Long_for']

for i in range(len(df)):
    if not pd.isnull(df.loc[i, 'RB']): 
        df.loc[i, 'Nappe']='Remblais'
        df.loc[i, 'Litho_top']=0
        if not pd.isnull(df.loc[i, 'Rb_base']):
            df.loc[i, 'Litho_base']=df.loc[i, 'Rb_base']
        else:
            df.loc[i, 'Litho_base']=df.loc[i, 'Long_for']
    
    if not pd.isnull(df.loc[i, 'ALL']):
        df.loc[i+.2,cols]=df.loc[i,cols]
        df.loc[i+.2, 'Nappe']='Alluvions'
        df.loc[i+.2, 'Litho_top']=df.loc[i, 'All_top']
        if not pd.isnull(df.loc[i, 'S_A']):
            df.loc[i+.2, 'Litho_base']=df.loc[i, 'Soc_alt_top']
        else:
            df.loc[i+.2, 'Litho_base']=df.loc[i, 'Long_for']
    
    if not pd.isnull(df.loc[i, 'S_A']):
        df.loc[i+.5,cols]=df.loc[i,cols]
        df.loc[i+.5, 'Nappe']='Socle altéré'
        df.loc[i+.5, 'Litho_top']=df.loc[i, 'Soc_alt_top']
        if not pd.isnull(df.loc[i, 'S_S']):
            df.loc[i+.5, 'Litho_base']=df.loc[i, 'Soc_sn_top']
        else:
            df.loc[i+.5, 'Litho_base']=df.loc[i, 'Long_for']
            
    if not pd.isnull(df.loc[i, 'S_S']):
        df.loc[i+.7,cols]=df.loc[i,cols]
        df.loc[i+.7, 'Nappe']='Socle sain'
        df.loc[i+.7, 'Litho_top']=df.loc[i, 'Soc_sn_top']
        df.loc[i+.7, 'Litho_base']=df.loc[i, 'Long_for']

df.drop(columns=['RB', 'ALL', 'S_A', 'S_S', 'Rb_base','All_top', 'Soc_alt_top','Soc_sn_top'], inplace=True)
df.sort_index(inplace=True)
df.reset_index(drop=True, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/tmp/ipykernel_27886/1801994457.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i+.2,cols]=df.loc[i,cols]


In [428]:
dataframe_viewer(df, rows=5, cols=15)

Rows : 51, columns : 9


interactive(children=(IntSlider(value=5, description='rows', max=51, min=5, readout=False), IntSlider(value=9,…

In [429]:
df.columns

Index(['ID', 'X', 'Y', 'Z', 'Refus', 'Long_for', 'Nappe', 'Litho_top',
       'Litho_base'],
      dtype='object')

In [430]:
bh=df[['ID','X','Y','Z','Long_for','Refus']]
bh['Type']='Forage'

litho=df[['ID','X','Y','Z','Litho_top','Litho_base','Nappe']]
source_litho=litho

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [431]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

bh.to_csv(tmp_dir+sheet+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
#source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:59 ; source_litho:51 ; source_Fac-uknw:0 ; source_an:104 ;
source_prv_sol:59 ;source_prv_eau:45 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Sond2017v2'**

In [432]:
tmp_dir='../../CF_data/Result_traitem/vUmons_logsFor/'
sheet='Sond2017v2'

In [433]:
df = pd.read_excel('../../CF_data/Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', 
                   sheet_name='Sond2017v2', skiprows=0)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

Rows : 71, columns : 18


interactive(children=(IntSlider(value=5, description='rows', max=71, min=5, readout=False), IntSlider(value=12…

In [434]:
df.replace('\*','', inplace=True, regex=True)
df['Refus']=df['Refus'].apply(lambda x: 'x' if x==1 else '')

In [435]:
name=['R_ID','ID','X','Y','Z','Refus','Date_ouv','Long_for','Z_fond','RB','ALL', 'S_A', 'S_S', 
      'Rb_base','cote_rb','All_top', 'Soc_alt_top','Soc_sn_top']
df=col_ren(df, mode=1, name=name)
df=df[['ID','X','Y','Z','Refus','Date_ouv','Long_for','Z_fond','RB','ALL', 'S_A', 'S_S', 
      'Rb_base','All_top', 'Soc_alt_top','Soc_sn_top']]

In [436]:
cols=['ID','Date_ouv','X','Y','Z','Z_fond','Refus','Long_for']

for i in range(len(df)):    
    if df.loc[i, 'RB']==1: 
        df.loc[i, 'Nappe']='Remblais'
        df.loc[i, 'Litho_top']=0
        
        if not pd.isnull(df.loc[i, 'Rb_base']): df.loc[i, 'Litho_base']=df.loc[i, 'Rb_base']
        else: df.loc[i, 'Litho_base']=df.loc[i, 'Long_for']
    
    val_def=df.loc[i, 'Litho_base'] # temporary value of litho_base if nan
    
    if df.loc[i, 'ALL']==1:
        df.loc[i+.2,cols]=df.loc[i,cols]
        df.loc[i+.2, 'Nappe']='Alluvions'
        
        if not pd.isnull(df.loc[i, 'All_top']): df.loc[i+.2, 'Litho_top']=df.loc[i, 'All_top']
        else: df.loc[i+.2, 'Litho_top']=val_def #df.loc[i, 'litho_base']
            
        if df.loc[i, 'S_A']==1: df.loc[i+.2, 'Litho_base']=df.loc[i, 'Soc_alt_top']
        else: df.loc[i+.2, 'Litho_base']=df.loc[i, 'Long_for']
    
    if df.loc[i, 'S_A']==1:
        df.loc[i+.5,cols]=df.loc[i,cols]
        df.loc[i+.5, 'Nappe']='Socle altéré'
        
        if not pd.isnull(df.loc[i, 'Soc_alt_top']): df.loc[i+.5, 'Litho_top']=df.loc[i, 'Soc_alt_top']
        else: df.loc[i+.5, 'Litho_top']=val_def #df.loc[i+.2, 'litho_base']
        
        if df.loc[i, 'S_S']==1: df.loc[i+.5, 'Litho_base']=df.loc[i, 'Soc_sn_top']
        else: df.loc[i+.5, 'Litho_base']=df.loc[i, 'Long_for']
            
    if df.loc[i, 'S_S']==1:
        df.loc[i+.7,cols]=df.loc[i,cols]
        df.loc[i+.7, 'Nappe']='Socle sain'
        df.loc[i+.7, 'Litho_top']=df.loc[i, 'Soc_sn_top']
        df.loc[i+.7, 'Litho_base']=df.loc[i, 'Long_for']

df.drop(columns=['RB', 'ALL', 'S_A', 'S_S','Rb_base','All_top', 'Soc_alt_top','Soc_sn_top'], inplace=True)
df.sort_index(inplace=True)
df.reset_index(drop=True, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [437]:
df=df[:-1]

In [438]:
dataframe_viewer(df, rows=5, cols=15)

Rows : 109, columns : 11


interactive(children=(IntSlider(value=5, description='rows', max=109, min=5, readout=False), IntSlider(value=1…

In [439]:
bh=df[['ID','X','Y','Z','Z_fond','Date_ouv','Long_for','Refus']]
bh['Type']='Forage'

litho=df[['ID','X','Y','Z','Litho_top','Litho_base','Nappe']]
source_litho = source_litho.merge(df, 'outer')

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [440]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

bh.to_csv(tmp_dir+sheet+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
#source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:59 ; source_litho:119 ; source_Fac-uknw:0 ; source_an:104 ;
source_prv_sol:59 ;source_prv_eau:45 ; source_mes_pz:0 ; source_mes_sol:0 ;


# Processing for new data added - April 2021

## 15-Profils de sol et données de terrain 2019.xlsx
* **Sheet : 'Log'**

In [441]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [442]:
tmp_dir='../../CF_data/Result_traitem/donnees_terrain_2019/'
sheet='Log'

In [443]:
df = pd.read_excel('../../CF_data/Data_UMONS/Profils_de_sol_Siterem- extension_pilote_et_pilote/'
                   'Profils de sol et donnees de terrain 2019.xlsx', 
                   sheet_name='Log', skiprows=0)
df=na_line_drop(df,0)
df=na_col_drop(df,2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)


Columns dropped :['Unnamed: 5']

Rows : 98, columns : 5


interactive(children=(IntSlider(value=5, description='rows', max=98, min=5, readout=False), IntSlider(value=5,…

In [444]:
name = ['ID','Litho_top', 'Litho_base', 'Keyword', 'Description']
df = col_ren(df, name=name, mode=1, )
df = df[1:]
df['Date_ouv'] = dtm.datetime(2019,12,18)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [445]:
df.drop(index=df.query('Litho_base.isnull() or Litho_top.isnull()').index, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [446]:
compute_BH_length(df)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [447]:
df.query('Litho_base.isnull() or Litho_top.isnull()')

,ID,Profondeur,Litho_top,Litho_base,Keyword,Description,Date_ouv


In [448]:
dataframe_viewer(df, rows=5, cols=15)

Rows : 93, columns : 7


interactive(children=(IntSlider(value=5, description='rows', max=93, min=5, readout=False), IntSlider(value=7,…

In [449]:
bh = df[1:62]
pza = df[65:80] #piezair
pz = df[83:]

In [450]:
bh.reset_index(drop=True, inplace=True)
pza.reset_index(drop=True, inplace=True)
pz.reset_index(drop=True, inplace=True)

In [451]:
bh.insert(1,'Type', 'Forage')
bh.insert(1,'Zone', 'Extension Pilote')
pza.insert(1,'Type', 'Piezair')
pza.insert(1,'Zone', 'Extension Pilote')
pz.insert(1,'Type', 'Piezo')
pz.insert(1,'Zone', 'Mini-Pilote')

In [452]:
litho=bh.append(pza)
litho=litho.append(pz)
litho=litho[['ID','Type','Zone','Litho_top','Litho_base','Description','Keyword']]
litho.reset_index(inplace=True, drop=True)

In [453]:
litho.loc[litho[litho['ID']==50].index, 'Type'] = 'Piezo'

In [454]:
dataframe_viewer(litho, rows=3)

Rows : 86, columns : 7


interactive(children=(IntSlider(value=3, description='rows', max=86, min=3, readout=False), IntSlider(value=7,…

In [455]:
source_litho=litho

In [456]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#pza.to_csv(tmp_dir+sheet+'_Piezairs.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
#source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:86 ; source_Fac-uknw:0 ; source_an:0 ;
source_prv_sol:0 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Echantillon'+'Organoleptique**

In [457]:
tmp_dir='../../CF_data/Result_traitem/donnees_terrain_2019/'
sheet='Echantillon'

In [458]:
df = pd.read_excel('../../CF_data/Data_UMONS/Profils_de_sol_Siterem- extension_pilote_et_pilote/'
                   'Profils de sol et donnees de terrain 2019.xlsx', 
                   sheet_name='Echantillon', skiprows=1)
df=na_line_drop(df,0)
df=na_col_drop(df,2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


16 NaN lines dropped

Columns dropped :['Unnamed: 4']

Rows : 67, columns : 4


interactive(children=(IntSlider(value=5, description='rows', max=67, min=5, readout=False), IntSlider(value=4,…

In [459]:
name=['ID','Ech_top', 'Ech_base', 'ID_ech']
df=col_ren(df, name=name, mode=1)
df.insert(1,'Type_ech','Sol')

In [460]:
df.drop(index=[43,44,55,56,66], inplace=True)
df.reset_index(drop=True, inplace=True)

In [461]:
ech=df.copy()

In [462]:
df = pd.read_excel('../../CF_data/Data_UMONS/Profils_de_sol_Siterem- extension_pilote_et_pilote/'
                   'Profils de sol et donnees de terrain 2019.xlsx', 
                   sheet_name='Organoleptique', skiprows=1)
df=na_line_drop(df,0)
df=na_col_drop(df,4)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)


Columns dropped :['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']

Rows : 20, columns : 5


interactive(children=(IntSlider(value=5, description='rows', max=20, min=5, readout=False), IntSlider(value=5,…

In [463]:
name=['ID','Pol_top', 'Pol_base','Polluant','Intensite']
df=col_ren(df, name=name, mode=1)

In [464]:
df.drop(index=[10,11,14,15], inplace=True)
df.reset_index(drop=True, inplace=True)

In [465]:
mdf, error_df =data_merger(ech, df, on='ID', how='outer')

In [466]:
dataframe_viewer(mdf)

Rows : 70, columns : 9


interactive(children=(IntSlider(value=10, description='rows', max=70, min=10, readout=False), IntSlider(value=…

In [467]:
prv_sol=mdf
source_prv_sol=prv_sol

In [468]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
#source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:86 ; source_Fac-uknw:0 ; source_an:0 ;
source_prv_sol:70 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Données de forage'**

In [469]:
tmp_dir='../../CF_data/Result_traitem/donnees_terrain_2019/'
sheet='Donnees_forage'

In [470]:
df = pd.read_excel('../../CF_data/Data_UMONS/Profils_de_sol_Siterem- extension_pilote_et_pilote/'
                   'Profils de sol et donnees de terrain 2019.xlsx', 
                   sheet_name='Données de forage', skiprows=1)
df=na_line_drop(df,0)
df=na_col_drop(df,2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)


Columns dropped :['RAS', 'RAS.1', 'Niv. Eau p/r sol', 'RAS.2', 'Unnamed: 18', 'PZ Prof.\nmesurée']

Rows : 27, columns : 14


interactive(children=(IntSlider(value=5, description='rows', max=27, min=5, readout=False), IntSlider(value=12…

In [471]:
name=['ID', 'X', 'Y', 'Z', 'Date_ouv', 'Long_for', 'Methode', 'Diam_for','Rmq', 'Long_pz', 'Diam_pz', 
      'Crep_long','Societe', 'Resp_chantier']
df=col_ren(df, name=name, mode=1)
df.drop(index=[16,23], inplace=True)
df.reset_index(drop=True, inplace=True)

In [472]:
df.insert(5, 'Type', '')
df.loc[:15,'Type']='Forage'
df.loc[16:21,'Type']='Piezair'
df.loc[22:,'Type']='Piezo'

In [473]:
df.loc[9,'ID']='224 bis'

In [474]:
df['Refus'] = ''
df['Type_refus']=''

for i in range(len(df['Rmq'])):
    val = str(df.loc[i,'Rmq'])
    if re.search('[Bb]loqué', val) :
        df.loc[i,'Refus'] = 'x'
        
        if re.search('[lL]aitier', val):
            df.loc[i,'Type_refus'] = 'Laitier'
        elif re.search('[Bb]éton', val):
            df.loc[i,'Type_refus'] = 'Béton'
        elif re.search('[Mm]atériaux', val):
            df.loc[i,'Type_refus'] = 'Matériaux indurés' 
    else: 
        df.loc[i,'Refus'] = '' 

df['Diam_int_pz'] = df['Diam_pz'].apply(lambda x: pd.to_numeric(x.replace('mm','').split('x')[1]) if not pd.isnull(x) else x)
df['Diam_ext_pz'] = df['Diam_pz'].apply(lambda x: pd.to_numeric(x.replace('mm','').split('x')[0]) if not pd.isnull(x) else x)
df['Diam_for'] = df['Diam_for'].apply(lambda x: pd.to_numeric(x) if not pd.isnull(x) else x)

df.insert(10, 'Diam_ext_pz', df.pop('Diam_ext_pz')) # move to a specified position
df.insert(11, 'Diam_int_pz', df.pop('Diam_int_pz'))
df.drop(columns=['Rmq', 'Diam_pz'], axis=1, inplace=True)
df.drop(df.query("ID!=ID").index, inplace=True) # delete all ID='NaN' lines
df.reset_index(drop=True, inplace=True)

gen_id_dated(df,'ID','Date_ouv')  

Generation of ID-dated...
Using column ' Date_ouv ' in the (geo)dataframe !
Process ended, check the (geo)dataframe


In [475]:
pz = df.query("Type=='Piezo'")
pza=df.query("Type=='Piezair'")
bh = df.query("Type=='Forage'")

pz.reset_index(inplace=True, drop=True)
pza.reset_index(inplace=True, drop=True)
bh.reset_index(inplace=True, drop=True)

In [476]:
dataframe_viewer(df, rows=3)

Rows : 25, columns : 18


interactive(children=(IntSlider(value=3, description='rows', max=25, min=3, readout=False), IntSlider(value=12…

In [477]:
source_pz = pz
source_pza = pza
source_bh = bh

In [478]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

bh.to_csv(tmp_dir+sheet+'_Boreholes.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
pza.to_csv(tmp_dir+sheet+'_Piezairs.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
source_pza.to_csv(tmp_dir+'source_Piezairs.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
#source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)}; source_pza:{len(source_pza)} ;'
      f'source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:16 ; source_pz:3; source_pza:6 ;source_litho:86 ; source_Fac-uknw:0 ; source_an:0 ;
source_prv_sol:70 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Equipement'**

In [479]:
tmp_dir='../../CF_data/Result_traitem/donnees_terrain_2019/'
sheet='Equipement'

In [480]:
df = pd.read_excel('../../CF_data/Data_UMONS/Profils_de_sol_Siterem- extension_pilote_et_pilote/'
                   'Profils de sol et donnees de terrain 2019.xlsx', 
                   sheet_name='Equipement', skiprows=1)
df=na_line_drop(df,0)
df=na_col_drop(df,2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

Rows : 35, columns : 7


interactive(children=(IntSlider(value=5, description='rows', max=35, min=5, readout=False), IntSlider(value=7,…

In [481]:
df.drop(columns=['Déplacement'], inplace=True)
name=['ID','Equip_top', 'Equip_base', 'Diam_for', 'Diam_ext_pz', 'Legende']
df=col_ren(df, mode=1, name=name)

In [482]:
df.drop(index=[24,25], inplace=True)
df.reset_index(drop=True, inplace=True)

In [483]:
compute_BH_length(df, top_col='Equip_top', base_col='Equip_base')

In [484]:
coi = ['ID', 'Profondeur', 'Diam_for', 'Diam_ext_pz']
pz=df[coi].drop_duplicates(['ID'])
pz['Type'] = 'Piezo'

In [485]:
dataframe_viewer(df)

Rows : 33, columns : 7


interactive(children=(IntSlider(value=10, description='rows', max=33, min=10, readout=False), IntSlider(value=…

In [486]:
equip=df
source_equip=equip

In [487]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)
equip.to_csv(tmp_dir+sheet+'_Equipment.csv', index=False)

#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
#source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)
source_equip.to_csv(tmp_dir+'source_Equipment.csv', index=False)

print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:16 ; source_pz:3 ; source_litho:86 ; source_Fac-uknw:0 ; source_an:0 ;
source_prv_sol:70 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'Piézométrie'**

In [488]:
tmp_dir='../../CF_data/Result_traitem/donnees_terrain_2019/'
sheet='piezometrie'

In [489]:
df = pd.read_excel('../../CF_data/Data_UMONS/Profils_de_sol_Siterem- extension_pilote_et_pilote/'
                   'Profils de sol et donnees de terrain 2019.xlsx', 
                   sheet_name='Piézométrie', skiprows=1)
df=na_line_drop(df,0)
df=na_col_drop(df,2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

Rows : 3, columns : 4


interactive(children=(IntSlider(value=3, description='rows', max=3, min=3, readout=False), IntSlider(value=4, …

In [490]:
name=['ID','Niv_pz_sol', 'Type_ech', 'Date_mes']
df=col_ren(df, name=name, mode=1)

In [491]:
mes_pz=df
source_mes_pz=mes_pz

In [492]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
#an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
#source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:16 ; source_pz:3 ; source_litho:86 ; source_Fac-uknw:0 ; source_an:0 ;
source_prv_sol:70 ;source_prv_eau:0 ; source_mes_pz:3 ; source_mes_sol:0 ;


In [493]:
dataframe_viewer(source_prv_sol, rows=5)

Rows : 70, columns : 9


interactive(children=(IntSlider(value=5, description='rows', max=70, min=5, readout=False), IntSlider(value=9,…

In [494]:
dataframe_viewer(excel_bhs, rows=5)

Rows : 25, columns : 23


interactive(children=(IntSlider(value=5, description='rows', max=25, min=5, readout=False), IntSlider(value=12…

### $\color{red}{\textbf{Excel sheets data merging}}$

In [495]:
excel_bhs, conflict_df = data_merger(source_bh, source_pz, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [496]:
excel_bhs, conflict_df = data_merger(excel_bhs, source_pza, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [497]:
excel_bh_litho, conflict_df = data_merger(excel_bhs, source_litho, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [498]:
excel_bh_soil_sp, conflict_df = data_merger(excel_bhs, source_prv_sol, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [499]:
excel_bh_equip, conflict_df = data_merger(excel_bhs, source_equip, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

Conflict values present. Please resolve this manually !


In [500]:
data_validation(overall_data=excel_bh_equip, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Diam_ext_pz_x':list(conflict_df.index)})

all conflicts have been fixed!


In [501]:
excel_bh_mes, conflict_df = data_merger(excel_bhs, source_mes_pz, how='outer', on='ID', dist_max=1., drop_skip_col=['index'])

In [502]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
#excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
#excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)

## 16-Résultats SOL extension pilote et piézairs.xlsx
* **Sheet : 'Résult SOL'**

In [503]:
# New file, so the source variables must be overwritten !!
_df=pd.DataFrame()
source_mes_pz, source_mes_sol, source_pz, source_prv_eau, source_prv_sol =  _df, _df, _df, _df, _df
source_ouv, source_an, source_litho, source_bh = _df, _df, _df, _df

In [504]:
tmp_dir='../../CF_data/Result_traitem/result_sol_ext_pilote/'
sheet='Result_Sol'

In [505]:
df = pd.read_excel('../../CF_data/Data_UMONS/Profils_de_sol_Siterem- extension_pilote_et_pilote/'
                   'Resultats SOL extension pilote et piezairs.xlsx', 
                   sheet_name='Résult SOL', skiprows=5)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

3 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67']

Rows : 135, columns : 60


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=135, min=5, readout=False), IntSlider(value=1…

In [506]:
prv_sol=df.loc[:35]
an=df.loc[36:]

In [507]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [508]:
prv_sol=prv_sol.transpose()
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=col_ren(prv_sol, 1)

In [509]:
prv_sol=dble_col_drop(prv_sol)

column(s) dropped: ['9:Autre zone suspecte investiguée', '27:température pour mes. pH', '30:pH (H20)']


In [510]:
prv_sol.drop(list(range(3)), axis=0, inplace=True)
prv_sol.reset_index(drop=True, inplace=True)
prv_sol=na_col_drop(prv_sol,3)
prv_sol=na_line_drop(prv_sol,3)
prv_sol.reset_index(drop=True, inplace=True)


Columns dropped :['Matières organiques', 'SPP/zone suspecte investiguée', 'Autre zone suspecte investiguée', 'Parcelle', 'X Lambert', 'Y Lambert', "Type d'étude (*)", 'Organoleptique couleur suspecte', 'Organoleptique odeur intensité (***)', 'Organoleptique odeur type', 'MO et COT', 'pH', 'GRANULOMETRIE', 'Fraction argileuse']



In [511]:
prv_sol=prv_sol[:-1]
prv_sol.drop(columns=['broyage'], inplace=True)

In [512]:
name=['ID_ech','Ech_top', 'Ech_base','MS','Date_prv','Long_for','Refus','Description','MO','COT','pH_KCl', 
      'Temp_pH_mes','pH_H20','Fract_2','Fract_2+', 'Fract_min_2µ','Fract_min_50µ','Fract_min_2']
prv_sol=col_ren(prv_sol, name=name, mode=1)

In [513]:
set(prv_sol.Description)

{'R', 'R ', 'TN', 'TN ', nan}

In [514]:
for i in range(len(prv_sol['Description'])):
    x = prv_sol.loc[i,'Description']
    if x in ['R','R ']: prv_sol.loc[i,'Description']='Remblais'
    elif x in ['TN','TN ']: prv_sol.loc[i,'Description']='Terrain naturel'

prv_sol['Refus']=prv_sol['Refus'].apply(lambda x: 'x' if not pd.isnull(x) else '')
prv_sol.insert(1,'Type_ech','Sol')#

In [515]:
for i in range(len(prv_sol)):
    x=prv_sol.loc[i,'ID_ech']
    r=re.search('([\w|\d]+)\n.+$', x)
    if r: 
        prv_sol.loc[i,'ID_ech']='226/'+r.group(1) # Rename borehole 304 to 226 because of conflict with piezair

In [516]:
dataframe_viewer(prv_sol, rows=3)

Rows : 55, columns : 19


interactive(children=(IntSlider(value=3, description='rows', max=55, min=3, readout=False), IntSlider(value=12…

In [517]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [518]:
an=col_ren(an, 1)

In [519]:
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [520]:
an.columns

Index(['ID_ech', 'METAUX LOURDS', 'Arsenic', 'Cadmium', 'Chrome', 'Chrome VI',
       'Cobalt', 'Cuivre', 'Mercure', 'Plomb', 'Nickel', 'Zinc', 'CYANURES',
       'cyanure (libre)', 'cyanure (totaux)', 'cyanure (APE)',
       'cyanure complex', 'thiocyanate', 'COMPOSES AROMATIQUES VOLATILS',
       'Benzène', 'Toluène', 'Éthylbenzène', 'Orthoxylène',
       'Para- et métaxylène', 'Xylènes', 'Styrène', 'BTEX totaux', 'PHENOLS',
       'Phénol', 'Indice phénol', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES',
       'Naphtalène', 'Acénaphtylène', 'Acénaphtène', 'Fluorène', 'col_35',
       'Anthracène', 'Fluoranthène', 'Pyrène', 'Benzo(a)anthracène',
       'Chrysène', 'Benzo(b)fluoranthène', 'Benzo(k)fluoranthène',
       'Benzo(a)pyrène', 'Dibenzo(ah)anthracène', 'Benzo(ghi)pérylène',
       'Indéno(1,2,3-cd)pyrène', 'HAP Totaux (16) - EPA',
       'COMPOSES ORGANOHALOGENES VOLATILS', 'Tétrachloroéthylène',
       'Trichloroéthylène', '1,1-dichloroéthène', 'Cis-1,2-dichloroéthène',
       '

In [521]:
an=an[an.columns[:-17]]
an.rename(columns={'col_35':'Phénanthrène'}, inplace=True)

In [522]:
an=col_ren(an, name=pol_field_model, mode=1)

In [523]:
an=dble_col_drop(an)

column(s) dropped: []


In [524]:
an.drop(list(range(3)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,1)
an = na_line_drop(an, 1)
an.insert(1,'Type_ech','Sol')


Columns dropped :['METAUX LOURDS', 'CYANURES', 'COMPOSES AROMATIQUES VOLATILS', 'BTEX_tot', 'PHENOLS', 'HYDROCARBURES AROMATIQUES POLYCYCLIQUES', 'COMPOSES ORGANOHALOGENES VOLATILS', 'HYDROCARBURES TOTAUX', 'METHYL-TERT-BUTYL-ETHER']

1 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [525]:
data = an
for i in range(len(data)):
    x=data.loc[i,'ID_ech']
    r=re.search('([\w|\d]+)\n.+$', x)
    if r: 
        data.loc[i,'ID_ech']='226/'+r.group(1) # Rename borehole 304 to 226 because of conflict with piezair

In [526]:
dataframe_viewer(an, rows=5) 

Rows : 55, columns : 75


interactive(children=(IntSlider(value=5, description='rows', max=55, min=5, readout=False), IntSlider(value=12…

In [527]:
source_prv_sol=prv_sol
source_an=an

In [528]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:55 ;
source_prv_sol:55 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


* **Sheet : 'inorganiques et composés majeur'**

In [529]:
tmp_dir='../../CF_data/Result_traitem/donnees_terrain_2019/'
sheet='Inorg_comp_majeur'

In [530]:
df = pd.read_excel('../../CF_data/Data_UMONS/Profils_de_sol_Siterem- extension_pilote_et_pilote/'
                   'Resultats SOL extension pilote et piezairs.xlsx', 
                   sheet_name='inorganiques et composés majeur', skiprows=1)
df=na_line_drop(df,0)
df=na_col_drop(df,1)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

dataframe_viewer(df, rows=5)

5 NaN lines dropped

Columns dropped :['Unnamed: 0', 'Unnamed: 2']

Rows : 64, columns : 46


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=5, description='rows', max=64, min=5, readout=False), IntSlider(value=12…

In [531]:
prv_sol=df.loc[:20] # not really interesting here!
an=df.loc[21:]

In [532]:
an.loc[0.5] = df.loc[0] # put data on first line
an = an.sort_index().reset_index(drop=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [533]:
an=an.transpose()
an.reset_index(drop=True, inplace=True)

In [534]:
an=col_ren(an, 1)

In [535]:
an.rename(columns={an.columns[0]:'ID_ech'}, inplace=True)

In [536]:
an=an[an.columns[:-7]]

In [537]:
an=col_ren(an, name=pol_field_model, mode=1)

In [538]:
an=dble_col_drop(an)

column(s) dropped: ['3:NH4', '4:NH4', '5:NH4', '9:HNO2', '10:HNO2', '11:HNO2', '13:HNO3', '14:HNO3', '15:HNO3']


In [539]:
an.drop(list(range(3)), axis=0, inplace=True)
an.reset_index(drop=True, inplace=True)
an=na_col_drop(an,2)
an = na_line_drop(an,1)
an.insert(1,'Type_ech','Sol')


Columns dropped :['COMPOSES AZOTES', 'NH3_libre', 'HNO2', 'HNO3', 'COMPOSES SOUFRES ', 'Sulfure_tot', 'sulfures (libre)', 'S_tot', 'sulfites', 'ELEMENTS MAJEURS', 'Fe_tot', 'Fe2', 'AUTRES ANALYSES', 'fluorures', 'cyanure (libre)', 'B_libre', 'CaCO3', 'Bicarb', 'METHYL-TERT-BUTYL-ETHER', 'MTBE']

35 NaN lines dropped


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [540]:
data = an
for i in range(len(data)):
    x=data.loc[i,'ID_ech']
    r=re.search('([\w|\d]+)\n.+$', x)
    if r: 
        data.loc[i,'ID_ech']='226/'+r.group(1) # Rename borehole 304 to 226 because of conflict with piezair

In [541]:
#source_prv_sol=prv_sol
source_an=data_merger(source_an, an, how='outer', on='ID_ech')[0]

In [542]:
dataframe_viewer(source_an, rows=5) 

Rows : 55, columns : 88


interactive(children=(IntSlider(value=5, description='rows', max=55, min=5, readout=False), IntSlider(value=12…

In [543]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

#bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#pz.to_csv(tmp_dir+sheet+'_Piezometers.csv', index=False)
#ouv.to_csv(tmp_dir+sheet+'_Unkown-facility.csv', index=False)
#litho.to_csv(tmp_dir+sheet+'_Lithology.csv', index=False)
#mes_pz.to_csv(tmp_dir+sheet+'_Measures.csv', index=False)
#mes_sol.to_csv(tmp_dir+sheet+'_Measures-soil.csv', index=False)
#prv_eau.to_csv(tmp_dir+sheet+'_Samples-water.csv', index=False)
#prv_sol.to_csv(tmp_dir+sheet+'_Samples-soil.csv', index=False)
an.to_csv(tmp_dir+sheet+'_Analysis.csv', index=False)


#source_bh.to_csv(tmp_dir+'source_Boreholes.csv', index=False)
#source_pz.to_csv(tmp_dir+'source_Piezometers.csv', index=False)
#source_ouv.to_csv(tmp_dir+'source_Unkown-facility.csv', index=False)
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False)
#source_mes_pz.to_csv(tmp_dir+'source_Measures.csv', index=False)
#source_mes_sol.to_csv(tmp_dir+'source_Measures-soil.csv', index=False)
#source_prv_eau.to_csv(tmp_dir+'source_Samples-water.csv', index=False)
#source_prv_sol.to_csv(tmp_dir+'source_Samples-soil.csv', index=False)
source_an.to_csv(tmp_dir+'source_Analysis.csv', index=False)


print(f'source_bh:{len(source_bh)} ; source_pz:{len(source_pz)} ; source_litho:{len(source_litho)} ; '
     f'source_Fac-uknw:{len(source_ouv)} ; source_an:{len(source_an)} ;\nsource_prv_sol:{len(source_prv_sol)} ;'
     f'source_prv_eau:{len(source_prv_eau)} ; source_mes_pz:{len(source_mes_pz)} ; source_mes_sol:{len(source_mes_sol)} ;')

source_bh:0 ; source_pz:0 ; source_litho:0 ; source_Fac-uknw:0 ; source_an:55 ;
source_prv_sol:55 ;source_prv_eau:0 ; source_mes_pz:0 ; source_mes_sol:0 ;


### $\color{red}{\textbf{merging of 'Donnees_2019' and 'result_sol' data}}$

In [544]:
excel_bh_soil_an, conflict_df = data_merger(excel_bh_soil_sp, source_an, how='outer', on='ID_ech', dist_max=1., drop_skip_col=['index'])

In [545]:
tmp_dir='../../CF_data/Result_traitem/donnees_terrain_2019/'

In [546]:
save_dir = tmp_dir + 'Final_merge/final_'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#excel_bhs.to_csv(save_dir + 'Boreholes.csv', index=False)
#excel_bh_litho.to_csv(save_dir+'Lithologies.csv', index=False)
#excel_bh_equip.to_csv(save_dir+'Equipments.csv', index=False)
#excel_bh_soil_sp.to_csv(save_dir+'Soil_samples.csv', index=False)
#excel_bh_water_sp.to_csv(save_dir+'Water_samples.csv', index=False)
excel_bh_soil_an.to_csv(save_dir+'Soil_analysis.csv', index=False)
#excel_bh_water_an.to_csv(save_dir+'water_analysis.csv', index=False)
#excel_bh_mes.to_csv(save_dir+'Measures.csv', index=False)
#excel_bh_fac.to_csv(save_dir+'Unknow_facilities.csv', index=False)